<a href="https://colab.research.google.com/github/ridwaniyas/petro_ai/blob/master/force_ml_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [116]:
cd '/content/drive/My Drive/FORCE Log 2020/Raw'

/content/drive/My Drive/FORCE Log 2020/Raw


In [3]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
import logging
import seaborn as sns
import pickle
from math import sqrt

import numpy as np
np.random.seed(0)
import pandas as pd
pd.set_option('display.max_columns', None)

import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier

from sklearn import preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error, f1_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV, LeaveOneGroupOut
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, Normalizer, PolynomialFeatures, LabelEncoder, OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier,  GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.multiclass import OneVsOneClassifier

import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Softmax, Flatten, Input, LSTM, Embedding, GRU, SimpleRNN
from keras.optimizers import SGD, Adam, RMSprop, Nadam, Adamax, Adagrad, Adadelta, Ftrl
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau

from scipy import stats
from scipy.stats import norm, skew
from scipy.signal import medfilt

import warnings
warnings.filterwarnings("ignore")

import time as tm

!pip3 install -U scikit-learn
import sklearn
sklearn.__version__

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.23.2)


'0.23.2'

In [4]:
df_train = pd.read_csv('./train.csv', sep=';')
df_train.head(2)

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,RDEP,RHOB,GR,SGR,NPHI,PEF,DTC,SP,BS,ROP,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE
0,15/9-13,494.528,437641.96875,6470972.5,-469.501831,NORDLAND GP.,NaN,19.480835,NaN,1.61141,1.798681,1.884186,80.200851,NaN,NaN,20.915468,161.13118,24.612379,NaN,34.63641,NaN,NaN,-0.574928,NaN,NaN,NaN,NaN,65000,1.0
1,15/9-13,494.680,437641.96875,6470972.5,-469.653809,NORDLAND GP.,NaN,19.468800,NaN,1.61807,1.795641,1.889794,79.262886,NaN,NaN,19.383013,160.60347,23.895531,NaN,34.63641,NaN,NaN,-0.570188,NaN,NaN,NaN,NaN,65000,1.0


In [ ]:
df_test = pd.read_csv('./test.csv', sep=';')
df_test.head(2)

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,RDEP,RHOB,GR,SGR,NPHI,PEF,DTC,SP,BS,ROP,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO
0,15/9-14,480.628001,423244.5,6461862.5,-455.624420,NORDLAND GP.,NaN,19.2031,NaN,1.613886,1.513937,2.171173,72.078377,NaN,0.551575,55.815659,174.347092,35.525719,NaN,96.461990,NaN,NaN,-0.538873,0.130611,NaN,NaN,NaN
1,15/9-14,480.780001,423244.5,6461862.5,-455.776428,NORDLAND GP.,NaN,19.2031,NaN,1.574376,1.480188,2.171840,72.147697,NaN,0.547566,41.862537,176.112915,36.158520,NaN,96.454399,NaN,NaN,-0.539232,0.130611,NaN,NaN,NaN


### Lithology Dummy

In [5]:
lithology_keys = {30000: 'Sandstone', 65030: 'Sandstone/Shale', 65000: 'Shale', 80000: 'Marl', 74000: 'Dolomite',
                 70000: 'Limestone', 70032: 'Chalk', 88000: 'Halite', 86000: 'Anhydrite', 99000: 'Tuff',
                 90000: 'Coal', 93000: 'Basement'}

In [6]:
lithology_num = {'Sandstone': 1, 'Sandstone/Shale': 2, 'Shale': 3, 'Marl': 4, 'Dolomite': 5, 'Limestone': 6,
                 'Chalk': 7, 'Halite': 8, 'Anhydrite': 9, 'Tuff': 10, 'Coal': 11, 'Basement': 12}

In [7]:
df_train['Lithology Keys'] = df_train['FORCE_2020_LITHOFACIES_LITHOLOGY'].replace(lithology_keys)
df_train['Lithology Num'] = df_train['Lithology Keys'].replace(lithology_num)
df_train['Confidence Level'] = df_train['FORCE_2020_LITHOFACIES_CONFIDENCE']

In [ ]:
#df_train_litho = pd.get_dummies(df_train['Lithology Keys'])
#df_train_litho.replace(0, np.nan, inplace=True)

In [8]:
#df_train = df_train.join(df_train_litho)
df_train = df_train[['WELL', 'DEPTH_MD', 'FORMATION',
                     'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 
                     'NPHI', 'SP', 'PEF', 
                     #'RMED', 'RDEP',
                     'FORCE_2020_LITHOFACIES_LITHOLOGY', 'FORCE_2020_LITHOFACIES_CONFIDENCE', 
                     'Lithology Keys', 'Lithology Num',
                     #'Sandstone', 'Sandstone/Shale', 'Shale', 'Marl', 'Dolomite', 'Limestone',
                     #'Chalk', 'Halite', 'Anhydrite', 'Tuff', 'Coal', 'Basement'
                     ]]
df_train.head(2)

,WELL,DEPTH_MD,FORMATION,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE,Lithology Keys,Lithology Num
0,15/9-13,494.528,NaN,80.200851,161.13118,19.480835,-0.574928,1.884186,NaN,24.612379,20.915468,65000,1.0,Shale,3
1,15/9-13,494.680,NaN,79.262886,160.60347,19.468800,-0.570188,1.889794,NaN,23.895531,19.383013,65000,1.0,Shale,3


In [ ]:
Umaa_1 = pd.DataFrame((df_train['PEF']*(((df_train['RHOB']+0.1883)/1.0704)-(df_train['NPHI']*1)))/1-df_train['NPHI'], 
                      columns=['Umaa_1'], index=df_train.index)

In [ ]:
Umaa_2 = pd.DataFrame((df_train['PEF']*(((df_train['DRHO']+0.1883)/1.0704)-(df_train['NPHI']*1)))/1-df_train['NPHI'], 
                      columns=['Umaa_2'], index=df_train.index)

In [ ]:
Cmaa_1 = pd.DataFrame((df_train['RHOB']-(df_train['NPHI']*1))/1-df_train['NPHI'],
                      columns=['Cmaa'], index=df_train.index)

In [ ]:
Cmaa_2 = pd.DataFrame((df_train['DRHO']-(df_train['NPHI']*1))/1-df_train['NPHI'],
                      columns=['Cmaa'], index=df_train.index)

In [ ]:
Aaa  = pd.DataFrame(df_train['DTC']-((df_train['NPHI']*df_train['DTC'])/0.68),
                    columns=['Aaa'], index=df_train.index)

## Imputation by WELL

In [ ]:
list(enumerate(df_train['WELL'].unique()))

In [ ]:
list(enumerate(df_test['WELL'].unique()))

## Imputation ALL Row

In [ ]:
df_train = df_train[['FORMATION', 'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 'NPHI', 'SP', 'PEF', 
                     #'RMED', 'RDEP'
                     ]]
df_train.head(2)

,FORMATION,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF,RMED,RDEP
0,NaN,80.200851,161.13118,19.480835,-0.574928,1.884186,NaN,24.612379,20.915468,1.61141,1.798681
1,NaN,79.262886,160.60347,19.468800,-0.570188,1.889794,NaN,23.895531,19.383013,1.61807,1.795641


In [9]:
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

In [10]:
imputer_all  = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=800, verbose=1, tol=0.00001)

In [ ]:
imputer_all.fit(df_train[['GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 'NPHI', 'SP', 'PEF']])

In [ ]:
all_trans  = imputer_all.transform(df_train[['GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 'NPHI', 'SP', 'PEF']])

[IterativeImputer] Completing matrix with shape (1170511, 8)


In [ ]:
df_all_trans  = pd.DataFrame(all_trans, columns=['GR_new', 'DTC_new', 'CALI_new', 'DRHO_new',
                                                 'RHOB_new', 'NPHI_new', 'SP_new', 'PEF_new'],
                             index=df_train.index)

In [ ]:
df_all_trans['Umaa_1'] = (df_all_trans['PEF_new']*(((df_all_trans['RHOB_new']+0.1883)/1.0704)-(df_all_trans['NPHI_new']*1)))/1-df_all_trans['NPHI_new']

In [ ]:
df_all_trans['Umaa_2'] = (df_all_trans['PEF_new']*(((df_all_trans['DRHO_new']+0.1883)/1.0704)-(df_all_trans['NPHI_new']*1)))/1-df_all_trans['NPHI_new']

In [ ]:
df_all_trans['Cmaa_1'] = (df_all_trans['RHOB_new']-(df_all_trans['NPHI_new']*1))/1-df_all_trans['NPHI_new']

In [ ]:
df_all_trans['Cmaa_2'] = (df_all_trans['DRHO_new']-(df_all_trans['NPHI_new']*1))/1-df_all_trans['NPHI_new']

In [ ]:
df_all_trans['Aaa']    = df_all_trans['DTC_new']-((df_all_trans['NPHI_new']*df_all_trans['DTC_new'])/0.68)

In [ ]:
sc = MinMaxScaler()
#sc = StandardScaler()

In [ ]:
sc_all  = MinMaxScaler().fit(df_all_trans)

In [ ]:
mm_cols = ['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm', 
           'NPHI_mm', 'SP_mm', 'PEF_mm',
           'Umaa_1_mm', 'Umaa_2_mm', 'Cmaa_1_mm', 'Cmaa_2_mm', 'Aaa_mm'
           #'RMED_mm', 'RDEP_mm'
           ]
#std_cols = ['GR_std', 'DTC_std', 'CALI_std', 'DRHO_std', 'RHOB_std', 'RMED_std', 'RDEP_std']

In [ ]:
all_mm  = pd.DataFrame(sc_all.transform(df_all_trans), df_all_trans.index, mm_cols)
all_mm.head(2)

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm,Umaa_1_mm,Umaa_2_mm,Cmaa_1_mm,Cmaa_2_mm,Aaa_mm
0,0.074375,0.007553,0.686896,0.997932,0.425020,0.00303,0.002411,0.114771,0.563287,0.973813,0.994038,0.997967,0.99998
1,0.073504,0.007527,0.686468,0.997932,0.427069,0.00301,0.002410,0.111026,0.562377,0.973864,0.994072,0.997969,0.99998


In [ ]:
df_all_trans_mm = df_all_trans.merge(all_mm, left_index=True, right_index=True, how='outer')
df_all_trans_mm.head(2)

,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,Umaa_1,Umaa_2,Cmaa_1,Cmaa_2,Aaa,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm,Umaa_1_mm,Umaa_2_mm,Cmaa_1_mm,Cmaa_2_mm,Aaa_mm
0,80.200851,161.13118,19.480835,-0.574928,1.884186,0.603659,24.612379,20.915468,27.266621,-20.784115,0.676868,-1.782245,18.089634,0.074375,0.007553,0.686896,0.997932,0.425020,0.00303,0.002411,0.114771,0.563287,0.973813,0.994038,0.997967,0.99998
1,79.262886,160.60347,19.468800,-0.570188,1.889794,0.599423,23.895531,19.383013,25.412482,-19.133353,0.690948,-1.769034,19.030849,0.073504,0.007527,0.686468,0.997932,0.427069,0.00301,0.002410,0.111026,0.562377,0.973864,0.994072,0.997969,0.99998


In [ ]:
df_train_all_mm = df_train.merge(df_all_trans_mm, left_index=True, right_index=True, how='outer')
df_train_all_mm.head(2)

,WELL,DEPTH_MD,FORMATION,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE,Lithology Keys,Lithology Num,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,Umaa_1,Umaa_2,Cmaa_1,Cmaa_2,Aaa,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm,Umaa_1_mm,Umaa_2_mm,Cmaa_1_mm,Cmaa_2_mm,Aaa_mm
0,15/9-13,494.528,NaN,80.200851,161.13118,19.480835,-0.574928,1.884186,NaN,24.612379,20.915468,65000,1.0,Shale,3,80.200851,161.13118,19.480835,-0.574928,1.884186,0.603659,24.612379,20.915468,27.266621,-20.784115,0.676868,-1.782245,18.089634,0.074375,0.007553,0.686896,0.997932,0.425020,0.00303,0.002411,0.114771,0.563287,0.973813,0.994038,0.997967,0.99998
1,15/9-13,494.680,NaN,79.262886,160.60347,19.468800,-0.570188,1.889794,NaN,23.895531,19.383013,65000,1.0,Shale,3,79.262886,160.60347,19.468800,-0.570188,1.889794,0.599423,23.895531,19.383013,25.412482,-19.133353,0.690948,-1.769034,19.030849,0.073504,0.007527,0.686468,0.997932,0.427069,0.00301,0.002410,0.111026,0.562377,0.973864,0.994072,0.997969,0.99998


In [ ]:
df_train_all_mm.info()

In [ ]:
df_train_all_mm.to_csv('../df_train_all_mm.csv')
#df_train_all_mm = pd.read_csv('../df_train_all_mm.csv', index_col=0)

### Training

In [ ]:
lithology_numbers = {30000: 0,           65030: 1,                 65000: 2,           80000: 3,      74000: 4,          70000: 5, 
                     70032: 6,           88000: 7,                 86000: 8,           99000: 9,      90000: 10,         93000: 11}
lithology_keys    = {30000: 'Sandstone', 65030: 'Sandstone/Shale', 65000: 'Shale',     80000: 'Marl', 74000: 'Dolomite', 70000: 'Limestone',
                     70032: 'Chalk',     88000: 'Halite',          86000: 'Anhydrite', 99000: 'Tuff', 90000: 'Coal',     93000: 'Basement'}
lithology_int     = {0: 'Sandstone',     1: 'Sandstone/Shale',     2: 'Shale',         3: 'Marl',     4: 'Dolomite',     5: 'Limestone',
                     6: 'Chalk',         7: 'Halite',              8: 'Anhydrite',     9: 'Tuff',     10: 'Coal',        11: 'Basement'}

In [ ]:
df_train_all_mm.columns

In [ ]:
df_X = df_train_all_mm[['GR_new',    'DTC_new',   'CALI_new',  'DRHO_new',  'RHOB_new', 'NPHI_new', 'SP_new', 'PEF_new', 
                        #'Umaa_1',    
                        #'Umaa_2',    
                        #'Cmaa_1',    
                        'Cmaa_2',    
                        'Aaa',
                        #'GR_mm',     'DTC_mm',    'CALI_mm',   'DRHO_mm',   'RHOB_mm',  'NPHI_mm',  'SP_mm',  'PEF_mm', 
                        #'Umaa_1_mm', 'Umaa_2_mm', 'Cmaa_1_mm', 'Cmaa_2_mm', 'Aaa_mm'
                        ]]
df_X.head(2)

,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,Cmaa_2,Aaa
0,80.200851,161.13118,19.480835,-0.574928,1.884186,0.603659,24.612379,20.915468,-1.782245,18.089634
1,79.262886,160.60347,19.468800,-0.570188,1.889794,0.599423,23.895531,19.383013,-1.769034,19.030849


In [ ]:
#df_y = pd.get_dummies(df_train_scaled['Lithology Keys'])
#df_y = df_train_scaled['FORCE_2020_LITHOFACIES_LITHOLOGY']

df_y = df_train_all_mm['FORCE_2020_LITHOFACIES_LITHOLOGY']
y_int = df_y.map(lithology_numbers)
y_str = df_y.map(lithology_keys)
y_str.value_counts()

Shale              720803
Sandstone          168937
Sandstone/Shale    150455
Limestone           56320
Marl                33329
Tuff                15245
Chalk               10513
Halite               8213
Coal                 3820
Dolomite             1688
Anhydrite            1085
Basement              103
Name: FORCE_2020_LITHOFACIES_LITHOLOGY, dtype: int64

In [ ]:
A = np.load('penalty_matrix.npy')

def score(y_true, y_pred):
    S = 0.0
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    for i in range(0, y_true.shape[0]):
        S -= A[y_true[i], y_pred[i]]
    return S/y_true.shape[0]

In [ ]:
X = df_X.copy()
#y = df_y.copy()
y = y_int.copy()

In [ ]:
# Split into training, validation, and test sets (80/20)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.0001, random_state=90, stratify=y)
#X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=2)

#print("Train set size: {0}, Val set size: {1}, Test set size: {2}".format(len(X_train), len(X_val), len(X_test)))
print("Train set size: {0}, Test set size: {1}".format(len(X_train), len(X_test)))

Train set size: 1170393, Test set size: 118


In [ ]:
X_train.head(2)

,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,Cmaa_2,Aaa
1060259,86.347878,122.818512,17.359400,-0.025004,2.217792,0.397004,181.695511,3.934099,-0.819013,51.113394
24528,81.131363,168.767715,12.562967,0.056305,1.978260,0.528013,66.578972,2.089088,-0.999721,37.721309


In [ ]:
# raw, all row; NPHI, SP, PEF; Umaa all
lgbmclass = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                           importance_type='split', learning_rate=0.1, max_depth=-1,
                           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                           n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                           subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model_1 = make_pipeline(lgbmclass)
model_1.fit(X_train, y_train)
y_pred_test = model_1.predict(X_test)

print('Model Score                   : ', model_1.score(X_train, y_train))
print('Root Mean Squared error       : ', sqrt(mean_squared_error(y_test.values, y_pred_test)))
print('Custom Score (penalty_matrix) : ', score(y_test.values, y_pred_test))
# Xeek score: 0.7446

Model Score                   :  0.812159676279677
Root Mean Squared error       :  1.5126584522688322
Custom Score (penalty_matrix) :  -0.7447033898305084


In [ ]:
# raw mm, all row; NPHI, SP, PEF; Umaa all
lgbmclass = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                           importance_type='split', learning_rate=0.1, max_depth=-1,
                           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                           n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                           subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model_1 = make_pipeline(lgbmclass)
model_1.fit(X_train, y_train)
y_pred_test = model_1.predict(X_test)

print('Model Score                   : ', model_1.score(X_train, y_train))
print('Root Mean Squared error       : ', sqrt(mean_squared_error(y_test.values, y_pred_test)))
print('Custom Score (penalty_matrix) : ', score(y_test.values, y_pred_test))
# Xeek score: 0.7545

Model Score                   :  0.8123655900197626
Root Mean Squared error       :  0.8972651479706051
Custom Score (penalty_matrix) :  -0.7129237288135594


In [ ]:
# mm, all row; NPHI, SP, PEF; Umaa_mm all
lgbmclass = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                           importance_type='split', learning_rate=0.1, max_depth=-1,
                           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                           n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                           subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model_1 = make_pipeline(lgbmclass)
model_1.fit(X_train, y_train)
y_pred_test = model_1.predict(X_test)

print('Model Score                   : ', model_1.score(X_train, y_train))
print('Root Mean Squared error       : ', sqrt(mean_squared_error(y_test.values, y_pred_test)))
print('Custom Score (penalty_matrix) : ', score(y_test.values, y_pred_test))
# Xeek score: 0.815

Model Score                   :  0.81019623323106
Root Mean Squared error       :  1.2178057511186369
Custom Score (penalty_matrix) :  -0.725635593220339


In [ ]:
# raw, all row; NPHI, SP, PEF;
lgbmclass = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                           importance_type='split', learning_rate=0.1, max_depth=-1,
                           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                           n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                           subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model_1 = make_pipeline(lgbmclass)
model_1.fit(X_train, y_train)
y_pred_test = model_1.predict(X_test)

print('Model Score                   : ', model_1.score(X_train, y_train))
print('Root Mean Squared error       : ', sqrt(mean_squared_error(y_test.values, y_pred_test)))
print('Custom Score (penalty_matrix) : ', score(y_test.values, y_pred_test))
# Xeek score: 0.7248

Model Score                   :  0.819388871943014
Root Mean Squared error       :  0.7972410051791008
Custom Score (penalty_matrix) :  -0.5889830508474576


In [ ]:
# raw, all row; NPHI, SP, PEF; cmaa-umaa 1
lgbmclass = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                           importance_type='split', learning_rate=0.1, max_depth=-1,
                           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                           n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                           subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model_1 = make_pipeline(lgbmclass)
model_1.fit(X_train, y_train)
y_pred_test = model_1.predict(X_test)

print('Model Score                   : ', model_1.score(X_train, y_train))
print('Root Mean Squared error       : ', sqrt(mean_squared_error(y_test.values, y_pred_test)))
print('Custom Score (penalty_matrix) : ', score(y_test.values, y_pred_test))
# Xeek score: 0.7507

Model Score                   :  0.8178654520319243
Root Mean Squared error       :  0.7865392983167531
Custom Score (penalty_matrix) :  -0.6101694915254238


In [ ]:
# raw, all row; NPHI, SP, PEF; cmaa-umaa 2
lgbmclass = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                           importance_type='split', learning_rate=0.1, max_depth=-1,
                           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                           n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                           subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model_1 = make_pipeline(lgbmclass)
model_1.fit(X_train, y_train)
y_pred_test = model_1.predict(X_test)

print('Model Score                   : ', model_1.score(X_train, y_train))
print('Root Mean Squared error       : ', sqrt(mean_squared_error(y_test.values, y_pred_test)))
print('Custom Score (penalty_matrix) : ', score(y_test.values, y_pred_test))
# Xeek score: 0.7405

Model Score                   :  0.8144554863195524
Root Mean Squared error       :  0.8182246179526309
Custom Score (penalty_matrix) :  -0.621822033898305


In [ ]:
# raw, all row; NPHI, SP, PEF; cmaa 1 - Aaa
lgbmclass = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                           importance_type='split', learning_rate=0.1, max_depth=-1,
                           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                           n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                           subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model_1 = make_pipeline(lgbmclass)
model_1.fit(X_train, y_train)
y_pred_test = model_1.predict(X_test)

print('Model Score                   : ', model_1.score(X_train, y_train))
print('Root Mean Squared error       : ', sqrt(mean_squared_error(y_test.values, y_pred_test)))
print('Custom Score (penalty_matrix) : ', score(y_test.values, y_pred_test))
# Xeek score: 0.7379

Model Score                   :  0.8106943565110181
Root Mean Squared error       :  1.0735652625161434
Custom Score (penalty_matrix) :  -0.6652542372881356


In [ ]:
# raw, all row; NPHI, SP, PEF; cmaa 2 - Aaa
lgbmclass = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                           importance_type='split', learning_rate=0.1, max_depth=-1,
                           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                           n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                           subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model_1 = make_pipeline(lgbmclass)
model_1.fit(X_train, y_train)
y_pred_test = model_1.predict(X_test)

print('Model Score                   : ', model_1.score(X_train, y_train))
print('Root Mean Squared error       : ', sqrt(mean_squared_error(y_test.values, y_pred_test)))
print('Custom Score (penalty_matrix) : ', score(y_test.values, y_pred_test))
# Xeek score: 0.7415

Model Score                   :  0.8019297791425615
Root Mean Squared error       :  1.089238577156585
Custom Score (penalty_matrix) :  -0.7341101694915254


### df_test ALL

In [ ]:
df_test = pd.read_csv('./test.csv', sep=';')
df_test = df_test[['WELL', 'DEPTH_MD', 'FORMATION',
                   'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 'NPHI', 'SP', 'PEF']]
df_test.head(2)

,WELL,DEPTH_MD,FORMATION,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF
0,15/9-14,480.628001,NaN,72.078377,174.347092,19.2031,-0.538873,2.171173,0.551575,35.525719,55.815659
1,15/9-14,480.780001,NaN,72.147697,176.112915,19.2031,-0.539232,2.171840,0.547566,36.158520,41.862537


In [ ]:
test_all_trans  = imputer_all.transform(df_test[['GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 'NPHI', 'SP', 'PEF']])

[IterativeImputer] Completing matrix with shape (136786, 8)


In [ ]:
test_df_all_trans  = pd.DataFrame(test_all_trans, columns=['GR_new', 'DTC_new', 'CALI_new', 'DRHO_new',
                                                           'RHOB_new', 'NPHI_new', 'SP_new', 'PEF_new'],
                                  index=df_test.index)

In [ ]:
test_df_all_trans['Umaa_1'] = (test_df_all_trans['PEF_new']*(((test_df_all_trans['RHOB_new']+0.1883)/1.0704)-(test_df_all_trans['NPHI_new']*1)))/1-test_df_all_trans['NPHI_new']

In [ ]:
test_df_all_trans['Umaa_2'] = (test_df_all_trans['PEF_new']*(((test_df_all_trans['DRHO_new']+0.1883)/1.0704)-(test_df_all_trans['NPHI_new']*1)))/1-test_df_all_trans['NPHI_new']

In [ ]:
test_df_all_trans['Cmaa_1'] = (test_df_all_trans['RHOB_new']-(test_df_all_trans['NPHI_new']*1))/1-test_df_all_trans['NPHI_new']

In [ ]:
test_df_all_trans['Cmaa_2'] = (test_df_all_trans['DRHO_new']-(test_df_all_trans['NPHI_new']*1))/1-test_df_all_trans['NPHI_new']

In [ ]:
test_df_all_trans['Aaa']    = test_df_all_trans['DTC_new']-((test_df_all_trans['NPHI_new']*test_df_all_trans['DTC_new'])/0.68)

In [ ]:
mm_cols = ['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm', 
           'NPHI_mm', 'SP_mm', 'PEF_mm',
           'Umaa_1_mm', 'Umaa_2_mm', 'Cmaa_1_mm', 'Cmaa_2_mm', 'Aaa_mm'
           #'RMED_mm', 'RDEP_mm'
           ]
#std_cols = ['GR_std', 'DTC_std', 'CALI_std', 'DRHO_std', 'RHOB_std', 'RMED_std', 'RDEP_std']

In [ ]:
test_all_mm  = pd.DataFrame(sc_all.transform(test_df_all_trans), test_df_all_trans.index, mm_cols)
test_all_mm.head(2)

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm,Umaa_1_mm,Umaa_2_mm,Cmaa_1_mm,Cmaa_2_mm,Aaa_mm
0,0.066833,0.008202,0.677012,0.997937,0.529880,0.002783,0.002437,0.200065,0.594925,0.972918,0.994964,0.997985,0.999982
1,0.066897,0.008289,0.677012,0.997937,0.530124,0.002764,0.002438,0.165964,0.583699,0.973304,0.994984,0.997986,0.999982


In [ ]:
test_df_all_trans_mm = test_df_all_trans.merge(test_all_mm, left_index=True, right_index=True, how='outer')
test_df_all_trans_mm.head(2)

,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,Umaa_1,Umaa_2,Cmaa_1,Cmaa_2,Aaa,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm,Umaa_1_mm,Umaa_2_mm,Cmaa_1_mm,Cmaa_2_mm,Aaa_mm
0,72.078377,174.347092,19.2031,-0.538873,2.171173,0.551575,35.525719,55.815659,91.695822,-49.618614,1.068022,-1.642023,32.927196,0.066833,0.008202,0.677012,0.997937,0.529880,0.002783,0.002437,0.200065,0.594925,0.972918,0.994964,0.997985,0.999982
1,72.147697,176.112915,19.2031,-0.539232,2.171840,0.547566,36.158520,41.862537,68.833219,-37.194772,1.076708,-1.634365,34.298981,0.066897,0.008289,0.677012,0.997937,0.530124,0.002764,0.002438,0.165964,0.583699,0.973304,0.994984,0.997986,0.999982


In [ ]:
df_test_all_mm = df_test.merge(test_df_all_trans_mm, left_index=True, right_index=True, how='outer')
df_test_all_mm.head(2)

,WELL,DEPTH_MD,FORMATION,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,Umaa_1,Umaa_2,Cmaa_1,Cmaa_2,Aaa,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm,Umaa_1_mm,Umaa_2_mm,Cmaa_1_mm,Cmaa_2_mm,Aaa_mm
0,15/9-14,480.628001,NaN,72.078377,174.347092,19.2031,-0.538873,2.171173,0.551575,35.525719,55.815659,72.078377,174.347092,19.2031,-0.538873,2.171173,0.551575,35.525719,55.815659,91.695822,-49.618614,1.068022,-1.642023,32.927196,0.066833,0.008202,0.677012,0.997937,0.529880,0.002783,0.002437,0.200065,0.594925,0.972918,0.994964,0.997985,0.999982
1,15/9-14,480.780001,NaN,72.147697,176.112915,19.2031,-0.539232,2.171840,0.547566,36.158520,41.862537,72.147697,176.112915,19.2031,-0.539232,2.171840,0.547566,36.158520,41.862537,68.833219,-37.194772,1.076708,-1.634365,34.298981,0.066897,0.008289,0.677012,0.997937,0.530124,0.002764,0.002438,0.165964,0.583699,0.973304,0.994984,0.997986,0.999982


In [ ]:
#df_test_all_mm.info()

In [ ]:
df_test_all_mm_predict = df_test_all_mm[['GR_new',    'DTC_new',   'CALI_new',  'DRHO_new',  'RHOB_new', 'NPHI_new', 'SP_new', 'PEF_new', 
                                         #'Umaa_1',    
                                         #'Umaa_2',    
                                         #'Cmaa_1',    
                                         'Cmaa_2',    
                                         'Aaa',
                                         #'GR_mm',     'DTC_mm',    'CALI_mm',   'DRHO_mm',   'RHOB_mm',  'NPHI_mm',  'SP_mm',  'PEF_mm', 
                                         #'Umaa_1_mm', 'Umaa_2_mm', 'Cmaa_1_mm', 'Cmaa_2_mm', 'Aaa_mm'
                                         ]]

In [ ]:
test_prediction = model_1.predict(df_test_all_mm_predict)
test_prediction

array([2, 2, 2, ..., 1, 1, 1])

In [ ]:
pd.Series(test_prediction).value_counts()

2     93292
0     22949
1     10309
5      5422
3      2434
10      736
6       500
9       356
8       255
4       241
7       167
11      125
dtype: int64

In [ ]:
category_to_lithology = {y:x for x,y in lithology_numbers.items()}
test_prediction_for_submission = np.vectorize(category_to_lithology.get)(test_prediction)
test_prediction_for_submission

array([65000, 65000, 65000, ..., 65030, 65030, 65030])

In [ ]:
np.savetxt('../Submission/test_predictions.csv', test_prediction_for_submission, header='lithology', fmt='%i')

In [ ]:
test_predictions = pd.read_csv('../Submission/test_predictions.csv')
test_predictions.columns = ['lithology']
test_predictions.head(2)

,lithology
0,65000
1,65000


In [ ]:
test_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136786 entries, 0 to 136785
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   lithology  136786 non-null  int64
dtypes: int64(1)
memory usage: 1.0 MB


## Imputation by LITHOLOGY

### df_train FORMATION based on df_test

In [ ]:
df_train = df_train[['Lithology Keys', 'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 
                     'NPHI', 'SP', 'PEF', 
                     #'RMED', 'RDEP'
                     ]]
df_train.head(2)

,Lithology Keys,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF
0,Shale,80.200851,161.13118,19.480835,-0.574928,1.884186,NaN,24.612379,20.915468
1,Shale,79.262886,160.60347,19.468800,-0.570188,1.889794,NaN,23.895531,19.383013


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1170511 entries, 0 to 1170510
Data columns (total 11 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   Lithology Keys  1170511 non-null  object 
 1   GR              1170511 non-null  float64
 2   DTC             1089648 non-null  float64
 3   CALI            1082634 non-null  float64
 4   DRHO            987857 non-null   float64
 5   RHOB            1009242 non-null  float64
 6   NPHI            765409 non-null   float64
 7   SP              864247 non-null   float64
 8   PEF             671692 non-null   float64
 9   RMED            1131518 non-null  float64
 10  RDEP            1159496 non-null  float64
dtypes: float64(10), object(1)
memory usage: 98.2+ MB


In [ ]:
litho = df_train['Lithology Keys'].unique()

In [ ]:
litho_0  = df_train[df_train['Lithology Keys'] == litho[0]]
litho_1  = df_train[df_train['Lithology Keys'] == litho[1]]
litho_2  = df_train[df_train['Lithology Keys'] == litho[2]]
litho_3  = df_train[df_train['Lithology Keys'] == litho[3]]
litho_4  = df_train[df_train['Lithology Keys'] == litho[4]]
litho_5  = df_train[df_train['Lithology Keys'] == litho[5]]
litho_6  = df_train[df_train['Lithology Keys'] == litho[6]]
litho_7  = df_train[df_train['Lithology Keys'] == litho[7]]
litho_8  = df_train[df_train['Lithology Keys'] == litho[8]]
litho_9  = df_train[df_train['Lithology Keys'] == litho[9]]
litho_10 = df_train[df_train['Lithology Keys'] == litho[10]]
litho_11 = df_train[df_train['Lithology Keys'] == litho[11]]

In [ ]:
litho_0  = litho_0[['Lithology Keys', 'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 'NPHI', 'SP', 'PEF']]
litho_1  = litho_0[['Lithology Keys', 'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 'NPHI', 'SP', 'PEF']]
litho_2  = litho_0[['Lithology Keys', 'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 'NPHI', 'SP', 'PEF']]
litho_3  = litho_0[['Lithology Keys', 'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 'NPHI', 'SP', 'PEF']]
litho_4  = litho_0[['Lithology Keys', 'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 'NPHI', 'SP', 'PEF']]
litho_5  = litho_0[['Lithology Keys', 'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 'NPHI', 'SP', 'PEF']]
litho_6  = litho_0[['Lithology Keys', 'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 'NPHI', 'SP', 'PEF']]
litho_7  = litho_0[['Lithology Keys', 'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 'NPHI', 'SP', 'PEF']]
litho_8  = litho_0[['Lithology Keys', 'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 'NPHI', 'SP', 'PEF']]
litho_9  = litho_0[['Lithology Keys', 'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 'NPHI', 'SP', 'PEF']]
litho_10  = litho_0[['Lithology Keys', 'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 'NPHI', 'SP', 'PEF']]

In [ ]:
litho

array(['Shale', 'Sandstone', 'Sandstone/Shale', 'Limestone', 'Tuff',
       'Marl', 'Anhydrite', 'Dolomite', 'Chalk', 'Coal', 'Halite',
       'Basement'], dtype=object)

In [ ]:
litho_merge = pd.concat([litho_0, litho_1,  litho_2,  litho_3,  litho_4,  litho_5,  
                         litho_6, litho_7, litho_8,  litho_9,  litho_10,  litho_11
                         ])

#### Imputation by Lithology

In [ ]:
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

In [ ]:
imputer_0  = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=800, verbose=1, tol=0.00001)
imputer_1  = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_2  = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_3  = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_4  = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_5  = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_6  = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_7  = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_8  = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_9  = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_10 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_11 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)

In [ ]:
imputer_0.fit(litho_0.drop(['Lithology Keys'], axis=1))
imputer_1.fit(litho_1.drop(['Lithology Keys'], axis=1))
imputer_2.fit(litho_2.drop(['Lithology Keys'], axis=1))
imputer_3.fit(litho_3.drop(['Lithology Keys'], axis=1))
imputer_4.fit(litho_4.drop(['Lithology Keys'], axis=1))
imputer_5.fit(litho_5.drop(['Lithology Keys'], axis=1))
imputer_6.fit(litho_6.drop(['Lithology Keys'], axis=1))
imputer_7.fit(litho_7.drop(['Lithology Keys'], axis=1))
imputer_8.fit(litho_8.drop(['Lithology Keys'], axis=1))
imputer_9.fit(litho_9.drop(['Lithology Keys'], axis=1))
imputer_10.fit(litho_10.drop(['Lithology Keys'], axis=1))
imputer_11.fit(litho_11.drop(['Lithology Keys'], axis=1))

[IterativeImputer] Completing matrix with shape (720803, 8)
[IterativeImputer] Change: 86050.61463216723, scaled tolerance: 0.07429338867000002 
[IterativeImputer] Change: 115907.5038121561, scaled tolerance: 0.07429338867000002 
[IterativeImputer] Change: 8250.435704986094, scaled tolerance: 0.07429338867000002 
[IterativeImputer] Change: 14182.172846993311, scaled tolerance: 0.07429338867000002 
[IterativeImputer] Change: 12824.972847578309, scaled tolerance: 0.07429338867000002 
[IterativeImputer] Change: 12162.670894390996, scaled tolerance: 0.07429338867000002 
[IterativeImputer] Change: 11546.041083190416, scaled tolerance: 0.07429338867000002 
[IterativeImputer] Change: 11077.44913701233, scaled tolerance: 0.07429338867000002 
[IterativeImputer] Change: 10697.748499927975, scaled tolerance: 0.07429338867000002 
[IterativeImputer] Change: 10325.484285075603, scaled tolerance: 0.07429338867000002 
[IterativeImputer] Change: 9961.46332269166, scaled tolerance: 0.07429338867000002 


KeyboardInterrupt: ignored

In [ ]:
litho_0_trans  = imputer_0.transform(litho_0.drop(['Lithology Keys'], axis=1))
litho_1_trans  = imputer_1.transform(litho_1.drop(['Lithology Keys'], axis=1))
litho_2_trans  = imputer_2.transform(litho_2.drop(['Lithology Keys'], axis=1))
litho_3_trans  = imputer_3.transform(litho_3.drop(['Lithology Keys'], axis=1))
litho_4_trans  = imputer_4.transform(litho_4.drop(['Lithology Keys'], axis=1))
litho_5_trans  = imputer_5.transform(litho_5.drop(['Lithology Keys'], axis=1))
litho_6_trans  = imputer_6.transform(litho_6.drop(['Lithology Keys'], axis=1))
litho_7_trans  = imputer_7.transform(litho_7.drop(['Lithology Keys'], axis=1))
litho_8_trans  = imputer_8.transform(litho_8.drop(['Lithology Keys'], axis=1))
litho_9_trans  = imputer_9.transform(litho_9.drop(['Lithology Keys'], axis=1))
litho_10_trans = imputer_10.transform(litho_10.drop(['Lithology Keys'], axis=1))
litho_11_trans = imputer_11.transform(litho_11.drop(['Lithology Keys'], axis=1))

In [ ]:
df_litho_0_trans  = pd.DataFrame(litho_0_trans, columns=litho_0.columns[1:], index=litho_0.index)
df_litho_1_trans  = pd.DataFrame(litho_1_trans, columns=litho_1.columns[1:], index=litho_1.index)
df_litho_2_trans  = pd.DataFrame(litho_2_trans, columns=litho_2.columns[1:], index=litho_2.index)
df_litho_3_trans  = pd.DataFrame(litho_3_trans, columns=litho_3.columns[1:], index=litho_3.index)
df_litho_4_trans  = pd.DataFrame(litho_4_trans, columns=litho_4.columns[1:], index=litho_4.index)
df_litho_5_trans  = pd.DataFrame(litho_5_trans, columns=litho_5.columns[1:], index=litho_5.index)
df_litho_6_trans  = pd.DataFrame(litho_6_trans, columns=litho_6.columns[1:], index=litho_6.index)
df_litho_7_trans  = pd.DataFrame(litho_7_trans, columns=litho_7.columns[1:], index=litho_7.index)
df_litho_8_trans  = pd.DataFrame(litho_8_trans, columns=litho_8.columns[1:], index=litho_8.index)
df_litho_9_trans  = pd.DataFrame(litho_9_trans, columns=litho_9.columns[1:], index=litho_9.index)
df_litho_10_trans = pd.DataFrame(litho_10_trans, columns=litho_10.columns[1:], index=litho_10.index)
df_litho_11_trans = pd.DataFrame(litho_11_trans, columns=litho_11.columns[1:], index=litho_11.index)

In [ ]:
sc = MinMaxScaler()
#sc = StandardScaler()

In [ ]:
sc_0  = MinMaxScaler().fit(df_litho_0_trans)
sc_1  = MinMaxScaler().fit(df_litho_1_trans)
sc_2  = MinMaxScaler().fit(df_litho_2_trans)
sc_3  = MinMaxScaler().fit(df_litho_3_trans)
sc_4  = MinMaxScaler().fit(df_litho_4_trans)
sc_5  = MinMaxScaler().fit(df_litho_5_trans)
sc_6  = MinMaxScaler().fit(df_litho_6_trans)
sc_7  = MinMaxScaler().fit(df_litho_7_trans)
sc_8  = MinMaxScaler().fit(df_litho_8_trans)
sc_9  = MinMaxScaler().fit(df_litho_9_trans)
sc_10 = MinMaxScaler().fit(df_litho_10_trans)
sc_11 = MinMaxScaler().fit(df_litho_11_trans)

In [ ]:
mm_cols = ['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm', 
           'NPHI_mm', 'SP_mm', 'PEF_mm', 
           #'RMED_mm', 'RDEP_mm'
           ]
#std_cols = ['GR_std', 'DTC_std', 'CALI_std', 'DRHO_std', 'RHOB_std', 'RMED_std', 'RDEP_std']

In [ ]:
litho_0_merge_mm  = pd.DataFrame(sc_0.translitho(df_litho_0_trans), df_litho_0_trans.index, mm_cols)
litho_1_merge_mm  = pd.DataFrame(sc_1.translitho(df_litho_1_trans), df_litho_1_trans.index, mm_cols)
litho_2_merge_mm  = pd.DataFrame(sc_2.translitho(df_litho_2_trans), df_litho_2_trans.index, mm_cols)
litho_3_merge_mm  = pd.DataFrame(sc_3.translitho(df_litho_3_trans), df_litho_3_trans.index, mm_cols)
litho_4_merge_mm  = pd.DataFrame(sc_4.translitho(df_litho_4_trans), df_litho_4_trans.index, mm_cols)
litho_5_merge_mm  = pd.DataFrame(sc_5.translitho(df_litho_5_trans), df_litho_5_trans.index, mm_cols)
litho_6_merge_mm  = pd.DataFrame(sc_6.translitho(df_litho_6_trans), df_litho_6_trans.index, mm_cols)
litho_7_merge_mm  = pd.DataFrame(sc_7.translitho(df_litho_7_trans), df_litho_7_trans.index, mm_cols)
litho_8_merge_mm  = pd.DataFrame(sc_8.translitho(df_litho_8_trans), df_litho_8_trans.index, mm_cols)
litho_9_merge_mm  = pd.DataFrame(sc_9.translitho(df_litho_9_trans), df_litho_9_trans.index, mm_cols)
litho_10_merge_mm = pd.DataFrame(sc_10.translitho(df_litho_10_trans), df_litho_10_trans.index, mm_cols)
litho_11_merge_mm = pd.DataFrame(sc_11.translitho(df_litho_11_trans), df_litho_11_trans.index, mm_cols)

In [ ]:
form_mm = pd.concat([form_0_merge_mm, form_1_merge_mm, form_2_merge_mm, form_3_merge_mm, form_4_merge_mm,  form_5_merge_mm,
                     form_6_merge_mm, form_7_merge_mm, form_8_merge_mm, form_9_merge_mm, form_10_merge_mm, form_11_merge_mm])

form_mm.columns = ['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm',
                   'NPHI_mm', 'SP_mm', 'PEF_mm', 
                   'RMED_mm', 'RDEP_mm']
form_mm.head(2)

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm,RMED_mm,RDEP_mm
0,0.268520,2.987523,2.299851,-3.185325,-0.807412,1.710108,0.751008,20.915468,0.127908,-0.004782
1,0.264152,2.975092,2.297358,-3.158027,-0.798748,1.732229,0.748897,19.383013,0.127922,-0.004787


In [ ]:
df_form_trans = pd.concat([df_form_0_trans,
                           df_form_1_trans,  df_form_2_trans,  df_form_3_trans,  df_form_4_trans,  df_form_5_trans,
                           df_form_42_trans, df_form_19_trans, df_form_6_trans,  df_form_7_trans,  df_form_8_trans,
                           df_form_9_trans,  df_form_10_trans, df_form_11_trans, df_form_12_trans, df_form_13_trans,
                           df_form_14_trans, df_form_15_trans, df_form_24_trans, df_form_18_trans, df_form_16_trans,
                           df_form_21_trans, df_form_22_trans, df_form_40_trans, df_form_33_trans, df_form_23_trans,
                           df_form_43_trans, df_form_46_trans, df_form_52_trans, df_form_51_trans, df_form_44_trans,
                           df_form_45_trans, df_form_55_trans, df_form_56_trans, df_form_57_trans, df_form_58_trans,
                           df_form_47_trans, df_form_48_trans, df_form_60_trans, df_form_53_trans, df_form_68_trans,
                           df_form_54_trans, df_form_35_trans])

df_form_trans.columns = ['GR_new', 'DTC_new', 'CALI_new', 'DRHO_new', 'RHOB_new',
                         'NPHI_new', 'SP_new', 'PEF_new', 
                         'RMED_new', 'RDEP_new']
df_form_trans.head(2)

,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,RMED_new,RDEP_new
0,80.200851,161.13118,19.480835,-0.574928,1.884186,0.382829,24.612379,20.915468,1.61141,1.798681
1,79.262886,160.60347,19.468800,-0.570188,1.889794,0.387158,23.895531,19.383013,1.61807,1.795641


In [ ]:
df_form_trans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1088984 entries, 0 to 1130565
Data columns (total 10 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   GR_new    1088984 non-null  float64
 1   DTC_new   1088984 non-null  float64
 2   CALI_new  1088984 non-null  float64
 3   DRHO_new  1088984 non-null  float64
 4   RHOB_new  1088984 non-null  float64
 5   NPHI_new  1088984 non-null  float64
 6   SP_new    1088984 non-null  float64
 7   PEF_new   1088984 non-null  float64
 8   RMED_new  1088984 non-null  float64
 9   RDEP_new  1088984 non-null  float64
dtypes: float64(10)
memory usage: 91.4 MB


In [ ]:
df_form_trans_merge_mm = df_form_trans.merge(form_mm, left_index=True, right_index=True, how='outer')
df_form_trans_merge_mm.head(2)

,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,RMED_new,RDEP_new,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm,RMED_mm,RDEP_mm
0,80.200851,161.13118,19.480835,-0.574928,1.884186,0.382829,24.612379,20.915468,1.61141,1.798681,0.268520,2.987523,2.299851,-3.185325,-0.807412,1.710108,0.751008,20.915468,0.127908,-0.004782
1,79.262886,160.60347,19.468800,-0.570188,1.889794,0.387158,23.895531,19.383013,1.61807,1.795641,0.264152,2.975092,2.297358,-3.158027,-0.798748,1.732229,0.748897,19.383013,0.127922,-0.004787


In [ ]:
df_form_trans_merge_mm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1088984 entries, 0 to 1130565
Data columns (total 20 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   GR_new    1088984 non-null  float64
 1   DTC_new   1088984 non-null  float64
 2   CALI_new  1088984 non-null  float64
 3   DRHO_new  1088984 non-null  float64
 4   RHOB_new  1088984 non-null  float64
 5   NPHI_new  1088984 non-null  float64
 6   SP_new    1088984 non-null  float64
 7   PEF_new   1088984 non-null  float64
 8   RMED_new  1088984 non-null  float64
 9   RDEP_new  1088984 non-null  float64
 10  GR_mm     1088984 non-null  float64
 11  DTC_mm    1088984 non-null  float64
 12  CALI_mm   1088984 non-null  float64
 13  DRHO_mm   1088984 non-null  float64
 14  RHOB_mm   1088984 non-null  float64
 15  NPHI_mm   1088984 non-null  float64
 16  SP_mm     1088984 non-null  float64
 17  PEF_mm    1088984 non-null  float64
 18  RMED_mm   1088984 non-null  float64
 19  RDEP_mm   1088984 non

In [ ]:
df_train = pd.read_csv('./train.csv', sep=';')
df_train['Lithology Keys'] = df_train['FORCE_2020_LITHOFACIES_LITHOLOGY'].replace(lithology_keys)
df_train['Lithology Num'] = df_train['Lithology Keys'].replace(lithology_num)
df_train['Confidence Level'] = df_train['FORCE_2020_LITHOFACIES_CONFIDENCE']
df_train = df_train[['WELL', 'DEPTH_MD', 'FORMATION',
                     'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 
                     'NPHI', 'SP', 'PEF',
                     'RMED', 'RDEP',
                     'FORCE_2020_LITHOFACIES_LITHOLOGY', 'FORCE_2020_LITHOFACIES_CONFIDENCE', 
                     'Lithology Keys', 'Lithology Num']]

In [ ]:
df_train_form_trans_merge_mm = df_train.merge(df_form_trans, left_index=True, right_index=True, how='inner')
df_train_form_trans_merge_mm.head(2)

,WELL,DEPTH_MD,FORMATION,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF,RMED,RDEP,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE,Lithology Keys,Lithology Num,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,RMED_new,RDEP_new
0,15/9-13,494.528,NaN,80.200851,161.13118,19.480835,-0.574928,1.884186,NaN,24.612379,20.915468,1.61141,1.798681,65000,1.0,Shale,3,80.200851,161.13118,19.480835,-0.574928,1.884186,0.382829,24.612379,20.915468,1.61141,1.798681
1,15/9-13,494.680,NaN,79.262886,160.60347,19.468800,-0.570188,1.889794,NaN,23.895531,19.383013,1.61807,1.795641,65000,1.0,Shale,3,79.262886,160.60347,19.468800,-0.570188,1.889794,0.387158,23.895531,19.383013,1.61807,1.795641


In [ ]:
#df_form_trans_merge.to_csv('./df_form_trans_merge_rmedrdep.csv')
#df_form_trans_merge = pd.read_csv('./df_form_trans_merge.csv', index_col=0)

In [ ]:
# Duplicate Lithology

In [ ]:
limestone = df_train_form_trans_merge_mm[df_train_form_trans_merge_mm['Lithology Keys'] == 'Limestone']
marl = df_train_form_trans_merge_mm[df_train_form_trans_merge_mm['Lithology Keys'] == 'Marl']
tuff = df_train_form_trans_merge_mm[df_train_form_trans_merge_mm['Lithology Keys'] == 'Tuff']
chalk = df_train_form_trans_merge_mm[df_train_form_trans_merge_mm['Lithology Keys'] == 'Chalk']
halite = df_train_form_trans_merge_mm[df_train_form_trans_merge_mm['Lithology Keys'] == 'Halite']
coal = df_train_form_trans_merge_mm[df_train_form_trans_merge_mm['Lithology Keys'] == 'Coal']
dolomite = df_train_form_trans_merge_mm[df_train_form_trans_merge_mm['Lithology Keys'] == 'Dolomite']
anhydrite = df_train_form_trans_merge_mm[df_train_form_trans_merge_mm['Lithology Keys'] == 'Anhydrite']
basement = df_train_form_trans_merge_mm[df_train_form_trans_merge_mm['Lithology Keys'] == 'Basement']

In [ ]:
df_tuff      = df_train_form_trans_merge_mm.append([tuff]*2, ignore_index=True)

In [ ]:
df_chalk     = df_tuff.append([chalk]*3, ignore_index=True)

In [ ]:
df_halite    = df_chalk.append([halite]*4, ignore_index=True)

In [ ]:
df_coal      = df_halite.append([coal]*8, ignore_index=True)

In [ ]:
df_dolomite  = df_coal.append([dolomite]*20, ignore_index=True)

In [ ]:
df_anhydrite = df_dolomite.append([anhydrite]*30, ignore_index=True)

In [ ]:
df_basement  = df_anhydrite.append([basement]*300, ignore_index=True)

In [ ]:
df_train_form_trans_merge_mm = df_basement.copy()

In [ ]:
df_form_trans = df_train_form_trans_merge_mm.copy()

In [ ]:
df_form_trans['Lithology Keys'].unique()

array(['Shale', 'Sandstone', 'Sandstone/Shale', 'Limestone', 'Tuff',
       'Marl', 'Anhydrite', 'Dolomite', 'Chalk', 'Coal', 'Halite',
       'Basement'], dtype=object)

## Imputation by FORMATION

### df_train FORMATION based on df_test

In [22]:
df_train = df_train[['FORMATION', 'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 
                     'NPHI', 'SP', 'PEF', 
                     #'RMED', 'RDEP'
                     ]]
df_train.head(2)

,FORMATION,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF
0,no_form,80.200851,161.13118,19.480835,-0.574928,1.884186,NaN,24.612379,20.915468
1,no_form,79.262886,160.60347,19.468800,-0.570188,1.889794,NaN,23.895531,19.383013


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1170511 entries, 0 to 1170510
Data columns (total 8 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   FORMATION  1170511 non-null  object 
 1   GR         1170511 non-null  float64
 2   DTC        1089648 non-null  float64
 3   CALI       1082634 non-null  float64
 4   DRHO       987857 non-null   float64
 5   RHOB       1009242 non-null  float64
 6   NPHI       765409 non-null   float64
 7   PEF        671692 non-null   float64
dtypes: float64(7), object(1)
memory usage: 71.4+ MB


In [23]:
df_train['FORMATION'].fillna('no_form', inplace=True)

In [24]:
form = df_train['FORMATION'].unique()

In [ ]:
list(enumerate(form))

In [25]:
# Primary Formation
form_1  = df_train[df_train['FORMATION'] == form[1]]
form_2  = df_train[df_train['FORMATION'] == form[2]]
form_3  = df_train[df_train['FORMATION'] == form[3]]
form_4  = df_train[df_train['FORMATION'] == form[4]]
form_5  = df_train[df_train['FORMATION'] == form[5]]
form_42 = df_train[df_train['FORMATION'] == form[42]]
form_19 = df_train[df_train['FORMATION'] == form[19]]
form_6  = df_train[df_train['FORMATION'] == form[6]]
form_7  = df_train[df_train['FORMATION'] == form[7]]
form_8  = df_train[df_train['FORMATION'] == form[8]]

form_9  = df_train[df_train['FORMATION'] == form[9]]
form_10 = df_train[df_train['FORMATION'] == form[10]]
form_11 = df_train[df_train['FORMATION'] == form[11]]
form_12 = df_train[df_train['FORMATION'] == form[12]]
form_13 = df_train[df_train['FORMATION'] == form[13]]
form_14 = df_train[df_train['FORMATION'] == form[14]]
form_15 = df_train[df_train['FORMATION'] == form[15]]
form_24 = df_train[df_train['FORMATION'] == form[24]]
form_18 = df_train[df_train['FORMATION'] == form[18]]
form_16 = df_train[df_train['FORMATION'] == form[16]]

form_21 = df_train[df_train['FORMATION'] == form[21]]
form_22 = df_train[df_train['FORMATION'] == form[22]]
form_40 = df_train[df_train['FORMATION'] == form[40]]
form_33 = df_train[df_train['FORMATION'] == form[33]]
form_23 = df_train[df_train['FORMATION'] == form[23]]
form_43 = df_train[df_train['FORMATION'] == form[43]]
form_46 = df_train[df_train['FORMATION'] == form[46]]
form_52 = df_train[df_train['FORMATION'] == form[52]]
form_51 = df_train[df_train['FORMATION'] == form[51]]
form_44 = df_train[df_train['FORMATION'] == form[44]]

form_45 = df_train[df_train['FORMATION'] == form[45]]
form_55 = df_train[df_train['FORMATION'] == form[55]]
form_56 = df_train[df_train['FORMATION'] == form[56]]
form_57 = df_train[df_train['FORMATION'] == form[57]]
form_58 = df_train[df_train['FORMATION'] == form[58]]
form_47 = df_train[df_train['FORMATION'] == form[47]]
form_48 = df_train[df_train['FORMATION'] == form[48]]
form_60 = df_train[df_train['FORMATION'] == form[60]]
form_53 = df_train[df_train['FORMATION'] == form[53]]
form_68 = df_train[df_train['FORMATION'] == form[68]]

form_54 = df_train[df_train['FORMATION'] == form[54]]

In [26]:
# Secondary Formation
form_0  = df_train[df_train['FORMATION'] == form[0]]  #no_form  
form_35 = df_train[df_train['FORMATION'] == form[35]] #Basement

form_17  = df_train[df_train['FORMATION'] == form[17]]
form_20  = df_train[df_train['FORMATION'] == form[20]]
form_25  = df_train[df_train['FORMATION'] == form[25]]
form_26  = df_train[df_train['FORMATION'] == form[26]]
form_27  = df_train[df_train['FORMATION'] == form[27]]
form_28  = df_train[df_train['FORMATION'] == form[28]]
form_29 = df_train[df_train['FORMATION'] == form[29]]
form_30  = df_train[df_train['FORMATION'] == form[30]]
form_31  = df_train[df_train['FORMATION'] == form[31]]
form_32  = df_train[df_train['FORMATION'] == form[32]]

form_34  = df_train[df_train['FORMATION'] == form[34]]
form_36  = df_train[df_train['FORMATION'] == form[36]]
form_37  = df_train[df_train['FORMATION'] == form[37]]
form_38  = df_train[df_train['FORMATION'] == form[38]]
form_39  = df_train[df_train['FORMATION'] == form[39]]
form_41  = df_train[df_train['FORMATION'] == form[41]]
form_49  = df_train[df_train['FORMATION'] == form[49]]
form_50  = df_train[df_train['FORMATION'] == form[50]]
form_59  = df_train[df_train['FORMATION'] == form[59]]
form_61  = df_train[df_train['FORMATION'] == form[61]]

form_62  = df_train[df_train['FORMATION'] == form[62]]
form_63  = df_train[df_train['FORMATION'] == form[63]]
form_64  = df_train[df_train['FORMATION'] == form[64]]
form_65  = df_train[df_train['FORMATION'] == form[65]]
form_66  = df_train[df_train['FORMATION'] == form[66]]
form_67  = df_train[df_train['FORMATION'] == form[67]]
form_69  = df_train[df_train['FORMATION'] == form[69]]

In [27]:
form_merge_primary = pd.concat([form_1,  form_2,  form_3,  form_4,  form_5,  form_42, form_19, form_6,  form_7,  form_8,
                                form_9,  form_10, form_11, form_12, form_13, form_14, form_15, form_24, form_18, form_16,
                                form_21, form_22, form_40, form_33, form_23, form_43, form_46, form_52, form_51, form_44,
                                form_45, form_55, form_56, form_57, form_58, form_47, form_48, form_60, form_53, form_68, 
                                form_54])

In [28]:
form_merge_second = pd.concat([form_0, form_35,
                               form_17, form_20, form_25, form_26, form_27, form_28, form_29, form_30, form_31, form_32, 
                               form_34, form_36, form_37, form_38, form_39, form_41, form_49, form_50, form_59, form_61, 
                               form_62, form_63, form_64, form_65, form_66, form_67, form_69                               
                               ])

In [ ]:
#form_merge = form_merge[['WELL', 'DEPTH_MD', 'FORMATION',
 #                        'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 'NPHI', 'SP', 'PEF', 'RMED', 'RDEP',
  #                       'FORCE_2020_LITHOFACIES_LITHOLOGY', 'FORCE_2020_LITHOFACIES_CONFIDENCE', 
   #                      'Lithology Keys', 'Lithology Num']]

In [ ]:
form_merge_second.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218767 entries, 0 to 1169030
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   FORMATION  218767 non-null  object 
 1   GR         218767 non-null  float64
 2   DTC        192763 non-null  float64
 3   CALI       186464 non-null  float64
 4   DRHO       159340 non-null  float64
 5   RHOB       158336 non-null  float64
 6   NPHI       118405 non-null  float64
 7   SP         171816 non-null  float64
 8   PEF        93689 non-null   float64
 9   RMED       207660 non-null  float64
 10  RDEP       214963 non-null  float64
dtypes: float64(10), object(1)
memory usage: 20.0+ MB


In [ ]:
form_merge_primary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 951744 entries, 2464 to 1116196
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   FORMATION  951744 non-null  object 
 1   GR         951744 non-null  float64
 2   DTC        896885 non-null  float64
 3   CALI       896170 non-null  float64
 4   DRHO       828517 non-null  float64
 5   RHOB       850906 non-null  float64
 6   NPHI       647004 non-null  float64
 7   SP         692431 non-null  float64
 8   PEF        578003 non-null  float64
 9   RMED       923858 non-null  float64
 10  RDEP       944533 non-null  float64
dtypes: float64(10), object(1)
memory usage: 87.1+ MB


#### Imputation by FORMATION

In [29]:
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

In [30]:
imputer_1 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_2 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_3 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_4 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_5 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_6 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_7 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_8 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_9 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_10 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)

imputer_11 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_12 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_13 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_14 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_15 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_16 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_17 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_18 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_19 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_20 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)

imputer_21 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_22 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_23 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_24 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_25 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_26 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_27 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_28 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_29 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_30 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)

imputer_31 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_32 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_33 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_34 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_35 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_36 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_37 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_38 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_39 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)
imputer_40 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)

imputer_41 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.00001)

imputer_sec = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=1000, verbose=1, tol=0.00001)

In [31]:
form_68['SP'] = 0.0

In [32]:
form_68.head(2)

,FORMATION,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF
1073249,Agat Fm.,90.022957,72.066521,8.538504,0.006742,2.60782,0.111282,0.0,4.121771
1073250,Agat Fm.,83.319473,71.390228,8.551457,-0.000420,2.59683,0.086492,0.0,4.028265


In [33]:
imputer_1.fit(form_1.drop(['FORMATION'], axis=1))
imputer_2.fit(form_2.drop(['FORMATION'], axis=1))
imputer_3.fit(form_3.drop(['FORMATION'], axis=1))
imputer_4.fit(form_4.drop(['FORMATION'], axis=1))
imputer_5.fit(form_5.drop(['FORMATION'], axis=1))
imputer_6.fit(form_42.drop(['FORMATION'], axis=1))
imputer_7.fit(form_19.drop(['FORMATION'], axis=1))
imputer_8.fit(form_6.drop(['FORMATION'], axis=1))
imputer_9.fit(form_7.drop(['FORMATION'], axis=1))
imputer_10.fit(form_8.drop(['FORMATION'], axis=1))

imputer_11.fit(form_9.drop(['FORMATION'], axis=1))
imputer_12.fit(form_10.drop(['FORMATION'], axis=1))
imputer_13.fit(form_11.drop(['FORMATION'], axis=1))
imputer_14.fit(form_12.drop(['FORMATION'], axis=1))
imputer_15.fit(form_13.drop(['FORMATION'], axis=1))
imputer_16.fit(form_14.drop(['FORMATION'], axis=1))
imputer_17.fit(form_15.drop(['FORMATION'], axis=1))
imputer_18.fit(form_24.drop(['FORMATION'], axis=1))
imputer_19.fit(form_18.drop(['FORMATION'], axis=1))
imputer_20.fit(form_16.drop(['FORMATION'], axis=1))

imputer_21.fit(form_21.drop(['FORMATION'], axis=1))
imputer_22.fit(form_22.drop(['FORMATION'], axis=1))
imputer_23.fit(form_40.drop(['FORMATION'], axis=1))
imputer_24.fit(form_33.drop(['FORMATION'], axis=1))
imputer_25.fit(form_23.drop(['FORMATION'], axis=1))
imputer_26.fit(form_43.drop(['FORMATION'], axis=1))
imputer_27.fit(form_46.drop(['FORMATION'], axis=1))
imputer_28.fit(form_52.drop(['FORMATION'], axis=1))
imputer_29.fit(form_51.drop(['FORMATION'], axis=1))
imputer_30.fit(form_44.drop(['FORMATION'], axis=1))

imputer_31.fit(form_45.drop(['FORMATION'], axis=1))
imputer_32.fit(form_55.drop(['FORMATION'], axis=1))
imputer_33.fit(form_56.drop(['FORMATION'], axis=1))
imputer_34.fit(form_57.drop(['FORMATION'], axis=1))
imputer_35.fit(form_58.drop(['FORMATION'], axis=1))
imputer_36.fit(form_47.drop(['FORMATION'], axis=1))
imputer_37.fit(form_48.drop(['FORMATION'], axis=1))
imputer_38.fit(form_60.drop(['FORMATION'], axis=1))
imputer_39.fit(form_53.drop(['FORMATION'], axis=1))
imputer_40.fit(form_68.drop(['FORMATION'], axis=1))

imputer_41.fit(form_54.drop(['FORMATION'], axis=1))

imputer_sec.fit(form_merge_second.drop(['FORMATION'], axis=1))

[IterativeImputer] Completing matrix with shape (172636, 8)
[IterativeImputer] Change: 423.04141652544365, scaled tolerance: 0.005265473022500001 
[IterativeImputer] Change: 62.259877055989776, scaled tolerance: 0.005265473022500001 
[IterativeImputer] Change: 17.835803434866897, scaled tolerance: 0.005265473022500001 
[IterativeImputer] Change: 11.459480449572789, scaled tolerance: 0.005265473022500001 
[IterativeImputer] Change: 10.746630423935079, scaled tolerance: 0.005265473022500001 
[IterativeImputer] Change: 9.870370410392802, scaled tolerance: 0.005265473022500001 
[IterativeImputer] Change: 8.891913415997182, scaled tolerance: 0.005265473022500001 
[IterativeImputer] Change: 7.882549546091757, scaled tolerance: 0.005265473022500001 
[IterativeImputer] Change: 6.894133823196729, scaled tolerance: 0.005265473022500001 
[IterativeImputer] Change: 5.959890872182514, scaled tolerance: 0.005265473022500001 
[IterativeImputer] Change: 5.099028175486553, scaled tolerance: 0.005265473

IterativeImputer(estimator=BayesianRidge(), imputation_order='descending',
                 max_iter=1000, tol=1e-05, verbose=1)

In [34]:
form_1_trans  = imputer_1.transform(form_1.drop(['FORMATION'], axis=1))
form_2_trans  = imputer_2.transform(form_2.drop(['FORMATION'], axis=1))
form_3_trans  = imputer_3.transform(form_3.drop(['FORMATION'], axis=1))
form_4_trans  = imputer_4.transform(form_4.drop(['FORMATION'], axis=1))
form_5_trans  = imputer_5.transform(form_5.drop(['FORMATION'], axis=1))
form_42_trans = imputer_6.transform(form_42.drop(['FORMATION'], axis=1))
form_19_trans = imputer_7.transform(form_19.drop(['FORMATION'], axis=1))
form_6_trans  = imputer_8.transform(form_6.drop(['FORMATION'], axis=1))
form_7_trans  = imputer_9.transform(form_7.drop(['FORMATION'], axis=1))
form_8_trans  = imputer_10.transform(form_8.drop(['FORMATION'], axis=1))

form_9_trans  = imputer_11.transform(form_9.drop(['FORMATION'], axis=1))
form_10_trans = imputer_12.transform(form_10.drop(['FORMATION'], axis=1))
form_11_trans = imputer_13.transform(form_11.drop(['FORMATION'], axis=1))
form_12_trans = imputer_14.transform(form_12.drop(['FORMATION'], axis=1))
form_13_trans = imputer_15.transform(form_13.drop(['FORMATION'], axis=1))
form_14_trans = imputer_16.transform(form_14.drop(['FORMATION'], axis=1))
form_15_trans = imputer_17.transform(form_15.drop(['FORMATION'], axis=1))
form_24_trans = imputer_18.transform(form_24.drop(['FORMATION'], axis=1))
form_18_trans = imputer_19.transform(form_18.drop(['FORMATION'], axis=1))
form_16_trans = imputer_20.transform(form_16.drop(['FORMATION'], axis=1))

form_21_trans = imputer_21.transform(form_21.drop(['FORMATION'], axis=1))
form_22_trans = imputer_22.transform(form_22.drop(['FORMATION'], axis=1))
form_40_trans = imputer_23.transform(form_40.drop(['FORMATION'], axis=1))
form_33_trans = imputer_24.transform(form_33.drop(['FORMATION'], axis=1))
form_23_trans = imputer_25.transform(form_23.drop(['FORMATION'], axis=1))
form_43_trans = imputer_26.transform(form_43.drop(['FORMATION'], axis=1))
form_46_trans = imputer_27.transform(form_46.drop(['FORMATION'], axis=1))
form_52_trans = imputer_28.transform(form_52.drop(['FORMATION'], axis=1))
form_51_trans = imputer_29.transform(form_51.drop(['FORMATION'], axis=1))
form_44_trans = imputer_30.transform(form_44.drop(['FORMATION'], axis=1))

form_45_trans = imputer_31.transform(form_45.drop(['FORMATION'], axis=1))
form_55_trans = imputer_32.transform(form_55.drop(['FORMATION'], axis=1))
form_56_trans = imputer_33.transform(form_56.drop(['FORMATION'], axis=1))
form_57_trans = imputer_34.transform(form_57.drop(['FORMATION'], axis=1))
form_58_trans = imputer_35.transform(form_58.drop(['FORMATION'], axis=1))
form_47_trans = imputer_36.transform(form_47.drop(['FORMATION'], axis=1))
form_48_trans = imputer_37.transform(form_48.drop(['FORMATION'], axis=1))
form_60_trans = imputer_38.transform(form_60.drop(['FORMATION'], axis=1))
form_53_trans = imputer_39.transform(form_53.drop(['FORMATION'], axis=1))
form_68_trans = imputer_40.transform(form_68.drop(['FORMATION'], axis=1))

form_54_trans = imputer_41.transform(form_54.drop(['FORMATION'], axis=1))

form_sec_trans = imputer_sec.transform(form_merge_second.drop(['FORMATION'], axis=1))

[IterativeImputer] Completing matrix with shape (172636, 8)
[IterativeImputer] Completing matrix with shape (25814, 8)
[IterativeImputer] Completing matrix with shape (25451, 8)
[IterativeImputer] Completing matrix with shape (71080, 8)
[IterativeImputer] Completing matrix with shape (9662, 8)
[IterativeImputer] Completing matrix with shape (5012, 8)
[IterativeImputer] Completing matrix with shape (2417, 8)
[IterativeImputer] Completing matrix with shape (13525, 8)
[IterativeImputer] Completing matrix with shape (11862, 8)
[IterativeImputer] Completing matrix with shape (3016, 8)
[IterativeImputer] Completing matrix with shape (15215, 8)
[IterativeImputer] Completing matrix with shape (17462, 8)
[IterativeImputer] Completing matrix with shape (3928, 8)
[IterativeImputer] Completing matrix with shape (14817, 8)
[IterativeImputer] Completing matrix with shape (24961, 8)
[IterativeImputer] Completing matrix with shape (65041, 8)
[IterativeImputer] Completing matrix with shape (4656, 8)
[I

In [35]:
df_form_1_trans = pd.DataFrame(form_1_trans, columns=form_1.columns[1:], index=form_1.index)
df_form_2_trans = pd.DataFrame(form_2_trans, columns=form_2.columns[1:], index=form_2.index)
df_form_3_trans = pd.DataFrame(form_3_trans, columns=form_3.columns[1:], index=form_3.index)
df_form_4_trans = pd.DataFrame(form_4_trans, columns=form_4.columns[1:], index=form_4.index)
df_form_5_trans = pd.DataFrame(form_5_trans, columns=form_5.columns[1:], index=form_5.index)
df_form_42_trans = pd.DataFrame(form_42_trans, columns=form_42.columns[1:], index=form_42.index)
df_form_19_trans = pd.DataFrame(form_19_trans, columns=form_19.columns[1:], index=form_19.index)
df_form_6_trans = pd.DataFrame(form_6_trans, columns=form_6.columns[1:], index=form_6.index)
df_form_7_trans = pd.DataFrame(form_7_trans, columns=form_7.columns[1:], index=form_7.index)
df_form_8_trans = pd.DataFrame(form_8_trans, columns=form_8.columns[1:], index=form_8.index)

df_form_9_trans = pd.DataFrame(form_9_trans, columns=form_9.columns[1:], index=form_9.index)
df_form_10_trans = pd.DataFrame(form_10_trans, columns=form_10.columns[1:], index=form_10.index)
df_form_11_trans = pd.DataFrame(form_11_trans, columns=form_11.columns[1:], index=form_11.index)
df_form_12_trans = pd.DataFrame(form_12_trans, columns=form_12.columns[1:], index=form_12.index)
df_form_13_trans = pd.DataFrame(form_13_trans, columns=form_13.columns[1:], index=form_13.index)
df_form_14_trans = pd.DataFrame(form_14_trans, columns=form_14.columns[1:], index=form_14.index)
df_form_15_trans = pd.DataFrame(form_15_trans, columns=form_15.columns[1:], index=form_15.index)
df_form_24_trans = pd.DataFrame(form_24_trans, columns=form_24.columns[1:], index=form_24.index)
df_form_18_trans = pd.DataFrame(form_18_trans, columns=form_18.columns[1:], index=form_18.index)
df_form_16_trans = pd.DataFrame(form_16_trans, columns=form_16.columns[1:], index=form_16.index)

df_form_21_trans = pd.DataFrame(form_21_trans, columns=form_21.columns[1:], index=form_21.index)
df_form_22_trans = pd.DataFrame(form_22_trans, columns=form_22.columns[1:], index=form_22.index)
df_form_40_trans = pd.DataFrame(form_40_trans, columns=form_40.columns[1:], index=form_40.index)
df_form_33_trans = pd.DataFrame(form_33_trans, columns=form_33.columns[1:], index=form_33.index)
df_form_23_trans = pd.DataFrame(form_23_trans, columns=form_23.columns[1:], index=form_23.index)
df_form_43_trans = pd.DataFrame(form_43_trans, columns=form_43.columns[1:], index=form_43.index)
df_form_46_trans = pd.DataFrame(form_46_trans, columns=form_46.columns[1:], index=form_46.index)
df_form_52_trans = pd.DataFrame(form_52_trans, columns=form_52.columns[1:], index=form_52.index)
df_form_51_trans = pd.DataFrame(form_51_trans, columns=form_51.columns[1:], index=form_51.index)
df_form_44_trans = pd.DataFrame(form_44_trans, columns=form_44.columns[1:], index=form_44.index)

df_form_45_trans = pd.DataFrame(form_45_trans, columns=form_45.columns[1:], index=form_45.index)
df_form_55_trans = pd.DataFrame(form_55_trans, columns=form_55.columns[1:], index=form_55.index)
df_form_56_trans = pd.DataFrame(form_56_trans, columns=form_56.columns[1:], index=form_56.index)
df_form_57_trans = pd.DataFrame(form_57_trans, columns=form_57.columns[1:], index=form_57.index)
df_form_58_trans = pd.DataFrame(form_58_trans, columns=form_58.columns[1:], index=form_58.index)
df_form_47_trans = pd.DataFrame(form_47_trans, columns=form_47.columns[1:], index=form_47.index)
df_form_48_trans = pd.DataFrame(form_48_trans, columns=form_48.columns[1:], index=form_48.index)
df_form_60_trans = pd.DataFrame(form_60_trans, columns=form_60.columns[1:], index=form_60.index)
df_form_53_trans = pd.DataFrame(form_53_trans, columns=form_53.columns[1:], index=form_53.index)
df_form_68_trans = pd.DataFrame(form_68_trans, columns=form_68.columns[1:], index=form_68.index)

df_form_54_trans = pd.DataFrame(form_54_trans, columns=form_54.columns[1:], index=form_54.index)

df_form_sec_trans = pd.DataFrame(form_sec_trans, columns=form_merge_second.columns[1:], index=form_merge_second.index)

In [36]:
df_form_trans = pd.concat([df_form_1_trans,  df_form_2_trans,  df_form_3_trans,  df_form_4_trans,  df_form_5_trans,
                           df_form_42_trans, df_form_19_trans, df_form_6_trans,  df_form_7_trans,  df_form_8_trans,
                           df_form_9_trans,  df_form_10_trans, df_form_11_trans, df_form_12_trans, df_form_13_trans,
                           df_form_14_trans, df_form_15_trans, df_form_24_trans, df_form_18_trans, df_form_16_trans,
                           df_form_21_trans, df_form_22_trans, df_form_40_trans, df_form_33_trans, df_form_23_trans,
                           df_form_43_trans, df_form_46_trans, df_form_52_trans, df_form_51_trans, df_form_44_trans,
                           df_form_45_trans, df_form_55_trans, df_form_56_trans, df_form_57_trans, df_form_58_trans,
                           df_form_47_trans, df_form_48_trans, df_form_60_trans, df_form_53_trans, df_form_68_trans,
                           df_form_54_trans, 
                           df_form_sec_trans])

df_form_trans.columns = ['GR_new', 'DTC_new', 'CALI_new', 'DRHO_new', 'RHOB_new',
                         'NPHI_new', 'SP_new', 'PEF_new', 
                          #'RMED_trans', 'RDEP_trans'
                        ]
df_form_trans.head(2)

,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new
2464,44.851158,138.449585,18.544184,0.000899,2.098715,0.527227,29.880625,1.576887
2465,37.774338,140.845428,18.908665,-0.008508,2.076638,0.532000,29.118149,1.453022


In [37]:
df_form_trans['Umaa_1'] = (df_form_trans['PEF_new']*(((df_form_trans['RHOB_new']+0.1883)/1.0704)-(df_form_trans['NPHI_new']*1)))/1-df_form_trans['NPHI_new']

In [38]:
df_form_trans['Umaa_2'] = (df_form_trans['PEF_new']*(((df_form_trans['DRHO_new']+0.1883)/1.0704)-(df_form_trans['NPHI_new']*1)))/1-df_form_trans['NPHI_new']

In [39]:
df_form_trans['Cmaa_1'] = (df_form_trans['RHOB_new']-(df_form_trans['NPHI_new']*1))/1-df_form_trans['NPHI_new']

In [40]:
df_form_trans['Cmaa_2'] = (df_form_trans['DRHO_new']-(df_form_trans['NPHI_new']*1))/1-df_form_trans['NPHI_new']

In [41]:
df_form_trans['Aaa']    = df_form_trans['DTC_new']-((df_form_trans['NPHI_new']*df_form_trans['DTC_new'])/0.68)

In [42]:
df_form_trans.head(2)

,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,Umaa_1,Umaa_2,Cmaa_1,Cmaa_2,Aaa
2464,44.851158,138.449585,18.544184,0.000899,2.098715,0.527227,29.880625,1.576887,2.010570,-1.079882,1.044261,-1.053556,31.104903
2465,37.774338,140.845428,18.908665,-0.008508,2.076638,0.532000,29.118149,1.453022,1.769548,-1.060949,1.012638,-1.072509,30.654540


In [43]:
df_train = pd.read_csv('./train.csv', sep=';')
df_train['Lithology Keys'] = df_train['FORCE_2020_LITHOFACIES_LITHOLOGY'].replace(lithology_keys)
df_train['Lithology Num'] = df_train['Lithology Keys'].replace(lithology_num)
df_train['Confidence Level'] = df_train['FORCE_2020_LITHOFACIES_CONFIDENCE']
df_train = df_train[['WELL', 'DEPTH_MD', 'FORMATION',
                     'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 
                     'NPHI', 'SP', 'PEF',
                     #'RMED', 'RDEP',
                     'FORCE_2020_LITHOFACIES_LITHOLOGY', 'FORCE_2020_LITHOFACIES_CONFIDENCE', 
                     'Lithology Keys', 'Lithology Num']]

In [44]:
df_train_form_trans = df_train.merge(df_form_trans, left_index=True, right_index=True, how='outer')
df_train_form_trans.head(2)

,WELL,DEPTH_MD,FORMATION,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE,Lithology Keys,Lithology Num,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,Umaa_1,Umaa_2,Cmaa_1,Cmaa_2,Aaa
0,15/9-13,494.528,NaN,80.200851,161.13118,19.480835,-0.574928,1.884186,NaN,24.612379,20.915468,65000,1.0,Shale,3,80.200851,161.13118,19.480835,-0.574928,1.884186,0.629070,24.612379,20.915468,26.709728,-21.341009,0.626046,-1.833067,12.068311
1,15/9-13,494.680,NaN,79.262886,160.60347,19.468800,-0.570188,1.889794,NaN,23.895531,19.383013,65000,1.0,Shale,3,79.262886,160.60347,19.468800,-0.570188,1.889794,0.636301,23.895531,19.383013,24.660790,-19.885044,0.617191,-1.842791,10.320867


In [45]:
df_train_form_trans['FORMATION'].fillna('no_form', inplace=True)

In [46]:
form = df_train_form_trans['FORMATION'].unique()

In [47]:
df_train_form_trans_split = df_train_form_trans[['FORMATION', 'GR_new', 'DTC_new', 'CALI_new', 'DRHO_new', 'RHOB_new', 'NPHI_new',
                                                 'SP_new', 'PEF_new', 'Umaa_1', 'Umaa_2', 'Cmaa_1', 'Cmaa_2', 'Aaa']]

In [48]:
# Primary Formation
form_1  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[1]]
form_2  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[2]]
form_3  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[3]]
form_4  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[4]]
form_5  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[5]]
form_42 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[42]]
form_19 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[19]]
form_6  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[6]]
form_7  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[7]]
form_8  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[8]]

form_9  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[9]]
form_10 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[10]]
form_11 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[11]]
form_12 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[12]]
form_13 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[13]]
form_14 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[14]]
form_15 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[15]]
form_24 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[24]]
form_18 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[18]]
form_16 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[16]]

form_21 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[21]]
form_22 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[22]]
form_40 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[40]]
form_33 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[33]]
form_23 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[23]]
form_43 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[43]]
form_46 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[46]]
form_52 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[52]]
form_51 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[51]]
form_44 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[44]]

form_45 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[45]]
form_55 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[55]]
form_56 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[56]]
form_57 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[57]]
form_58 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[58]]
form_47 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[47]]
form_48 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[48]]
form_60 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[60]]
form_53 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[53]]
form_68 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[68]]

form_54 = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[54]]

In [49]:
# Secondary Formation
form_0   = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[0]]  #no_form  
form_35  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[35]] #Basement

form_17  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[17]]
form_20  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[20]]
form_25  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[25]]
form_26  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[26]]
form_27  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[27]]
form_28  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[28]]
form_29  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[29]]
form_30  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[30]]
form_31  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[31]]
form_32  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[32]]

form_34  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[34]]
form_36  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[36]]
form_37  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[37]]
form_38  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[38]]
form_39  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[39]]
form_41  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[41]]
form_49  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[49]]
form_50  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[50]]
form_59  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[59]]
form_61  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[61]]

form_62  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[62]]
form_63  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[63]]
form_64  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[64]]
form_65  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[65]]
form_66  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[66]]
form_67  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[67]]
form_69  = df_train_form_trans_split[df_train_form_trans_split['FORMATION'] == form[69]]

In [50]:
form_merge_second = pd.concat([form_0, form_35,
                               form_17, form_20, form_25, form_26, form_27, form_28, form_29, form_30, form_31, form_32, 
                               form_34, form_36, form_37, form_38, form_39, form_41, form_49, form_50, form_59, form_61, 
                               form_62, form_63, form_64, form_65, form_66, form_67, form_69                               
                               ])

In [51]:
form_merge = pd.concat([form_1,  form_2,  form_3,  form_4,  form_5,  form_42, form_19, form_6,  form_7,  form_8,
                        form_9,  form_10, form_11, form_12, form_13, form_14, form_15, form_24, form_18, form_16,
                        form_21, form_22, form_40, form_33, form_23, form_43, form_46, form_52, form_51, form_44,
                        form_45, form_55, form_56, form_57, form_58, form_47, form_48, form_60, form_53, form_68, 
                        form_54, form_merge_second])

In [52]:
form_merge.head(2)

,FORMATION,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,Umaa_1,Umaa_2,Cmaa_1,Cmaa_2,Aaa
2464,Utsira Fm.,44.851158,138.449585,18.544184,0.000899,2.098715,0.527227,29.880625,1.576887,2.010570,-1.079882,1.044261,-1.053556,31.104903
2465,Utsira Fm.,37.774338,140.845428,18.908665,-0.008508,2.076638,0.532000,29.118149,1.453022,1.769548,-1.060949,1.012638,-1.072509,30.654540


In [53]:
sc = MinMaxScaler()
#sc = StandardScaler()
#sc = RobustScaler()

In [54]:
sc_1 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_1.drop(['FORMATION'], axis=1))
sc_2 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_2.drop(['FORMATION'], axis=1))
sc_3 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_3.drop(['FORMATION'], axis=1))
sc_4 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_4.drop(['FORMATION'], axis=1))
sc_5 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_5.drop(['FORMATION'], axis=1))
sc_6 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_42.drop(['FORMATION'], axis=1))
sc_7 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_19.drop(['FORMATION'], axis=1))
sc_8 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_6.drop(['FORMATION'], axis=1))
sc_9 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_7.drop(['FORMATION'], axis=1))
sc_10 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_8.drop(['FORMATION'], axis=1))

sc_11 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_9.drop(['FORMATION'], axis=1))
sc_12 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_10.drop(['FORMATION'], axis=1))
sc_13 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_11.drop(['FORMATION'], axis=1))
sc_14 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_12.drop(['FORMATION'], axis=1))
sc_15 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_13.drop(['FORMATION'], axis=1))
sc_16 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_14.drop(['FORMATION'], axis=1))
sc_17 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_15.drop(['FORMATION'], axis=1))
sc_18 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_24.drop(['FORMATION'], axis=1))
sc_19 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_18.drop(['FORMATION'], axis=1))
sc_20 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_16.drop(['FORMATION'], axis=1))

sc_21 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_21.drop(['FORMATION'], axis=1))
sc_22 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_22.drop(['FORMATION'], axis=1))
sc_23 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_40.drop(['FORMATION'], axis=1))
sc_24 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_33.drop(['FORMATION'], axis=1))
sc_25 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_23.drop(['FORMATION'], axis=1))
sc_26 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_43.drop(['FORMATION'], axis=1))
sc_27 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_46.drop(['FORMATION'], axis=1))
sc_28 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_52.drop(['FORMATION'], axis=1))
sc_29 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_51.drop(['FORMATION'], axis=1))
sc_30 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_44.drop(['FORMATION'], axis=1))

sc_31 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_45.drop(['FORMATION'], axis=1))
sc_32 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_55.drop(['FORMATION'], axis=1))
sc_33 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_56.drop(['FORMATION'], axis=1))
sc_34 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_57.drop(['FORMATION'], axis=1))
sc_35 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_58.drop(['FORMATION'], axis=1))
sc_36 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_47.drop(['FORMATION'], axis=1))
sc_37 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_48.drop(['FORMATION'], axis=1))
sc_38 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_60.drop(['FORMATION'], axis=1))
sc_39 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_53.drop(['FORMATION'], axis=1))
sc_40 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_68.drop(['FORMATION'], axis=1))

sc_41 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_54.drop(['FORMATION'], axis=1))

sc_sec = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_merge_second.drop(['FORMATION'], axis=1))

In [55]:
mm_cols = ['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm', 
           'NPHI_mm', 'SP_mm', 'PEF_mm', 
           'Umaa_1_mm', 'Umaa_2_mm', 'Cmaa_1_mm', 'Cmaa_2_mm', 'Aaa_mm'
           #'RMED_mm', 'RDEP_mm'
           ]
#std_cols = ['GR_std', 'DTC_std', 'CALI_std', 'DRHO_std', 'RHOB_std', 'RMED_std', 'RDEP_std']

In [56]:
form_1_merge_mm  = pd.DataFrame(sc_1.transform(form_1.drop(['FORMATION'], axis=1)), form_1.index, mm_cols)
form_2_merge_mm  = pd.DataFrame(sc_2.transform(form_2.drop(['FORMATION'], axis=1)), form_2.index, mm_cols)
form_3_merge_mm  = pd.DataFrame(sc_3.transform(form_3.drop(['FORMATION'], axis=1)), form_3.index, mm_cols)
form_4_merge_mm  = pd.DataFrame(sc_4.transform(form_4.drop(['FORMATION'], axis=1)), form_4.index, mm_cols)
form_5_merge_mm  = pd.DataFrame(sc_5.transform(form_5.drop(['FORMATION'], axis=1)), form_5.index, mm_cols)
form_42_merge_mm = pd.DataFrame(sc_6.transform(form_42.drop(['FORMATION'], axis=1)), form_42.index, mm_cols)
form_19_merge_mm = pd.DataFrame(sc_7.transform(form_19.drop(['FORMATION'], axis=1)), form_19.index, mm_cols)
form_6_merge_mm  = pd.DataFrame(sc_8.transform(form_6.drop(['FORMATION'], axis=1)), form_6.index, mm_cols)
form_7_merge_mm  = pd.DataFrame(sc_9.transform(form_7.drop(['FORMATION'], axis=1)), form_7.index, mm_cols)
form_8_merge_mm  = pd.DataFrame(sc_10.transform(form_8.drop(['FORMATION'], axis=1)), form_8.index, mm_cols)

form_9_merge_mm  = pd.DataFrame(sc_11.transform(form_9.drop(['FORMATION'], axis=1)), form_9.index, mm_cols)
form_10_merge_mm = pd.DataFrame(sc_12.transform(form_10.drop(['FORMATION'], axis=1)), form_10.index, mm_cols)
form_11_merge_mm = pd.DataFrame(sc_13.transform(form_11.drop(['FORMATION'], axis=1)), form_11.index, mm_cols)
form_12_merge_mm = pd.DataFrame(sc_14.transform(form_12.drop(['FORMATION'], axis=1)), form_12.index, mm_cols)
form_13_merge_mm = pd.DataFrame(sc_15.transform(form_13.drop(['FORMATION'], axis=1)), form_13.index, mm_cols)
form_14_merge_mm = pd.DataFrame(sc_16.transform(form_14.drop(['FORMATION'], axis=1)), form_14.index, mm_cols)
form_15_merge_mm = pd.DataFrame(sc_17.transform(form_15.drop(['FORMATION'], axis=1)), form_15.index, mm_cols)
form_24_merge_mm = pd.DataFrame(sc_18.transform(form_24.drop(['FORMATION'], axis=1)), form_24.index, mm_cols)
form_18_merge_mm = pd.DataFrame(sc_19.transform(form_18.drop(['FORMATION'], axis=1)), form_18.index, mm_cols)
form_16_merge_mm = pd.DataFrame(sc_20.transform(form_16.drop(['FORMATION'], axis=1)), form_16.index, mm_cols)

form_21_merge_mm = pd.DataFrame(sc_21.transform(form_21.drop(['FORMATION'], axis=1)), form_21.index, mm_cols)
form_22_merge_mm = pd.DataFrame(sc_22.transform(form_22.drop(['FORMATION'], axis=1)), form_22.index, mm_cols)
form_40_merge_mm = pd.DataFrame(sc_23.transform(form_40.drop(['FORMATION'], axis=1)), form_40.index, mm_cols)
form_33_merge_mm = pd.DataFrame(sc_24.transform(form_33.drop(['FORMATION'], axis=1)), form_33.index, mm_cols)
form_23_merge_mm = pd.DataFrame(sc_25.transform(form_23.drop(['FORMATION'], axis=1)), form_23.index, mm_cols)
form_43_merge_mm = pd.DataFrame(sc_26.transform(form_43.drop(['FORMATION'], axis=1)), form_43.index, mm_cols)
form_46_merge_mm = pd.DataFrame(sc_27.transform(form_46.drop(['FORMATION'], axis=1)), form_46.index, mm_cols)
form_52_merge_mm = pd.DataFrame(sc_28.transform(form_52.drop(['FORMATION'], axis=1)), form_52.index, mm_cols)
form_51_merge_mm = pd.DataFrame(sc_29.transform(form_51.drop(['FORMATION'], axis=1)), form_51.index, mm_cols)
form_44_merge_mm = pd.DataFrame(sc_30.transform(form_44.drop(['FORMATION'], axis=1)), form_44.index, mm_cols)

form_45_merge_mm = pd.DataFrame(sc_31.transform(form_45.drop(['FORMATION'], axis=1)), form_45.index, mm_cols)
form_55_merge_mm = pd.DataFrame(sc_32.transform(form_55.drop(['FORMATION'], axis=1)), form_55.index, mm_cols)
form_56_merge_mm = pd.DataFrame(sc_33.transform(form_56.drop(['FORMATION'], axis=1)), form_56.index, mm_cols)
form_57_merge_mm = pd.DataFrame(sc_34.transform(form_57.drop(['FORMATION'], axis=1)), form_57.index, mm_cols)
form_58_merge_mm = pd.DataFrame(sc_35.transform(form_58.drop(['FORMATION'], axis=1)), form_58.index, mm_cols)
form_47_merge_mm = pd.DataFrame(sc_36.transform(form_47.drop(['FORMATION'], axis=1)), form_47.index, mm_cols)
form_48_merge_mm = pd.DataFrame(sc_37.transform(form_48.drop(['FORMATION'], axis=1)), form_48.index, mm_cols)
form_60_merge_mm = pd.DataFrame(sc_38.transform(form_60.drop(['FORMATION'], axis=1)), form_60.index, mm_cols)
form_53_merge_mm = pd.DataFrame(sc_39.transform(form_53.drop(['FORMATION'], axis=1)), form_53.index, mm_cols)
form_68_merge_mm = pd.DataFrame(sc_40.transform(form_68.drop(['FORMATION'], axis=1)), form_68.index, mm_cols)

form_54_merge_mm = pd.DataFrame(sc_41.transform(form_54.drop(['FORMATION'], axis=1)), form_54.index, mm_cols)

form_sec_merge_mm = pd.DataFrame(sc_sec.transform(form_merge_second.drop(['FORMATION'], axis=1)), form_merge_second.index, mm_cols)

In [57]:
form_mm = pd.concat([form_1_merge_mm,  form_2_merge_mm,  form_3_merge_mm,  form_4_merge_mm,  form_5_merge_mm,
                     form_42_merge_mm, form_19_merge_mm, form_6_merge_mm,  form_7_merge_mm,  form_8_merge_mm,
                     form_9_merge_mm,  form_10_merge_mm, form_11_merge_mm, form_12_merge_mm, form_13_merge_mm,
                     form_14_merge_mm, form_15_merge_mm, form_24_merge_mm, form_18_merge_mm, form_16_merge_mm,
                     form_21_merge_mm, form_22_merge_mm, form_40_merge_mm, form_33_merge_mm, form_23_merge_mm,
                     form_43_merge_mm, form_46_merge_mm, form_52_merge_mm, form_51_merge_mm, form_44_merge_mm,
                     form_45_merge_mm, form_55_merge_mm, form_56_merge_mm, form_57_merge_mm, form_58_merge_mm,
                     form_47_merge_mm, form_48_merge_mm, form_60_merge_mm, form_53_merge_mm, form_68_merge_mm,
                     form_54_merge_mm, 
                     form_sec_merge_mm])

form_mm.columns = ['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm', 
                   'NPHI_mm', 'SP_mm', 'PEF_mm', 
                   'Umaa_1_mm', 'Umaa_2_mm', 'Cmaa_1_mm', 'Cmaa_2_mm', 'Aaa_mm',
                   #'RMED_mm', 'RDEP_mm'
                   ]
form_mm.head(2)

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm,Umaa_1_mm,Umaa_2_mm,Cmaa_1_mm,Cmaa_2_mm,Aaa_mm
2464,0.170890,0.709759,0.551347,0.910346,0.774629,0.731788,0.338467,0.109019,0.264118,0.849450,0.558395,0.736560,0.232962
2465,0.143545,0.717318,0.568145,0.908929,0.768528,0.734493,0.337451,0.108690,0.263926,0.849461,0.553005,0.734563,0.231696


In [58]:
df_train_form_trans_mm = df_train_form_trans.merge(form_mm, left_index=True, right_index=True, how='outer')
df_train_form_trans_mm.head(2)

,WELL,DEPTH_MD,FORMATION,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE,Lithology Keys,Lithology Num,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,Umaa_1,Umaa_2,Cmaa_1,Cmaa_2,Aaa,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm,Umaa_1_mm,Umaa_2_mm,Cmaa_1_mm,Cmaa_2_mm,Aaa_mm
0,15/9-13,494.528,no_form,80.200851,161.13118,19.480835,-0.574928,1.884186,NaN,24.612379,20.915468,65000,1.0,Shale,3,80.200851,161.13118,19.480835,-0.574928,1.884186,0.629070,24.612379,20.915468,26.709728,-21.341009,0.626046,-1.833067,12.068311,0.119691,0.610691,0.685534,0.628666,0.370705,0.823229,0.749343,0.239484,0.226526,0.897033,0.241304,0.510664,0.191601
1,15/9-13,494.680,no_form,79.262886,160.60347,19.468800,-0.570188,1.889794,NaN,23.895531,19.383013,65000,1.0,Shale,3,79.262886,160.60347,19.468800,-0.570188,1.889794,0.636301,23.895531,19.383013,24.660790,-19.885044,0.617191,-1.842791,10.320867,0.118229,0.608232,0.685240,0.629206,0.373100,0.826679,0.748983,0.236103,0.224670,0.897739,0.239488,0.509581,0.188035


In [4]:
#df_train_form_trans_mm.to_csv('../df_train_form_trans_mm.csv')
df_train_form_trans_mm = pd.read_csv('../df_train_form_trans_mm.csv', index_col=0)
df_train_form_trans_mm.head(2)

,WELL,DEPTH_MD,FORMATION,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE,Lithology Keys,Lithology Num,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,Umaa_1,Umaa_2,Cmaa_1,Cmaa_2,Aaa,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm,Umaa_1_mm,Umaa_2_mm,Cmaa_1_mm,Cmaa_2_mm,Aaa_mm
0,15/9-13,494.528,no_form,80.200851,161.13118,19.480835,-0.574928,1.884186,NaN,24.612379,20.915468,65000,1.0,Shale,3,80.200851,161.13118,19.480835,-0.574928,1.884186,0.629070,24.612379,20.915468,26.709728,-21.341009,0.626046,-1.833067,12.068311,0.119691,0.610691,0.685534,0.628666,0.370705,0.823229,0.749343,0.239484,0.226526,0.897033,0.241304,0.510664,0.191601
1,15/9-13,494.680,no_form,79.262886,160.60347,19.468800,-0.570188,1.889794,NaN,23.895531,19.383013,65000,1.0,Shale,3,79.262886,160.60347,19.468800,-0.570188,1.889794,0.636301,23.895531,19.383013,24.660790,-19.885044,0.617191,-1.842791,10.320867,0.118229,0.608232,0.685240,0.629206,0.373100,0.826679,0.748983,0.236103,0.224670,0.897739,0.239488,0.509581,0.188035


In [ ]:
# Duplicate Lithology

In [5]:
sandstone = df_train_form_trans_mm[df_train_form_trans_mm['Lithology Keys'] == 'Sandstone']
sandstone_shale = df_train_form_trans_mm[df_train_form_trans_mm['Lithology Keys'] == 'Sandstone/Shale']
limestone = df_train_form_trans_mm[df_train_form_trans_mm['Lithology Keys'] == 'Limestone']
marl = df_train_form_trans_mm[df_train_form_trans_mm['Lithology Keys'] == 'Marl']
tuff = df_train_form_trans_mm[df_train_form_trans_mm['Lithology Keys'] == 'Tuff']
chalk = df_train_form_trans_mm[df_train_form_trans_mm['Lithology Keys'] == 'Chalk']
halite = df_train_form_trans_mm[df_train_form_trans_mm['Lithology Keys'] == 'Halite']
coal = df_train_form_trans_mm[df_train_form_trans_mm['Lithology Keys'] == 'Coal']
dolomite = df_train_form_trans_mm[df_train_form_trans_mm['Lithology Keys'] == 'Dolomite']
anhydrite = df_train_form_trans_mm[df_train_form_trans_mm['Lithology Keys'] == 'Anhydrite']
basement = df_train_form_trans_mm[df_train_form_trans_mm['Lithology Keys'] == 'Basement']

In [6]:
df_sandstone = df_train_form_trans_mm.append([sandstone]*2, ignore_index=True)

In [7]:
df_sandstone_shale = df_sandstone.append([sandstone_shale]*2, ignore_index=True)

In [8]:
df_limestone = df_sandstone_shale.append([limestone]*5, ignore_index=True)

In [9]:
df_marl      = df_limestone.append([marl]*7, ignore_index=True)

In [10]:
df_tuff      = df_marl.append([tuff]*10, ignore_index=True)

In [11]:
df_chalk     = df_tuff.append([chalk]*10, ignore_index=True)

In [12]:
df_halite    = df_chalk.append([halite]*15, ignore_index=True)

In [13]:
df_coal      = df_halite.append([coal]*15, ignore_index=True)

In [14]:
df_dolomite  = df_coal.append([dolomite]*30, ignore_index=True)

In [15]:
df_anhydrite = df_dolomite.append([anhydrite]*30, ignore_index=True)

In [16]:
df_train_form_trans_mm  = df_anhydrite.append([basement]*30, ignore_index=True)

In [17]:
df_train_form_trans_mm = df_basement.copy()

In [ ]:
df_form_trans = df_train_form_trans_merge_mm_copylith.copy()

In [ ]:
df_form_trans['Lithology Keys'].unique()

array(['Shale', 'Sandstone', 'Sandstone/Shale', 'Limestone', 'Tuff',
       'Marl', 'Anhydrite', 'Dolomite', 'Chalk', 'Coal', 'Halite',
       'Basement'], dtype=object)

In [27]:
df_train_form_trans_mm.to_csv('../df_train_form_trans_mm_dense.csv')

### Training

In [17]:
lithology_numbers = {30000: 0,           65030: 1,                 65000: 2,           80000: 3,      74000: 4,          70000: 5, 
                     70032: 6,           88000: 7,                 86000: 8,           99000: 9,      90000: 10,         93000: 11}
lithology_keys    = {30000: 'Sandstone', 65030: 'Sandstone/Shale', 65000: 'Shale',     80000: 'Marl', 74000: 'Dolomite', 70000: 'Limestone',
                     70032: 'Chalk',     88000: 'Halite',          86000: 'Anhydrite', 99000: 'Tuff', 90000: 'Coal',     93000: 'Basement'}
lithology_int     = {0: 'Sandstone',     1: 'Sandstone/Shale',     2: 'Shale',         3: 'Marl',     4: 'Dolomite',     5: 'Limestone',
                     6: 'Chalk',         7: 'Halite',              8: 'Anhydrite',     9: 'Tuff',     10: 'Coal',        11: 'Basement'}

In [ ]:
df_train_form_trans_mm.columns

In [18]:
df_X = df_train_form_trans_mm[['GR_new',    'DTC_new',   'CALI_new',  'DRHO_new',  'RHOB_new', 'NPHI_new', 'SP_new', 'PEF_new', 
                               'Umaa_1',    
                               'Umaa_2',    
                               'Cmaa_1',    
                               'Cmaa_2',    
                               'Aaa',
                               #'GR_mm',     'DTC_mm',    'CALI_mm',   'DRHO_mm',   'RHOB_mm',  'NPHI_mm',  'SP_mm',  'PEF_mm', 
                               #'Umaa_1_mm', 'Umaa_2_mm', 'Cmaa_1_mm', 'Cmaa_2_mm', 'Aaa_mm'
                               ]]
df_X.head(2)

,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,Umaa_1,Umaa_2,Cmaa_1,Cmaa_2,Aaa
0,80.200851,161.13118,19.480835,-0.574928,1.884186,0.629070,24.612379,20.915468,26.709728,-21.341009,0.626046,-1.833067,12.068311
1,79.262886,160.60347,19.468800,-0.570188,1.889794,0.636301,23.895531,19.383013,24.660790,-19.885044,0.617191,-1.842791,10.320867


In [19]:
#df_y = pd.get_dummies(df_train_scaled['Lithology Keys'])
#df_y = df_train_scaled['FORCE_2020_LITHOFACIES_LITHOLOGY']

df_y = df_train_form_trans_mm['FORCE_2020_LITHOFACIES_LITHOLOGY']
y_int = df_y.map(lithology_numbers)
y_str = df_y.map(lithology_keys)
y_str.value_counts()

Shale              720803
Sandstone          506811
Sandstone/Shale    451365
Limestone          337920
Marl               266632
Tuff               167695
Halite             131408
Chalk              115643
Coal                61120
Dolomite            52328
Anhydrite           33635
Basement             3193
Name: FORCE_2020_LITHOFACIES_LITHOLOGY, dtype: int64

In [20]:
A = np.load('penalty_matrix.npy')

def score(y_true, y_pred):
    S = 0.0
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    for i in range(0, y_true.shape[0]):
        S -= A[y_true[i], y_pred[i]]
    return S/y_true.shape[0]

In [21]:
X = df_X.copy()
#y = df_y.copy()
y = y_int.copy()

In [22]:
# Split into training, validation, and test sets (80/20)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.0001, random_state=90, stratify=y)
#X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=2)

#print("Train set size: {0}, Val set size: {1}, Test set size: {2}".format(len(X_train), len(X_val), len(X_test)))
print("Train set size: {0}, Test set size: {1}".format(len(X_train), len(X_test)))

Train set size: 2848268, Test set size: 285


In [23]:
X_train.head(2)

,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,Umaa_1,Umaa_2,Cmaa_1,Cmaa_2,Aaa
1663875,103.257843,132.224701,12.779608,0.055886,2.232710,0.440304,87.613411,4.413873,7.599462,-1.376832,1.352101,-0.824723,46.608335
1581649,42.160271,109.506477,20.331656,0.104135,2.278053,0.397472,34.299679,7.858446,14.585986,-1.374041,1.483109,-0.690808,45.498040


In [24]:
# raw, all row; NPHI, SP, PEF; Umaa all
lgbmclass = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                           importance_type='split', learning_rate=0.1, max_depth=-1,
                           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                           n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                           subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model_1 = make_pipeline(lgbmclass)
model_1.fit(X_train, y_train)
y_pred_test = model_1.predict(X_test)

print('Model Score                   : ', model_1.score(X_train, y_train))
print('Root Mean Squared error       : ', sqrt(mean_squared_error(y_test.values, y_pred_test)))
print('Custom Score (penalty_matrix) : ', score(y_test.values, y_pred_test))
# Xeek score: 0.

Model Score                   :  0.7819387080148357
Root Mean Squared error       :  1.5649673501442591
Custom Score (penalty_matrix) :  -0.5364035087719298


In [ ]:
# raw mm, all row; NPHI, SP, PEF; Umaa all
lgbmclass = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                           importance_type='split', learning_rate=0.1, max_depth=-1,
                           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                           n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                           subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model_1 = make_pipeline(lgbmclass)
model_1.fit(X_train, y_train)
y_pred_test = model_1.predict(X_test)

print('Model Score                   : ', model_1.score(X_train, y_train))
print('Root Mean Squared error       : ', sqrt(mean_squared_error(y_test.values, y_pred_test)))
print('Custom Score (penalty_matrix) : ', score(y_test.values, y_pred_test))
# Xeek score: 0.7545

Model Score                   :  0.8123655900197626
Root Mean Squared error       :  0.8972651479706051
Custom Score (penalty_matrix) :  -0.7129237288135594


In [ ]:
# mm, all row; NPHI, SP, PEF; Umaa_mm all
lgbmclass = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                           importance_type='split', learning_rate=0.1, max_depth=-1,
                           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                           n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                           subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model_1 = make_pipeline(lgbmclass)
model_1.fit(X_train, y_train)
y_pred_test = model_1.predict(X_test)

print('Model Score                   : ', model_1.score(X_train, y_train))
print('Root Mean Squared error       : ', sqrt(mean_squared_error(y_test.values, y_pred_test)))
print('Custom Score (penalty_matrix) : ', score(y_test.values, y_pred_test))
# Xeek score: 0.815

Model Score                   :  0.81019623323106
Root Mean Squared error       :  1.2178057511186369
Custom Score (penalty_matrix) :  -0.725635593220339


In [ ]:
# raw, all row; NPHI, SP, PEF;
lgbmclass = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                           importance_type='split', learning_rate=0.1, max_depth=-1,
                           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                           n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                           subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model_1 = make_pipeline(lgbmclass)
model_1.fit(X_train, y_train)
y_pred_test = model_1.predict(X_test)

print('Model Score                   : ', model_1.score(X_train, y_train))
print('Root Mean Squared error       : ', sqrt(mean_squared_error(y_test.values, y_pred_test)))
print('Custom Score (penalty_matrix) : ', score(y_test.values, y_pred_test))
# Xeek score: 0.7248

Model Score                   :  0.819388871943014
Root Mean Squared error       :  0.7972410051791008
Custom Score (penalty_matrix) :  -0.5889830508474576


In [ ]:
# raw, all row; NPHI, SP, PEF; cmaa-umaa 1
lgbmclass = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                           importance_type='split', learning_rate=0.1, max_depth=-1,
                           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                           n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                           subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model_1 = make_pipeline(lgbmclass)
model_1.fit(X_train, y_train)
y_pred_test = model_1.predict(X_test)

print('Model Score                   : ', model_1.score(X_train, y_train))
print('Root Mean Squared error       : ', sqrt(mean_squared_error(y_test.values, y_pred_test)))
print('Custom Score (penalty_matrix) : ', score(y_test.values, y_pred_test))
# Xeek score: 0.7507

Model Score                   :  0.8178654520319243
Root Mean Squared error       :  0.7865392983167531
Custom Score (penalty_matrix) :  -0.6101694915254238


In [ ]:
# raw, all row; NPHI, SP, PEF; cmaa-umaa 2
lgbmclass = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                           importance_type='split', learning_rate=0.1, max_depth=-1,
                           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                           n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                           subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model_1 = make_pipeline(lgbmclass)
model_1.fit(X_train, y_train)
y_pred_test = model_1.predict(X_test)

print('Model Score                   : ', model_1.score(X_train, y_train))
print('Root Mean Squared error       : ', sqrt(mean_squared_error(y_test.values, y_pred_test)))
print('Custom Score (penalty_matrix) : ', score(y_test.values, y_pred_test))
# Xeek score: 0.7405

Model Score                   :  0.8144554863195524
Root Mean Squared error       :  0.8182246179526309
Custom Score (penalty_matrix) :  -0.621822033898305


In [ ]:
# raw, all row; NPHI, SP, PEF; cmaa 1 - Aaa
lgbmclass = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                           importance_type='split', learning_rate=0.1, max_depth=-1,
                           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                           n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                           subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model_1 = make_pipeline(lgbmclass)
model_1.fit(X_train, y_train)
y_pred_test = model_1.predict(X_test)

print('Model Score                   : ', model_1.score(X_train, y_train))
print('Root Mean Squared error       : ', sqrt(mean_squared_error(y_test.values, y_pred_test)))
print('Custom Score (penalty_matrix) : ', score(y_test.values, y_pred_test))
# Xeek score: 0.7379

Model Score                   :  0.8106943565110181
Root Mean Squared error       :  1.0735652625161434
Custom Score (penalty_matrix) :  -0.6652542372881356


In [ ]:
# raw, all row; NPHI, SP, PEF; cmaa 2 - Aaa
lgbmclass = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                           importance_type='split', learning_rate=0.1, max_depth=-1,
                           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                           n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                           subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model_1 = make_pipeline(lgbmclass)
model_1.fit(X_train, y_train)
y_pred_test = model_1.predict(X_test)

print('Model Score                   : ', model_1.score(X_train, y_train))
print('Root Mean Squared error       : ', sqrt(mean_squared_error(y_test.values, y_pred_test)))
print('Custom Score (penalty_matrix) : ', score(y_test.values, y_pred_test))
# Xeek score: 0.7415

Model Score                   :  0.8019297791425615
Root Mean Squared error       :  1.089238577156585
Custom Score (penalty_matrix) :  -0.7341101694915254


### df_test ALL

In [25]:
pickle.dump(model_1, open('../Submission/model_1_umaa.pkl', 'wb'))
#pickle.dump(scaler, open('../Submission/scaler_xgb_3.pkl', 'wb'))
#pickle.dump(imputer_all, open('../Submission/imputer_all.pkl', 'wb'))

In [59]:
model_1_umaa = pickle.load(open('../Submission/model_1_umaa.pkl', 'rb'))
#scaler = pickle.load(open('../Submission/scaler_lgbm_5.pkl', 'rb'))
#imputer_all = pickle.load(open('../Submission/imputer_all.pkl', 'rb'))

In [60]:
df_test = pd.read_csv('./test.csv', sep=';')
df_test = df_test[['WELL', 'DEPTH_MD', 'FORMATION',
                   'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 'NPHI', 'SP', 'PEF']]
df_test.head(2)

,WELL,DEPTH_MD,FORMATION,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF
0,15/9-14,480.628001,NaN,72.078377,174.347092,19.2031,-0.538873,2.171173,0.551575,35.525719,55.815659
1,15/9-14,480.780001,NaN,72.147697,176.112915,19.2031,-0.539232,2.171840,0.547566,36.158520,41.862537


In [61]:
df_test = df_test[['FORMATION', 'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 
                   'NPHI', 'SP', 'PEF',
                   #'RMED', 'RDEP'
                   ]]

In [62]:
df_test['FORMATION'].fillna('no_form', inplace=True)

In [63]:
test_form = df_test['FORMATION'].unique()

In [64]:
test_form_1  = df_test[df_test['FORMATION'] == test_form[1]]
test_form_2  = df_test[df_test['FORMATION'] == test_form[2]]
test_form_3  = df_test[df_test['FORMATION'] == test_form[3]]
test_form_4  = df_test[df_test['FORMATION'] == test_form[4]]
test_form_5  = df_test[df_test['FORMATION'] == test_form[5]]
test_form_6  = df_test[df_test['FORMATION'] == test_form[6]]
test_form_7  = df_test[df_test['FORMATION'] == test_form[7]]
test_form_8  = df_test[df_test['FORMATION'] == test_form[8]]
test_form_9  = df_test[df_test['FORMATION'] == test_form[9]]
test_form_10  = df_test[df_test['FORMATION'] == test_form[10]]

test_form_11  = df_test[df_test['FORMATION'] == test_form[11]]
test_form_12  = df_test[df_test['FORMATION'] == test_form[12]]
test_form_13  = df_test[df_test['FORMATION'] == test_form[13]]
test_form_14  = df_test[df_test['FORMATION'] == test_form[14]]
test_form_15  = df_test[df_test['FORMATION'] == test_form[15]]
test_form_16  = df_test[df_test['FORMATION'] == test_form[16]]
test_form_17  = df_test[df_test['FORMATION'] == test_form[17]]
test_form_18  = df_test[df_test['FORMATION'] == test_form[18]]
test_form_19  = df_test[df_test['FORMATION'] == test_form[19]]
test_form_20  = df_test[df_test['FORMATION'] == test_form[20]]

test_form_21  = df_test[df_test['FORMATION'] == test_form[21]]
test_form_22  = df_test[df_test['FORMATION'] == test_form[22]]
test_form_23  = df_test[df_test['FORMATION'] == test_form[23]]
test_form_24  = df_test[df_test['FORMATION'] == test_form[24]]
test_form_25  = df_test[df_test['FORMATION'] == test_form[25]]
test_form_26  = df_test[df_test['FORMATION'] == test_form[26]]
test_form_27  = df_test[df_test['FORMATION'] == test_form[27]]
test_form_28  = df_test[df_test['FORMATION'] == test_form[28]]
test_form_29  = df_test[df_test['FORMATION'] == test_form[29]]
test_form_30  = df_test[df_test['FORMATION'] == test_form[30]]

test_form_31  = df_test[df_test['FORMATION'] == test_form[31]]
test_form_32  = df_test[df_test['FORMATION'] == test_form[32]]
test_form_33  = df_test[df_test['FORMATION'] == test_form[33]]
test_form_34  = df_test[df_test['FORMATION'] == test_form[34]]
test_form_35  = df_test[df_test['FORMATION'] == test_form[35]]
test_form_36  = df_test[df_test['FORMATION'] == test_form[36]]
test_form_37  = df_test[df_test['FORMATION'] == test_form[37]]
test_form_38  = df_test[df_test['FORMATION'] == test_form[38]]
test_form_39  = df_test[df_test['FORMATION'] == test_form[39]]
test_form_40  = df_test[df_test['FORMATION'] == test_form[40]]

test_form_41  = df_test[df_test['FORMATION'] == test_form[41]]

test_form_sec  = df_test[df_test['FORMATION'] == test_form[0]]

In [65]:
test_form_merge = pd.concat([test_form_1,  test_form_2,  test_form_3,  test_form_4,  test_form_5,  
                             test_form_6,  test_form_7,  test_form_8,  test_form_9,  test_form_10,
                             test_form_11, test_form_12, test_form_13, test_form_14, test_form_15, 
                             test_form_16, test_form_17, test_form_18, test_form_19, test_form_20,
                             test_form_21, test_form_22, test_form_23, test_form_24, test_form_25, 
                             test_form_26, test_form_27, test_form_28, test_form_29, test_form_30,
                             test_form_31, test_form_32, test_form_33, test_form_34, test_form_35, 
                             test_form_36, test_form_37, test_form_38, test_form_39, test_form_40, 
                             test_form_41,
                             test_form_sec])

In [ ]:
test_form_1_trans  = imputer_1.transform(test_form_1.drop(['FORMATION'], axis=1))
test_form_2_trans  = imputer_2.transform(test_form_2.drop(['FORMATION'], axis=1))
test_form_3_trans  = imputer_3.transform(test_form_3.drop(['FORMATION'], axis=1))
test_form_4_trans  = imputer_4.transform(test_form_4.drop(['FORMATION'], axis=1))
test_form_5_trans  = imputer_5.transform(test_form_5.drop(['FORMATION'], axis=1))
test_form_6_trans  = imputer_6.transform(test_form_6.drop(['FORMATION'], axis=1))
test_form_7_trans  = imputer_7.transform(test_form_7.drop(['FORMATION'], axis=1))
test_form_8_trans  = imputer_8.transform(test_form_8.drop(['FORMATION'], axis=1))
test_form_9_trans  = imputer_9.transform(test_form_9.drop(['FORMATION'], axis=1))
test_form_10_trans = imputer_10.transform(test_form_10.drop(['FORMATION'], axis=1))

test_form_11_trans  = imputer_11.transform(test_form_11.drop(['FORMATION'], axis=1))
test_form_12_trans  = imputer_12.transform(test_form_12.drop(['FORMATION'], axis=1))
test_form_13_trans  = imputer_13.transform(test_form_13.drop(['FORMATION'], axis=1))
test_form_14_trans  = imputer_14.transform(test_form_14.drop(['FORMATION'], axis=1))
test_form_15_trans  = imputer_15.transform(test_form_15.drop(['FORMATION'], axis=1))
test_form_16_trans  = imputer_16.transform(test_form_16.drop(['FORMATION'], axis=1))
test_form_17_trans  = imputer_17.transform(test_form_17.drop(['FORMATION'], axis=1))
test_form_18_trans  = imputer_18.transform(test_form_18.drop(['FORMATION'], axis=1))
test_form_19_trans  = imputer_19.transform(test_form_19.drop(['FORMATION'], axis=1))
test_form_20_trans  = imputer_20.transform(test_form_20.drop(['FORMATION'], axis=1))

test_form_21_trans  = imputer_21.transform(test_form_21.drop(['FORMATION'], axis=1))
test_form_22_trans  = imputer_22.transform(test_form_22.drop(['FORMATION'], axis=1))
test_form_23_trans  = imputer_23.transform(test_form_23.drop(['FORMATION'], axis=1))
test_form_24_trans  = imputer_24.transform(test_form_24.drop(['FORMATION'], axis=1))
test_form_25_trans  = imputer_25.transform(test_form_25.drop(['FORMATION'], axis=1))
test_form_26_trans  = imputer_26.transform(test_form_26.drop(['FORMATION'], axis=1))
test_form_27_trans  = imputer_27.transform(test_form_27.drop(['FORMATION'], axis=1))
test_form_28_trans  = imputer_28.transform(test_form_28.drop(['FORMATION'], axis=1))
test_form_29_trans  = imputer_29.transform(test_form_29.drop(['FORMATION'], axis=1))
test_form_30_trans = imputer_30.transform(test_form_30.drop(['FORMATION'], axis=1))

test_form_31_trans  = imputer_31.transform(test_form_31.drop(['FORMATION'], axis=1))
test_form_32_trans  = imputer_32.transform(test_form_32.drop(['FORMATION'], axis=1))
test_form_33_trans  = imputer_33.transform(test_form_33.drop(['FORMATION'], axis=1))
test_form_34_trans  = imputer_34.transform(test_form_34.drop(['FORMATION'], axis=1))
test_form_35_trans  = imputer_35.transform(test_form_35.drop(['FORMATION'], axis=1))
test_form_36_trans  = imputer_36.transform(test_form_36.drop(['FORMATION'], axis=1))
test_form_37_trans  = imputer_37.transform(test_form_37.drop(['FORMATION'], axis=1))
test_form_38_trans  = imputer_38.transform(test_form_38.drop(['FORMATION'], axis=1))
test_form_39_trans  = imputer_39.transform(test_form_39.drop(['FORMATION'], axis=1))
test_form_40_trans  = imputer_40.transform(test_form_40.drop(['FORMATION'], axis=1))

test_form_41_trans  = imputer_41.transform(test_form_41.drop(['FORMATION'], axis=1))

test_form_sec_trans  = imputer_sec.transform(test_form_sec.drop(['FORMATION'], axis=1))

In [67]:
df_test_form_1_trans = pd.DataFrame(test_form_1_trans, columns=test_form_1.columns[1:], index=test_form_1.index)
df_test_form_2_trans = pd.DataFrame(test_form_2_trans, columns=test_form_2.columns[1:], index=test_form_2.index)
df_test_form_3_trans = pd.DataFrame(test_form_3_trans, columns=test_form_3.columns[1:], index=test_form_3.index)
df_test_form_4_trans = pd.DataFrame(test_form_4_trans, columns=test_form_4.columns[1:], index=test_form_4.index)
df_test_form_5_trans = pd.DataFrame(test_form_5_trans, columns=test_form_5.columns[1:], index=test_form_5.index)
df_test_form_6_trans = pd.DataFrame(test_form_6_trans, columns=test_form_6.columns[1:], index=test_form_6.index)
df_test_form_7_trans = pd.DataFrame(test_form_7_trans, columns=test_form_7.columns[1:], index=test_form_7.index)
df_test_form_8_trans = pd.DataFrame(test_form_8_trans, columns=test_form_8.columns[1:], index=test_form_8.index)
df_test_form_9_trans = pd.DataFrame(test_form_9_trans, columns=test_form_9.columns[1:], index=test_form_9.index)
df_test_form_10_trans = pd.DataFrame(test_form_10_trans, columns=test_form_10.columns[1:], index=test_form_10.index)

df_test_form_11_trans = pd.DataFrame(test_form_11_trans, columns=test_form_11.columns[1:], index=test_form_11.index)
df_test_form_12_trans = pd.DataFrame(test_form_12_trans, columns=test_form_12.columns[1:], index=test_form_12.index)
df_test_form_13_trans = pd.DataFrame(test_form_13_trans, columns=test_form_13.columns[1:], index=test_form_13.index)
df_test_form_14_trans = pd.DataFrame(test_form_14_trans, columns=test_form_14.columns[1:], index=test_form_14.index)
df_test_form_15_trans = pd.DataFrame(test_form_15_trans, columns=test_form_15.columns[1:], index=test_form_15.index)
df_test_form_16_trans = pd.DataFrame(test_form_16_trans, columns=test_form_16.columns[1:], index=test_form_16.index)
df_test_form_17_trans = pd.DataFrame(test_form_17_trans, columns=test_form_17.columns[1:], index=test_form_17.index)
df_test_form_18_trans = pd.DataFrame(test_form_18_trans, columns=test_form_18.columns[1:], index=test_form_18.index)
df_test_form_19_trans = pd.DataFrame(test_form_19_trans, columns=test_form_19.columns[1:], index=test_form_19.index)
df_test_form_20_trans = pd.DataFrame(test_form_20_trans, columns=test_form_20.columns[1:], index=test_form_20.index)

df_test_form_21_trans = pd.DataFrame(test_form_21_trans, columns=test_form_21.columns[1:], index=test_form_21.index)
df_test_form_22_trans = pd.DataFrame(test_form_22_trans, columns=test_form_22.columns[1:], index=test_form_22.index)
df_test_form_23_trans = pd.DataFrame(test_form_23_trans, columns=test_form_23.columns[1:], index=test_form_23.index)
df_test_form_24_trans = pd.DataFrame(test_form_24_trans, columns=test_form_24.columns[1:], index=test_form_24.index)
df_test_form_25_trans = pd.DataFrame(test_form_25_trans, columns=test_form_25.columns[1:], index=test_form_25.index)
df_test_form_26_trans = pd.DataFrame(test_form_26_trans, columns=test_form_26.columns[1:], index=test_form_26.index)
df_test_form_27_trans = pd.DataFrame(test_form_27_trans, columns=test_form_27.columns[1:], index=test_form_27.index)
df_test_form_28_trans = pd.DataFrame(test_form_28_trans, columns=test_form_28.columns[1:], index=test_form_28.index)
df_test_form_29_trans = pd.DataFrame(test_form_29_trans, columns=test_form_29.columns[1:], index=test_form_29.index)
df_test_form_30_trans = pd.DataFrame(test_form_30_trans, columns=test_form_30.columns[1:], index=test_form_30.index)

df_test_form_31_trans = pd.DataFrame(test_form_31_trans, columns=test_form_31.columns[1:], index=test_form_31.index)
df_test_form_32_trans = pd.DataFrame(test_form_32_trans, columns=test_form_32.columns[1:], index=test_form_32.index)
df_test_form_33_trans = pd.DataFrame(test_form_33_trans, columns=test_form_33.columns[1:], index=test_form_33.index)
df_test_form_34_trans = pd.DataFrame(test_form_34_trans, columns=test_form_34.columns[1:], index=test_form_34.index)
df_test_form_35_trans = pd.DataFrame(test_form_35_trans, columns=test_form_35.columns[1:], index=test_form_35.index)
df_test_form_36_trans = pd.DataFrame(test_form_36_trans, columns=test_form_36.columns[1:], index=test_form_36.index)
df_test_form_37_trans = pd.DataFrame(test_form_37_trans, columns=test_form_37.columns[1:], index=test_form_37.index)
df_test_form_38_trans = pd.DataFrame(test_form_38_trans, columns=test_form_38.columns[1:], index=test_form_38.index)
df_test_form_39_trans = pd.DataFrame(test_form_39_trans, columns=test_form_39.columns[1:], index=test_form_39.index)
df_test_form_40_trans = pd.DataFrame(test_form_40_trans, columns=test_form_40.columns[1:], index=test_form_40.index)

df_test_form_41_trans = pd.DataFrame(test_form_41_trans, columns=test_form_41.columns[1:], index=test_form_41.index)

df_test_form_sec_trans = pd.DataFrame(test_form_sec_trans, columns=test_form_sec.columns[1:], index=test_form_sec.index)

In [68]:
df_test_form_trans = pd.concat([df_test_form_1_trans,  df_test_form_2_trans,  df_test_form_3_trans,  df_test_form_4_trans,  df_test_form_5_trans,
                                df_test_form_6_trans,  df_test_form_7_trans,  df_test_form_8_trans,  df_test_form_9_trans,  df_test_form_10_trans,
                                df_test_form_11_trans, df_test_form_12_trans, df_test_form_13_trans, df_test_form_14_trans, df_test_form_15_trans,
                                df_test_form_16_trans, df_test_form_17_trans, df_test_form_18_trans, df_test_form_19_trans, df_test_form_20_trans,
                                df_test_form_21_trans, df_test_form_22_trans, df_test_form_23_trans, df_test_form_24_trans, df_test_form_25_trans,
                                df_test_form_26_trans, df_test_form_27_trans, df_test_form_28_trans, df_test_form_29_trans, df_test_form_30_trans,
                                df_test_form_31_trans, df_test_form_32_trans, df_test_form_33_trans, df_test_form_34_trans, df_test_form_35_trans,
                                df_test_form_36_trans, df_test_form_37_trans, df_test_form_38_trans, df_test_form_39_trans, df_test_form_40_trans,
                                df_test_form_41_trans,
                                df_test_form_sec_trans])

df_test_form_trans.columns = ['GR_new', 'DTC_new', 'CALI_new', 'DRHO_new', 'RHOB_new', 
                              'NPHI_new', 'SP_new', 'PEF_new',
                              #'RMED_new', 'RDEP_new'
                              ]
df_test_form_trans.head(2)

,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new
2924,68.656174,142.063416,17.359400,0.012434,2.106125,0.488468,34.949097,2.085944
2925,64.126785,144.903183,17.348019,0.009665,2.110649,0.482300,35.064674,2.041033


In [82]:
df_test_form_trans_index = df_test.merge(df_test_form_trans, left_index=True, right_index=True, how='outer').drop(['GR', 'DTC', 'CALI',
                                                                                                                   'DRHO', 'RHOB', 'NPHI', 'SP', 'PEF'],
                                                                                                                  axis=1)
df_test_form_trans_index.head(2)

,FORMATION,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new
0,no_form,72.078377,174.347092,19.2031,-0.538873,2.171173,0.551575,35.525719,55.815659
1,no_form,72.147697,176.112915,19.2031,-0.539232,2.171840,0.547566,36.158520,41.862537


In [83]:
df_test_form_trans_index['Umaa_1'] = (df_test_form_trans_index['PEF_new']*(((df_test_form_trans_index['RHOB_new']+0.1883)/1.0704)-(df_test_form_trans_index['NPHI_new']*1)))/1-df_test_form_trans_index['NPHI_new']

In [84]:
df_test_form_trans_index['Umaa_2'] = (df_test_form_trans_index['PEF_new']*(((df_test_form_trans_index['DRHO_new']+0.1883)/1.0704)-(df_test_form_trans_index['NPHI_new']*1)))/1-df_test_form_trans_index['NPHI_new']

In [85]:
df_test_form_trans_index['Cmaa_1'] = (df_test_form_trans_index['RHOB_new']-(df_test_form_trans_index['NPHI_new']*1))/1-df_test_form_trans_index['NPHI_new']

In [86]:
df_test_form_trans_index['Cmaa_2'] = (df_test_form_trans_index['DRHO_new']-(df_test_form_trans_index['NPHI_new']*1))/1-df_test_form_trans_index['NPHI_new']

In [87]:
df_test_form_trans_index['Aaa']    = df_test_form_trans_index['DTC_new']-((df_test_form_trans_index['NPHI_new']*df_test_form_trans_index['DTC_new'])/0.68)

In [88]:
df_test_form_trans_index.head(2)

,FORMATION,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,Umaa_1,Umaa_2,Cmaa_1,Cmaa_2,Aaa
0,no_form,72.078377,174.347092,19.2031,-0.538873,2.171173,0.551575,35.525719,55.815659,91.695822,-49.618614,1.068022,-1.642023,32.927196
1,no_form,72.147697,176.112915,19.2031,-0.539232,2.171840,0.547566,36.158520,41.862537,68.833219,-37.194772,1.076708,-1.634365,34.298981


In [90]:
test_form = df_test_form_trans_index['FORMATION'].unique()

In [ ]:
df_train_form_trans_split = df_train_form_trans[['FORMATION', 'GR_new', 'DTC_new', 'CALI_new', 'DRHO_new', 'RHOB_new', 'NPHI_new',
                                                 'SP_new', 'PEF_new', 'Umaa_1', 'Umaa_2', 'Cmaa_1', 'Cmaa_2', 'Aaa']]

In [91]:
test_form_1  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[1]]
test_form_2  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[2]]
test_form_3  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[3]]
test_form_4  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[4]]
test_form_5  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[5]]
test_form_6  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[6]]
test_form_7  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[7]]
test_form_8  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[8]]
test_form_9  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[9]]
test_form_10  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[10]]

test_form_11  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[11]]
test_form_12  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[12]]
test_form_13  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[13]]
test_form_14  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[14]]
test_form_15  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[15]]
test_form_16  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[16]]
test_form_17  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[17]]
test_form_18  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[18]]
test_form_19  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[19]]
test_form_20  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[20]]

test_form_21  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[21]]
test_form_22  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[22]]
test_form_23  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[23]]
test_form_24  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[24]]
test_form_25  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[25]]
test_form_26  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[26]]
test_form_27  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[27]]
test_form_28  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[28]]
test_form_29  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[29]]
test_form_30  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[30]]

test_form_31  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[31]]
test_form_32  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[32]]
test_form_33  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[33]]
test_form_34  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[34]]
test_form_35  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[35]]
test_form_36  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[36]]
test_form_37  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[37]]
test_form_38  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[38]]
test_form_39  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[39]]
test_form_40  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[40]]

test_form_41  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[41]]

test_form_sec  = df_test_form_trans_index[df_test_form_trans_index['FORMATION'] == test_form[0]]

In [92]:
test_form_merge = pd.concat([test_form_1,  test_form_2,  test_form_3,  test_form_4,  test_form_5,  
                             test_form_6,  test_form_7,  test_form_8,  test_form_9,  test_form_10,
                             test_form_11, test_form_12, test_form_13, test_form_14, test_form_15, 
                             test_form_16, test_form_17, test_form_18, test_form_19, test_form_20,
                             test_form_21, test_form_22, test_form_23, test_form_24, test_form_25, 
                             test_form_26, test_form_27, test_form_28, test_form_29, test_form_30,
                             test_form_31, test_form_32, test_form_33, test_form_34, test_form_35, 
                             test_form_36, test_form_37, test_form_38, test_form_39, test_form_40, 
                             test_form_41,
                             test_form_sec])

In [93]:
mm_cols = ['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm', 
           'NPHI_mm', 'SP_mm', 'PEF_mm',
           'Umaa_1_mm', 'Umaa_2_mm', 'Cmaa_1_mm', 'Cmaa_2_mm', 'Aaa_mm'
           #'RMED_mm', 'RDEP_mm'
           ]
#std_cols = ['GR_std', 'DTC_std', 'CALI_std', 'DRHO_std', 'RHOB_std', 'RMED_std', 'RDEP_std']

In [ ]:
df_test_form_1_trans

In [94]:
test_form_1_merge_mm   = pd.DataFrame(sc_1.transform(test_form_1.drop(['FORMATION'], axis=1)), test_form_1.index, mm_cols)
test_form_2_merge_mm   = pd.DataFrame(sc_2.transform(test_form_2.drop(['FORMATION'], axis=1)), test_form_2.index, mm_cols)
test_form_3_merge_mm   = pd.DataFrame(sc_3.transform(test_form_3.drop(['FORMATION'], axis=1)), test_form_3.index, mm_cols)
test_form_4_merge_mm   = pd.DataFrame(sc_4.transform(test_form_4.drop(['FORMATION'], axis=1)), test_form_4.index, mm_cols)
test_form_5_merge_mm   = pd.DataFrame(sc_5.transform(test_form_5.drop(['FORMATION'], axis=1)), test_form_5.index, mm_cols)
test_form_6_merge_mm   = pd.DataFrame(sc_6.transform(test_form_6.drop(['FORMATION'], axis=1)), test_form_6.index, mm_cols)
test_form_7_merge_mm   = pd.DataFrame(sc_7.transform(test_form_7.drop(['FORMATION'], axis=1)), test_form_7.index, mm_cols)
test_form_8_merge_mm   = pd.DataFrame(sc_8.transform(test_form_8.drop(['FORMATION'], axis=1)), test_form_8.index, mm_cols)
test_form_9_merge_mm   = pd.DataFrame(sc_9.transform(test_form_9.drop(['FORMATION'], axis=1)), test_form_9.index, mm_cols)
test_form_10_merge_mm  = pd.DataFrame(sc_10.transform(test_form_10.drop(['FORMATION'], axis=1)), test_form_10.index, mm_cols)

test_form_11_merge_mm  = pd.DataFrame(sc_11.transform(test_form_11.drop(['FORMATION'], axis=1)), test_form_11.index, mm_cols)
test_form_12_merge_mm  = pd.DataFrame(sc_12.transform(test_form_12.drop(['FORMATION'], axis=1)), test_form_12.index, mm_cols)
test_form_13_merge_mm  = pd.DataFrame(sc_13.transform(test_form_13.drop(['FORMATION'], axis=1)), test_form_13.index, mm_cols)
test_form_14_merge_mm  = pd.DataFrame(sc_14.transform(test_form_14.drop(['FORMATION'], axis=1)), test_form_14.index, mm_cols)
test_form_15_merge_mm  = pd.DataFrame(sc_15.transform(test_form_15.drop(['FORMATION'], axis=1)), test_form_15.index, mm_cols)
test_form_16_merge_mm  = pd.DataFrame(sc_16.transform(test_form_16.drop(['FORMATION'], axis=1)), test_form_16.index, mm_cols)
test_form_17_merge_mm  = pd.DataFrame(sc_17.transform(test_form_17.drop(['FORMATION'], axis=1)), test_form_17.index, mm_cols)
test_form_18_merge_mm  = pd.DataFrame(sc_18.transform(test_form_18.drop(['FORMATION'], axis=1)), test_form_18.index, mm_cols)
test_form_19_merge_mm  = pd.DataFrame(sc_19.transform(test_form_19.drop(['FORMATION'], axis=1)), test_form_19.index, mm_cols)
test_form_20_merge_mm  = pd.DataFrame(sc_20.transform(test_form_20.drop(['FORMATION'], axis=1)), test_form_20.index, mm_cols)

test_form_21_merge_mm  = pd.DataFrame(sc_21.transform(test_form_21.drop(['FORMATION'], axis=1)), test_form_21.index, mm_cols)
test_form_22_merge_mm  = pd.DataFrame(sc_22.transform(test_form_22.drop(['FORMATION'], axis=1)), test_form_22.index, mm_cols)
test_form_23_merge_mm  = pd.DataFrame(sc_23.transform(test_form_23.drop(['FORMATION'], axis=1)), test_form_23.index, mm_cols)
test_form_24_merge_mm  = pd.DataFrame(sc_24.transform(test_form_24.drop(['FORMATION'], axis=1)), test_form_24.index, mm_cols)
test_form_25_merge_mm  = pd.DataFrame(sc_25.transform(test_form_25.drop(['FORMATION'], axis=1)), test_form_25.index, mm_cols)
test_form_26_merge_mm  = pd.DataFrame(sc_26.transform(test_form_26.drop(['FORMATION'], axis=1)), test_form_26.index, mm_cols)
test_form_27_merge_mm  = pd.DataFrame(sc_27.transform(test_form_27.drop(['FORMATION'], axis=1)), test_form_27.index, mm_cols)
test_form_28_merge_mm  = pd.DataFrame(sc_28.transform(test_form_28.drop(['FORMATION'], axis=1)), test_form_28.index, mm_cols)
test_form_29_merge_mm  = pd.DataFrame(sc_29.transform(test_form_29.drop(['FORMATION'], axis=1)), test_form_29.index, mm_cols)
test_form_30_merge_mm  = pd.DataFrame(sc_30.transform(test_form_30.drop(['FORMATION'], axis=1)), test_form_30.index, mm_cols)

test_form_31_merge_mm  = pd.DataFrame(sc_31.transform(test_form_31.drop(['FORMATION'], axis=1)), test_form_31.index, mm_cols)
test_form_32_merge_mm  = pd.DataFrame(sc_32.transform(test_form_32.drop(['FORMATION'], axis=1)), test_form_32.index, mm_cols)
test_form_33_merge_mm  = pd.DataFrame(sc_33.transform(test_form_33.drop(['FORMATION'], axis=1)), test_form_33.index, mm_cols)
test_form_34_merge_mm  = pd.DataFrame(sc_34.transform(test_form_34.drop(['FORMATION'], axis=1)), test_form_34.index, mm_cols)
test_form_35_merge_mm  = pd.DataFrame(sc_35.transform(test_form_35.drop(['FORMATION'], axis=1)), test_form_35.index, mm_cols)
test_form_36_merge_mm  = pd.DataFrame(sc_36.transform(test_form_36.drop(['FORMATION'], axis=1)), test_form_36.index, mm_cols)
test_form_37_merge_mm  = pd.DataFrame(sc_37.transform(test_form_37.drop(['FORMATION'], axis=1)), test_form_37.index, mm_cols)
test_form_38_merge_mm  = pd.DataFrame(sc_38.transform(test_form_38.drop(['FORMATION'], axis=1)), test_form_38.index, mm_cols)
test_form_39_merge_mm  = pd.DataFrame(sc_39.transform(test_form_39.drop(['FORMATION'], axis=1)), test_form_39.index, mm_cols)
test_form_40_merge_mm  = pd.DataFrame(sc_40.transform(test_form_40.drop(['FORMATION'], axis=1)), test_form_40.index, mm_cols)

test_form_41_merge_mm  = pd.DataFrame(sc_41.transform(test_form_41.drop(['FORMATION'], axis=1)), test_form_41.index, mm_cols)

test_form_sec_merge_mm  = pd.DataFrame(sc_sec.transform(test_form_sec.drop(['FORMATION'], axis=1)), test_form_sec.index, mm_cols)

In [95]:
test_form_mm = pd.concat([test_form_1_merge_mm,  test_form_2_merge_mm,  test_form_3_merge_mm,  test_form_4_merge_mm,  test_form_5_merge_mm,
                          test_form_6_merge_mm,  test_form_7_merge_mm,  test_form_8_merge_mm,  test_form_9_merge_mm,  test_form_10_merge_mm,
                          test_form_11_merge_mm, test_form_12_merge_mm, test_form_13_merge_mm, test_form_14_merge_mm, test_form_15_merge_mm,
                          test_form_16_merge_mm, test_form_17_merge_mm, test_form_18_merge_mm, test_form_19_merge_mm, test_form_20_merge_mm,
                          test_form_21_merge_mm, test_form_22_merge_mm, test_form_23_merge_mm, test_form_24_merge_mm, test_form_25_merge_mm,
                          test_form_26_merge_mm, test_form_27_merge_mm, test_form_28_merge_mm, test_form_29_merge_mm, test_form_30_merge_mm,
                          test_form_31_merge_mm, test_form_32_merge_mm, test_form_33_merge_mm, test_form_34_merge_mm, test_form_35_merge_mm,
                          test_form_36_merge_mm, test_form_37_merge_mm, test_form_38_merge_mm, test_form_39_merge_mm, test_form_40_merge_mm,
                          test_form_41_merge_mm,
                          test_form_sec_merge_mm])

test_form_mm.columns = ['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm',
                        'NPHI_mm', 'SP_mm', 'PEF_mm',
                        'Umaa_1_mm', 'Umaa_2_mm', 'Cmaa_1_mm', 'Cmaa_2_mm', 'Aaa_mm'
                        #'RMED_mm', 'RDEP_mm'
                        ]
test_form_mm.head(2)

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm,Umaa_1_mm,Umaa_2_mm,Cmaa_1_mm,Cmaa_2_mm,Aaa_mm
2924,0.262871,0.721162,0.496743,0.912084,0.776676,0.709823,0.345218,0.110372,0.264877,0.849431,0.572869,0.745943,0.258007
2925,0.245370,0.730122,0.496219,0.911666,0.777927,0.706327,0.345372,0.110252,0.264839,0.849445,0.575742,0.746951,0.263950


In [98]:
df_test_mm_form = df_test_form_trans_index.merge(test_form_mm, left_index=True, right_index=True, how='outer')
df_test_mm_form.head(2)

,FORMATION,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,Umaa_1,Umaa_2,Cmaa_1,Cmaa_2,Aaa,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm,Umaa_1_mm,Umaa_2_mm,Cmaa_1_mm,Cmaa_2_mm,Aaa_mm
0,no_form,72.078377,174.347092,19.2031,-0.538873,2.171173,0.551575,35.525719,55.815659,91.695822,-49.618614,1.068022,-1.642023,32.927196,0.107031,0.672277,0.678751,0.632776,0.493260,0.786255,0.754834,0.316494,0.285396,0.883321,0.331961,0.531935,0.234166
1,no_form,72.147697,176.112915,19.2031,-0.539232,2.171840,0.547566,36.158520,41.862537,68.833219,-37.194772,1.076708,-1.634365,34.298981,0.107140,0.680506,0.678751,0.632735,0.493545,0.784342,0.755153,0.285705,0.264685,0.889345,0.333743,0.532787,0.236965


In [102]:
#df_test_mm_form.to_csv('../df_test_mm_form_umaa.csv')
df_test_mm_form = pd.read_csv('../df_test_mm_form_umaa.csv', index_col=0)
df_test_mm_form.head(2)

In [99]:
df_test_all_mm_predict = df_test_mm_form[['GR_new',    'DTC_new',   'CALI_new',  'DRHO_new',  'RHOB_new', 'NPHI_new', 'SP_new', 'PEF_new', 
                                          'Umaa_1',    
                                          'Umaa_2',    
                                          'Cmaa_1',    
                                          'Cmaa_2',    
                                          'Aaa',
                                          #'GR_mm',     'DTC_mm',    'CALI_mm',   'DRHO_mm',   'RHOB_mm',  'NPHI_mm',  'SP_mm',  'PEF_mm', 
                                          #'Umaa_1_mm', 'Umaa_2_mm', 'Cmaa_1_mm', 'Cmaa_2_mm', 'Aaa_mm'
                                          ]]

In [109]:
test_prediction = model_1_umaa.predict(df_test_all_mm_predict)
test_prediction

array([2, 2, 2, ..., 1, 1, 1])

In [110]:
pd.Series(test_prediction).value_counts()

2     69821
0     25415
1     22088
3      7614
5      6463
9      3665
10     1354
4       219
6        83
8        63
7         1
dtype: int64

In [104]:
lithology_numbers = {30000: 0,           65030: 1,                 65000: 2,           80000: 3,      74000: 4,          70000: 5, 
                     70032: 6,           88000: 7,                 86000: 8,           99000: 9,      90000: 10,         93000: 11}
lithology_keys    = {30000: 'Sandstone', 65030: 'Sandstone/Shale', 65000: 'Shale',     80000: 'Marl', 74000: 'Dolomite', 70000: 'Limestone',
                     70032: 'Chalk',     88000: 'Halite',          86000: 'Anhydrite', 99000: 'Tuff', 90000: 'Coal',     93000: 'Basement'}
lithology_int     = {0: 'Sandstone',     1: 'Sandstone/Shale',     2: 'Shale',         3: 'Marl',     4: 'Dolomite',     5: 'Limestone',
                     6: 'Chalk',         7: 'Halite',              8: 'Anhydrite',     9: 'Tuff',     10: 'Coal',        11: 'Basement'}

In [111]:
category_to_lithology = {y:x for x,y in lithology_numbers.items()}
test_prediction_for_submission = np.vectorize(category_to_lithology.get)(test_prediction)
test_prediction_for_submission

array([65000, 65000, 65000, ..., 65030, 65030, 65030])

In [122]:
test_prediction_for_submission = pd.DataFrame(test_prediction_for_submission, columns=['lithology'])

In [123]:
test_prediction_for_submission

,lithology
0,65000
1,65000
2,65000
3,65000
4,65000
...,...
136781,30000
136782,30000
136783,65030
136784,65030


In [ ]:
test_prediction_for_submission.to_csv('../Submission/test_predictions.csv')

In [113]:
pwd

'/content/drive/My Drive/FORCE Log 2020/Raw'

In [ ]:
np.savetxt('../Submission/test_predictions.csv', test_prediction_for_submission, header='lithology', fmt='%i')

In [ ]:
test_predictions = pd.read_csv('../Submission/test_predictions.csv')
test_predictions.columns = ['lithology']
test_predictions.head(2)

In [ ]:
test_predictions.info()

## Global Scaling, then Impute (based on FORMATION)

In [ ]:
df_train['FORMATION'].fillna('no_form', inplace=True)

In [ ]:
df_train_2 = df_train[['FORMATION', 
                       'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 
                       'NPHI', 'SP', 'PEF', 
                      #'RMED', 'RDEP'
                     ]]
df_train_2.head(2)

,FORMATION,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF
0,no_form,80.200851,161.13118,19.480835,-0.574928,1.884186,NaN,24.612379,20.915468
1,no_form,79.262886,160.60347,19.468800,-0.570188,1.889794,NaN,23.895531,19.383013


In [ ]:
limestone = new_df_form_trans_last[new_df_form_trans_last['Lithology Keys'] == 'Limestone']
marl = new_df_form_trans_last[new_df_form_trans_last['Lithology Keys'] == 'Marl']
tuff = new_df_form_trans_last[new_df_form_trans_last['Lithology Keys'] == 'Tuff']
chalk = new_df_form_trans_last[new_df_form_trans_last['Lithology Keys'] == 'Chalk']
halite = new_df_form_trans_last[new_df_form_trans_last['Lithology Keys'] == 'Halite']
coal = new_df_form_trans_last[new_df_form_trans_last['Lithology Keys'] == 'Coal']
dolomite = new_df_form_trans_last[new_df_form_trans_last['Lithology Keys'] == 'Dolomite']
anhydrite = new_df_form_trans_last[new_df_form_trans_last['Lithology Keys'] == 'Anhydrite']
basement = new_df_form_trans_last[new_df_form_trans_last['Lithology Keys'] == 'Basement']

In [ ]:
df_tuff      = new_df_form_trans_last.append([tuff]*2, ignore_index=True)

In [ ]:
df_chalk     = df_tuff.append([chalk]*3, ignore_index=True)

In [ ]:
df_halite    = df_chalk.append([halite]*4, ignore_index=True)

In [ ]:
df_coal      = df_halite.append([coal]*8, ignore_index=True)

In [ ]:
df_dolomite  = df_coal.append([dolomite]*20, ignore_index=True)

In [ ]:
df_anhydrite = df_dolomite.append([anhydrite]*30, ignore_index=True)

In [ ]:
df_basement  = df_anhydrite.append([basement]*300, ignore_index=True)

In [ ]:
df_train_form_trans_merge_mm_copylith = df_basement.copy()

In [ ]:
df_form_trans = df_train_form_trans_merge_mm_copylith.copy()

In [ ]:
df_form_trans['Lithology Keys'].unique()

array(['Shale', 'Sandstone', 'Sandstone/Shale', 'Limestone', 'Tuff',
       'Marl', 'Anhydrite', 'Dolomite', 'Chalk', 'Coal', 'Halite',
       'Basement'], dtype=object)

In [ ]:
form = df_train_2['FORMATION'].unique()

In [ ]:
# Primary Formation
form_1  = df_train_2[df_train_2['FORMATION'] == form[1]]
form_2  = df_train_2[df_train_2['FORMATION'] == form[2]]
form_3  = df_train_2[df_train_2['FORMATION'] == form[3]]
form_4  = df_train_2[df_train_2['FORMATION'] == form[4]]
form_5  = df_train_2[df_train_2['FORMATION'] == form[5]]
form_42 = df_train_2[df_train_2['FORMATION'] == form[42]]
form_19 = df_train_2[df_train_2['FORMATION'] == form[19]]
form_6  = df_train_2[df_train_2['FORMATION'] == form[6]]
form_7  = df_train_2[df_train_2['FORMATION'] == form[7]]
form_8  = df_train_2[df_train_2['FORMATION'] == form[8]]

form_9  = df_train_2[df_train_2['FORMATION'] == form[9]]
form_10 = df_train_2[df_train_2['FORMATION'] == form[10]]
form_11 = df_train_2[df_train_2['FORMATION'] == form[11]]
form_12 = df_train_2[df_train_2['FORMATION'] == form[12]]
form_13 = df_train_2[df_train_2['FORMATION'] == form[13]]
form_14 = df_train_2[df_train_2['FORMATION'] == form[14]]
form_15 = df_train_2[df_train_2['FORMATION'] == form[15]]
form_24 = df_train_2[df_train_2['FORMATION'] == form[24]]
form_18 = df_train_2[df_train_2['FORMATION'] == form[18]]
form_16 = df_train_2[df_train_2['FORMATION'] == form[16]]

form_21 = df_train_2[df_train_2['FORMATION'] == form[21]]
form_22 = df_train_2[df_train_2['FORMATION'] == form[22]]
form_40 = df_train_2[df_train_2['FORMATION'] == form[40]]
form_33 = df_train_2[df_train_2['FORMATION'] == form[33]]
form_23 = df_train_2[df_train_2['FORMATION'] == form[23]]
form_43 = df_train_2[df_train_2['FORMATION'] == form[43]]
form_46 = df_train_2[df_train_2['FORMATION'] == form[46]]
form_52 = df_train_2[df_train_2['FORMATION'] == form[52]]
form_51 = df_train_2[df_train_2['FORMATION'] == form[51]]
form_44 = df_train_2[df_train_2['FORMATION'] == form[44]]

form_45 = df_train_2[df_train_2['FORMATION'] == form[45]]
form_55 = df_train_2[df_train_2['FORMATION'] == form[55]]
form_56 = df_train_2[df_train_2['FORMATION'] == form[56]]
form_57 = df_train_2[df_train_2['FORMATION'] == form[57]]
form_58 = df_train_2[df_train_2['FORMATION'] == form[58]]
form_47 = df_train_2[df_train_2['FORMATION'] == form[47]]
form_48 = df_train_2[df_train_2['FORMATION'] == form[48]]
form_60 = df_train_2[df_train_2['FORMATION'] == form[60]]
form_53 = df_train_2[df_train_2['FORMATION'] == form[53]]
form_68 = df_train_2[df_train_2['FORMATION'] == form[68]]

form_54 = df_train_2[df_train_2['FORMATION'] == form[54]]

In [ ]:
# Secondary Formation
form_0  = df_train_2[df_train_2['FORMATION'] == form[0]]  #no_form  
form_35 = df_train_2[df_train_2['FORMATION'] == form[35]] #Basement

form_17  = df_train_2[df_train_2['FORMATION'] == form[17]]
form_20  = df_train_2[df_train_2['FORMATION'] == form[20]]
form_25  = df_train_2[df_train_2['FORMATION'] == form[25]]
form_26  = df_train_2[df_train_2['FORMATION'] == form[26]]
form_27  = df_train_2[df_train_2['FORMATION'] == form[27]]
form_28  = df_train_2[df_train_2['FORMATION'] == form[28]]
form_29 = df_train_2[df_train_2['FORMATION'] == form[29]]
form_30  = df_train_2[df_train_2['FORMATION'] == form[30]]
form_31  = df_train_2[df_train_2['FORMATION'] == form[31]]
form_32  = df_train_2[df_train_2['FORMATION'] == form[32]]

form_34  = df_train_2[df_train_2['FORMATION'] == form[34]]
form_36  = df_train_2[df_train_2['FORMATION'] == form[36]]
form_37  = df_train_2[df_train_2['FORMATION'] == form[37]]
form_38  = df_train_2[df_train_2['FORMATION'] == form[38]]
form_39  = df_train_2[df_train_2['FORMATION'] == form[39]]
form_41  = df_train_2[df_train_2['FORMATION'] == form[41]]
form_49  = df_train_2[df_train_2['FORMATION'] == form[49]]
form_50  = df_train_2[df_train_2['FORMATION'] == form[50]]
form_59  = df_train_2[df_train_2['FORMATION'] == form[59]]
form_61  = df_train_2[df_train_2['FORMATION'] == form[61]]

form_62  = df_train_2[df_train_2['FORMATION'] == form[62]]
form_63  = df_train_2[df_train_2['FORMATION'] == form[63]]
form_64  = df_train_2[df_train_2['FORMATION'] == form[64]]
form_65  = df_train_2[df_train_2['FORMATION'] == form[65]]
form_66  = df_train_2[df_train_2['FORMATION'] == form[66]]
form_67  = df_train_2[df_train_2['FORMATION'] == form[67]]
form_69  = df_train_2[df_train_2['FORMATION'] == form[69]]

In [ ]:
form_merge_primary = pd.concat([form_1,  form_2,  form_3,  form_4,  form_5,  form_42, form_19, form_6,  form_7,  form_8,
                                form_9,  form_10, form_11, form_12, form_13, form_14, form_15, form_24, form_18, form_16,
                                form_21, form_22, form_40, form_33, form_23, form_43, form_46, form_52, form_51, form_44,
                                form_45, form_55, form_56, form_57, form_58, form_47, form_48, form_60, form_53, form_68, 
                                form_54])

In [ ]:
form_merge_second = pd.concat([form_0, form_35,
                               form_17, form_20, form_25, form_26, form_27, form_28, form_29, form_30, form_31, form_32, 
                               form_34, form_36, form_37, form_38, form_39, form_41, form_49, form_50, form_59, form_61, 
                               form_62, form_63, form_64, form_65, form_66, form_67, form_69                               
                               ])

In [ ]:
form_merge_second.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218767 entries, 0 to 1169030
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   FORMATION  218767 non-null  object 
 1   GR         218767 non-null  float64
 2   DTC        192763 non-null  float64
 3   CALI       186464 non-null  float64
 4   DRHO       159340 non-null  float64
 5   RHOB       158336 non-null  float64
 6   NPHI       118405 non-null  float64
 7   SP         171816 non-null  float64
 8   PEF        93689 non-null   float64
dtypes: float64(8), object(1)
memory usage: 16.7+ MB


In [ ]:
sc = MinMaxScaler(feature_range=(0.0, 1.0))
#sc = StandardScaler()
#sc = RobustScaler()

In [ ]:
sc_all = MinMaxScaler(feature_range=(0.0, 1.0)).fit(df_train_2.drop(['FORMATION'], axis=1))

In [ ]:
sc_1 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_1.drop(['FORMATION'], axis=1))
sc_2 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_2.drop(['FORMATION'], axis=1))
sc_3 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_3.drop(['FORMATION'], axis=1))
sc_4 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_4.drop(['FORMATION'], axis=1))
sc_5 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_5.drop(['FORMATION'], axis=1))
sc_6 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_42.drop(['FORMATION'], axis=1))
sc_7 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_19.drop(['FORMATION'], axis=1))
sc_8 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_6.drop(['FORMATION'], axis=1))
sc_9 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_7.drop(['FORMATION'], axis=1))
sc_10 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_8.drop(['FORMATION'], axis=1))

sc_11 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_9.drop(['FORMATION'], axis=1))
sc_12 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_10.drop(['FORMATION'], axis=1))
sc_13 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_11.drop(['FORMATION'], axis=1))
sc_14 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_12.drop(['FORMATION'], axis=1))
sc_15 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_13.drop(['FORMATION'], axis=1))
sc_16 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_14.drop(['FORMATION'], axis=1))
sc_17 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_15.drop(['FORMATION'], axis=1))
sc_18 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_24.drop(['FORMATION'], axis=1))
sc_19 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_18.drop(['FORMATION'], axis=1))
sc_20 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_16.drop(['FORMATION'], axis=1))

sc_21 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_21.drop(['FORMATION'], axis=1))
sc_22 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_22.drop(['FORMATION'], axis=1))
sc_23 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_40.drop(['FORMATION'], axis=1))
sc_24 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_33.drop(['FORMATION'], axis=1))
sc_25 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_23.drop(['FORMATION'], axis=1))
sc_26 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_43.drop(['FORMATION'], axis=1))
sc_27 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_46.drop(['FORMATION'], axis=1))
sc_28 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_52.drop(['FORMATION'], axis=1))
sc_29 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_51.drop(['FORMATION'], axis=1))
sc_30 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_44.drop(['FORMATION'], axis=1))

sc_31 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_45.drop(['FORMATION'], axis=1))
sc_32 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_55.drop(['FORMATION'], axis=1))
sc_33 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_56.drop(['FORMATION'], axis=1))
sc_34 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_57.drop(['FORMATION'], axis=1))
sc_35 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_58.drop(['FORMATION'], axis=1))
sc_36 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_47.drop(['FORMATION'], axis=1))
sc_37 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_48.drop(['FORMATION'], axis=1))
sc_38 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_60.drop(['FORMATION'], axis=1))
sc_39 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_53.drop(['FORMATION'], axis=1))
sc_40 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_68.drop(['FORMATION'], axis=1))

sc_41 = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_54.drop(['FORMATION'], axis=1))

sc_sec = MinMaxScaler(feature_range=(0.0, 1.0)).fit(form_merge_second.drop(['FORMATION'], axis=1))

In [ ]:
mm_cols = ['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm', 
           'NPHI_mm', 'SP_mm', 'PEF_mm', 
           #'RMED_mm', 'RDEP_mm'
           ]
#std_cols = ['GR_std', 'DTC_std', 'CALI_std', 'DRHO_std', 'RHOB_std', 'RMED_std', 'RDEP_std']

In [ ]:
form_1_mm  = pd.DataFrame(sc_1.transform(form_1.drop(['FORMATION'], axis=1)), form_1.index, mm_cols)
form_2_mm  = pd.DataFrame(sc_2.transform(form_2.drop(['FORMATION'], axis=1)), form_2.index, mm_cols)
form_3_mm  = pd.DataFrame(sc_3.transform(form_3.drop(['FORMATION'], axis=1)), form_3.index, mm_cols)
form_4_mm  = pd.DataFrame(sc_4.transform(form_4.drop(['FORMATION'], axis=1)), form_4.index, mm_cols)
form_5_mm  = pd.DataFrame(sc_5.transform(form_5.drop(['FORMATION'], axis=1)), form_5.index, mm_cols)
form_42_mm = pd.DataFrame(sc_6.transform(form_42.drop(['FORMATION'], axis=1)), form_42.index, mm_cols)
form_19_mm = pd.DataFrame(sc_7.transform(form_19.drop(['FORMATION'], axis=1)), form_19.index, mm_cols)
form_6_mm  = pd.DataFrame(sc_8.transform(form_6.drop(['FORMATION'], axis=1)), form_6.index, mm_cols)
form_7_mm  = pd.DataFrame(sc_9.transform(form_7.drop(['FORMATION'], axis=1)), form_7.index, mm_cols)
form_8_mm  = pd.DataFrame(sc_10.transform(form_8.drop(['FORMATION'], axis=1)), form_8.index, mm_cols)

form_9_mm  = pd.DataFrame(sc_11.transform(form_9.drop(['FORMATION'], axis=1)), form_9.index, mm_cols)
form_10_mm = pd.DataFrame(sc_12.transform(form_10.drop(['FORMATION'], axis=1)), form_10.index, mm_cols)
form_11_mm = pd.DataFrame(sc_13.transform(form_11.drop(['FORMATION'], axis=1)), form_11.index, mm_cols)
form_12_mm = pd.DataFrame(sc_14.transform(form_12.drop(['FORMATION'], axis=1)), form_12.index, mm_cols)
form_13_mm = pd.DataFrame(sc_15.transform(form_13.drop(['FORMATION'], axis=1)), form_13.index, mm_cols)
form_14_mm = pd.DataFrame(sc_16.transform(form_14.drop(['FORMATION'], axis=1)), form_14.index, mm_cols)
form_15_mm = pd.DataFrame(sc_17.transform(form_15.drop(['FORMATION'], axis=1)), form_15.index, mm_cols)
form_24_mm = pd.DataFrame(sc_18.transform(form_24.drop(['FORMATION'], axis=1)), form_24.index, mm_cols)
form_18_mm = pd.DataFrame(sc_19.transform(form_18.drop(['FORMATION'], axis=1)), form_18.index, mm_cols)
form_16_mm = pd.DataFrame(sc_20.transform(form_16.drop(['FORMATION'], axis=1)), form_16.index, mm_cols)

form_21_mm = pd.DataFrame(sc_21.transform(form_21.drop(['FORMATION'], axis=1)), form_21.index, mm_cols)
form_22_mm = pd.DataFrame(sc_22.transform(form_22.drop(['FORMATION'], axis=1)), form_22.index, mm_cols)
form_40_mm = pd.DataFrame(sc_23.transform(form_40.drop(['FORMATION'], axis=1)), form_40.index, mm_cols)
form_33_mm = pd.DataFrame(sc_24.transform(form_33.drop(['FORMATION'], axis=1)), form_33.index, mm_cols)
form_23_mm = pd.DataFrame(sc_25.transform(form_23.drop(['FORMATION'], axis=1)), form_23.index, mm_cols)
form_43_mm = pd.DataFrame(sc_26.transform(form_43.drop(['FORMATION'], axis=1)), form_43.index, mm_cols)
form_46_mm = pd.DataFrame(sc_27.transform(form_46.drop(['FORMATION'], axis=1)), form_46.index, mm_cols)
form_52_mm = pd.DataFrame(sc_28.transform(form_52.drop(['FORMATION'], axis=1)), form_52.index, mm_cols)
form_51_mm = pd.DataFrame(sc_29.transform(form_51.drop(['FORMATION'], axis=1)), form_51.index, mm_cols)
form_44_mm = pd.DataFrame(sc_30.transform(form_44.drop(['FORMATION'], axis=1)), form_44.index, mm_cols)

form_45_mm = pd.DataFrame(sc_31.transform(form_45.drop(['FORMATION'], axis=1)), form_45.index, mm_cols)
form_55_mm = pd.DataFrame(sc_32.transform(form_55.drop(['FORMATION'], axis=1)), form_55.index, mm_cols)
form_56_mm = pd.DataFrame(sc_33.transform(form_56.drop(['FORMATION'], axis=1)), form_56.index, mm_cols)
form_57_mm = pd.DataFrame(sc_34.transform(form_57.drop(['FORMATION'], axis=1)), form_57.index, mm_cols)
form_58_mm = pd.DataFrame(sc_35.transform(form_58.drop(['FORMATION'], axis=1)), form_58.index, mm_cols)
form_47_mm = pd.DataFrame(sc_36.transform(form_47.drop(['FORMATION'], axis=1)), form_47.index, mm_cols)
form_48_mm = pd.DataFrame(sc_37.transform(form_48.drop(['FORMATION'], axis=1)), form_48.index, mm_cols)
form_60_mm = pd.DataFrame(sc_38.transform(form_60.drop(['FORMATION'], axis=1)), form_60.index, mm_cols)
form_53_mm = pd.DataFrame(sc_39.transform(form_53.drop(['FORMATION'], axis=1)), form_53.index, mm_cols)
form_68_mm = pd.DataFrame(sc_40.transform(form_68.drop(['FORMATION'], axis=1)), form_68.index, mm_cols)

form_54_mm = pd.DataFrame(sc_41.transform(form_54.drop(['FORMATION'], axis=1)), form_54.index, mm_cols)

form_sec_mm = pd.DataFrame(sc_sec.transform(form_merge_second.drop(['FORMATION'], axis=1)), form_merge_second.index, mm_cols)

In [ ]:
form_all_mm = pd.DataFrame(sc_all.transform(df_train_2.drop(['FORMATION'], axis=1)), df_train_2.index, mm_cols)
form_all_mm.describe()

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
count,1.170511e+06,1.089648e+06,1.082634e+06,987857.000000,1.009242e+06,765409.000000,864247.000000,671692.000000
mean,6.575114e-02,3.384070e-01,4.180327e-01,0.999620,5.714658e-01,0.355216,0.694199,0.016239
std,3.178840e-02,9.579677e-02,1.464780e-01,0.001006,9.254567e-02,0.126022,0.050195,0.028602
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,4.412660e-02,2.568509e-01,2.732104e-01,0.999617,5.010257e-01,0.267712,0.676087,0.008654
50%,6.338678e-02,3.263554e-01,3.937441e-01,0.999619,5.847078e-01,0.350227,0.691156,0.011001
75%,8.257961e-02,4.259814e-01,5.539521e-01,0.999621,6.458555e-01,0.443092,0.709511,0.015323
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,1.000000,1.000000,1.000000


In [ ]:
form_mm = pd.concat([form_1_mm,  form_2_mm,  form_3_mm,  form_4_mm,  form_5_mm,
                     form_42_mm, form_19_mm, form_6_mm,  form_7_mm,  form_8_mm,
                     form_9_mm,  form_10_mm, form_11_mm, form_12_mm, form_13_mm,
                     form_14_mm, form_15_mm, form_24_mm, form_18_mm, form_16_mm,
                     form_21_mm, form_22_mm, form_40_mm, form_33_mm, form_23_mm,
                     form_43_mm, form_46_mm, form_52_mm, form_51_mm, form_44_mm,
                     form_45_mm, form_55_mm, form_56_mm, form_57_mm, form_58_mm,
                     form_47_mm, form_48_mm, form_60_mm, form_53_mm, form_68_mm,
                     form_54_mm, 
                     form_sec_mm])

form_mm.columns = ['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm',
                   'NPHI_mm', 'SP_mm', 'PEF_mm', 
                   #'RMED_mm', 'RDEP_mm'
                   ]
form_mm.head(2)

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
2464,0.170890,0.587551,0.551347,0.910346,0.572659,NaN,0.144274,0.002181
2465,0.143545,0.598294,0.568145,0.908929,0.561091,NaN,0.142960,0.001812


In [ ]:
form_mm.describe()

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
count,1.170511e+06,1.089648e+06,1.082634e+06,987857.000000,1.009242e+06,765409.000000,864247.000000,671692.000000
mean,2.207431e-01,4.779328e-01,3.934205e-01,0.494353,5.376962e-01,0.403804,0.461448,0.045983
std,1.598461e-01,1.623266e-01,2.029732e-01,0.256086,1.345109e-01,0.143931,0.233964,0.086186
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,9.980838e-02,3.493102e-01,2.583671e-01,0.284359,4.494543e-01,0.315752,0.259963,0.008744
50%,1.692883e-01,4.840533e-01,3.778403e-01,0.437260,5.293405e-01,0.409822,0.457203,0.016337
75%,3.073412e-01,6.132138e-01,5.320281e-01,0.648306,6.272014e-01,0.487652,0.618983,0.039007
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,1.000000,1.000000,1.000000


In [ ]:
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

In [ ]:
imputer_all = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=800, verbose=1, tol=0.00001)

In [ ]:
imputer_1 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_2 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_3 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_4 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_5 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_6 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_7 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_8 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_9 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_10 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)

imputer_11 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_12 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_13 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_14 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_15 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_16 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_17 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_18 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_19 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_20 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)

imputer_21 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_22 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_23 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_24 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_25 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_26 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_27 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_28 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_29 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_30 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)

imputer_31 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_32 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_33 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_34 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_35 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_36 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_37 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_38 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_39 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)
imputer_40 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)

imputer_41 = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=400, verbose=1, tol=0.0001)

imputer_sec = IterativeImputer(estimator=BayesianRidge(), imputation_order='descending', max_iter=1000, verbose=1, tol=0.0001)

In [ ]:
imputer_all.fit(form_all_mm)

In [ ]:
#form_68_mm['SP_mm'] = 0.0
#form_68_mm.head(2)

In [ ]:
imputer_1.fit(form_1_mm)
imputer_2.fit(form_2_mm)
imputer_3.fit(form_3_mm)
imputer_4.fit(form_4_mm)
imputer_5.fit(form_5_mm)
imputer_6.fit(form_42_mm)
imputer_7.fit(form_19_mm)
imputer_8.fit(form_6_mm)
imputer_9.fit(form_7_mm)
imputer_10.fit(form_8_mm)

imputer_11.fit(form_9_mm)
imputer_12.fit(form_10_mm)
imputer_13.fit(form_11_mm)
imputer_14.fit(form_12_mm)
imputer_15.fit(form_13_mm)
imputer_16.fit(form_14_mm)
imputer_17.fit(form_15_mm)
imputer_18.fit(form_24_mm)
imputer_19.fit(form_18_mm)
imputer_20.fit(form_16_mm)

imputer_21.fit(form_21_mm)
imputer_22.fit(form_22_mm)
imputer_23.fit(form_40_mm)
imputer_24.fit(form_33_mm)
imputer_25.fit(form_23_mm)
imputer_26.fit(form_43_mm)
imputer_27.fit(form_46_mm)
imputer_28.fit(form_52_mm)
imputer_29.fit(form_51_mm)
imputer_30.fit(form_44_mm)

imputer_31.fit(form_45_mm)
imputer_32.fit(form_55_mm)
imputer_33.fit(form_56_mm)
imputer_34.fit(form_57_mm)
imputer_35.fit(form_58_mm)
imputer_36.fit(form_47_mm)
imputer_37.fit(form_48_mm)
imputer_38.fit(form_60_mm)
imputer_39.fit(form_53_mm)
imputer_40.fit(form_68_mm.drop(['SP_mm'], axis=1))

imputer_41.fit(form_54_mm)

imputer_sec.fit(form_sec_mm)

In [ ]:
form_1_trans  = imputer_1.transform(form_1_mm)
form_2_trans  = imputer_2.transform(form_2_mm)
form_3_trans  = imputer_3.transform(form_3_mm)
form_4_trans  = imputer_4.transform(form_4_mm)
form_5_trans  = imputer_5.transform(form_5_mm)
form_42_trans = imputer_6.transform(form_42_mm)
form_19_trans = imputer_7.transform(form_19_mm)
form_6_trans  = imputer_8.transform(form_6_mm)
form_7_trans  = imputer_9.transform(form_7_mm)
form_8_trans  = imputer_10.transform(form_8_mm)

form_9_trans  = imputer_11.transform(form_9_mm)
form_10_trans = imputer_12.transform(form_10_mm)
form_11_trans = imputer_13.transform(form_11_mm)
form_12_trans = imputer_14.transform(form_12_mm)
form_13_trans = imputer_15.transform(form_13_mm)
form_14_trans = imputer_16.transform(form_14_mm)
form_15_trans = imputer_17.transform(form_15_mm)
form_24_trans = imputer_18.transform(form_24_mm)
form_18_trans = imputer_19.transform(form_18_mm)
form_16_trans = imputer_20.transform(form_16_mm)

form_21_trans = imputer_21.transform(form_21_mm)
form_22_trans = imputer_22.transform(form_22_mm)
form_40_trans = imputer_23.transform(form_40_mm)
form_33_trans = imputer_24.transform(form_33_mm)
form_23_trans = imputer_25.transform(form_23_mm)
form_43_trans = imputer_26.transform(form_43_mm)
form_46_trans = imputer_27.transform(form_46_mm)
form_52_trans = imputer_28.transform(form_52_mm)
form_51_trans = imputer_29.transform(form_51_mm)
form_44_trans = imputer_30.transform(form_44_mm)

form_45_trans = imputer_31.transform(form_45_mm)
form_55_trans = imputer_32.transform(form_55_mm)
form_56_trans = imputer_33.transform(form_56_mm)
form_57_trans = imputer_34.transform(form_57_mm)
form_58_trans = imputer_35.transform(form_58_mm)
form_47_trans = imputer_36.transform(form_47_mm)
form_48_trans = imputer_37.transform(form_48_mm)
form_60_trans = imputer_38.transform(form_60_mm)
form_53_trans = imputer_39.transform(form_53_mm)
form_68_trans = imputer_40.transform(form_68_mm.drop(['SP_mm'], axis=1))

form_54_trans = imputer_41.transform(form_54_mm)

form_sec_trans = imputer_sec.transform(form_sec_mm)

[IterativeImputer] Completing matrix with shape (172636, 8)
[IterativeImputer] Completing matrix with shape (25814, 8)
[IterativeImputer] Completing matrix with shape (25451, 8)
[IterativeImputer] Completing matrix with shape (71080, 8)
[IterativeImputer] Completing matrix with shape (9662, 8)
[IterativeImputer] Completing matrix with shape (5012, 8)
[IterativeImputer] Completing matrix with shape (2417, 8)
[IterativeImputer] Completing matrix with shape (13525, 8)
[IterativeImputer] Completing matrix with shape (11862, 8)
[IterativeImputer] Completing matrix with shape (3016, 8)
[IterativeImputer] Completing matrix with shape (15215, 8)
[IterativeImputer] Completing matrix with shape (17462, 8)
[IterativeImputer] Completing matrix with shape (3928, 8)
[IterativeImputer] Completing matrix with shape (14817, 8)
[IterativeImputer] Completing matrix with shape (24961, 8)
[IterativeImputer] Completing matrix with shape (65041, 8)
[IterativeImputer] Completing matrix with shape (4656, 8)
[I

In [ ]:
df_form_1_trans = pd.DataFrame(form_1_trans, columns=form_1_mm.columns, index=form_1_mm.index)
df_form_2_trans = pd.DataFrame(form_2_trans, columns=form_2_mm.columns, index=form_2_mm.index)
df_form_3_trans = pd.DataFrame(form_3_trans, columns=form_3_mm.columns, index=form_3_mm.index)
df_form_4_trans = pd.DataFrame(form_4_trans, columns=form_4_mm.columns, index=form_4_mm.index)
df_form_5_trans = pd.DataFrame(form_5_trans, columns=form_5_mm.columns, index=form_5_mm.index)
df_form_42_trans = pd.DataFrame(form_42_trans, columns=form_42_mm.columns, index=form_42_mm.index)
df_form_19_trans = pd.DataFrame(form_19_trans, columns=form_19_mm.columns, index=form_19_mm.index)
df_form_6_trans = pd.DataFrame(form_6_trans, columns=form_6_mm.columns, index=form_6_mm.index)
df_form_7_trans = pd.DataFrame(form_7_trans, columns=form_7_mm.columns, index=form_7_mm.index)
df_form_8_trans = pd.DataFrame(form_8_trans, columns=form_8_mm.columns, index=form_8_mm.index)

df_form_9_trans = pd.DataFrame(form_9_trans, columns=form_9_mm.columns, index=form_9_mm.index)
df_form_10_trans = pd.DataFrame(form_10_trans, columns=form_10_mm.columns, index=form_10_mm.index)
df_form_11_trans = pd.DataFrame(form_11_trans, columns=form_11_mm.columns, index=form_11_mm.index)
df_form_12_trans = pd.DataFrame(form_12_trans, columns=form_12_mm.columns, index=form_12_mm.index)
df_form_13_trans = pd.DataFrame(form_13_trans, columns=form_13_mm.columns, index=form_13_mm.index)
df_form_14_trans = pd.DataFrame(form_14_trans, columns=form_14_mm.columns, index=form_14_mm.index)
df_form_15_trans = pd.DataFrame(form_15_trans, columns=form_15_mm.columns, index=form_15_mm.index)
df_form_24_trans = pd.DataFrame(form_24_trans, columns=form_24_mm.columns, index=form_24_mm.index)
df_form_18_trans = pd.DataFrame(form_18_trans, columns=form_18_mm.columns, index=form_18_mm.index)
df_form_16_trans = pd.DataFrame(form_16_trans, columns=form_16_mm.columns, index=form_16_mm.index)

df_form_21_trans = pd.DataFrame(form_21_trans, columns=form_21_mm.columns, index=form_21_mm.index)
df_form_22_trans = pd.DataFrame(form_22_trans, columns=form_22_mm.columns, index=form_22_mm.index)
df_form_40_trans = pd.DataFrame(form_40_trans, columns=form_40_mm.columns, index=form_40_mm.index)
df_form_33_trans = pd.DataFrame(form_33_trans, columns=form_33_mm.columns, index=form_33_mm.index)
df_form_23_trans = pd.DataFrame(form_23_trans, columns=form_23_mm.columns, index=form_23_mm.index)
df_form_43_trans = pd.DataFrame(form_43_trans, columns=form_43_mm.columns, index=form_43_mm.index)
df_form_46_trans = pd.DataFrame(form_46_trans, columns=form_46_mm.columns, index=form_46_mm.index)
df_form_52_trans = pd.DataFrame(form_52_trans, columns=form_52_mm.columns, index=form_52_mm.index)
df_form_51_trans = pd.DataFrame(form_51_trans, columns=form_51_mm.columns, index=form_51_mm.index)
df_form_44_trans = pd.DataFrame(form_44_trans, columns=form_44_mm.columns, index=form_44_mm.index)

df_form_45_trans = pd.DataFrame(form_45_trans, columns=form_45_mm.columns, index=form_45_mm.index)
df_form_55_trans = pd.DataFrame(form_55_trans, columns=form_55_mm.columns, index=form_55_mm.index)
df_form_56_trans = pd.DataFrame(form_56_trans, columns=form_56_mm.columns, index=form_56_mm.index)
df_form_57_trans = pd.DataFrame(form_57_trans, columns=form_57_mm.columns, index=form_57_mm.index)
df_form_58_trans = pd.DataFrame(form_58_trans, columns=form_58_mm.columns, index=form_58_mm.index)
df_form_47_trans = pd.DataFrame(form_47_trans, columns=form_47_mm.columns, index=form_47_mm.index)
df_form_48_trans = pd.DataFrame(form_48_trans, columns=form_48_mm.columns, index=form_48_mm.index)
df_form_60_trans = pd.DataFrame(form_60_trans, columns=form_60_mm.columns, index=form_60_mm.index)
df_form_53_trans = pd.DataFrame(form_53_trans, columns=form_53_mm.columns, index=form_53_mm.index)
df_form_68_trans = pd.DataFrame(form_68_trans, columns=['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm', 'NPHI_mm', 'PEF_mm'],
                                index=form_68_mm.index)

df_form_54_trans = pd.DataFrame(form_54_trans, columns=form_54_mm.columns, index=form_54_mm.index)

df_form_sec_trans = pd.DataFrame(form_sec_trans, columns=form_sec_mm.columns, index=form_sec_mm.index)

In [ ]:
df_form_68_trans['SP_mm'] = np.nan
df_form_68_trans = df_form_68_trans[['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm', 'NPHI_mm', 'SP_mm', 'PEF_mm']]
df_form_68_trans.head(2)

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
1073249,0.434194,0.188523,0.276054,0.028375,0.536723,0.191386,NaN,0.406240
1073250,0.377015,0.175124,0.286334,0.025688,0.522386,0.142282,NaN,0.396794


In [ ]:
df_form_trans = pd.concat([df_form_1_trans,  df_form_2_trans,  df_form_3_trans,  df_form_4_trans,  df_form_5_trans,
                           df_form_42_trans, df_form_19_trans, df_form_6_trans,  df_form_7_trans,  df_form_8_trans,
                           df_form_9_trans,  df_form_10_trans, df_form_11_trans, df_form_12_trans, df_form_13_trans,
                           df_form_14_trans, df_form_15_trans, df_form_24_trans, df_form_18_trans, df_form_16_trans,
                           df_form_21_trans, df_form_22_trans, df_form_40_trans, df_form_33_trans, df_form_23_trans,
                           df_form_43_trans, df_form_46_trans, df_form_52_trans, df_form_51_trans, df_form_44_trans,
                           df_form_45_trans, df_form_55_trans, df_form_56_trans, df_form_57_trans, df_form_58_trans,
                           df_form_47_trans, df_form_48_trans, df_form_60_trans, df_form_53_trans, df_form_68_trans,
                           df_form_54_trans, 
                           df_form_sec_trans])

df_form_trans.columns = ['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm',
                         'NPHI_mm', 'SP_mm', 'PEF_mm', 
                          #'RMED_trans', 'RDEP_trans'
                        ]
df_form_trans.head(2)

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
2464,0.170890,0.587551,0.551347,0.910346,0.572659,0.489129,0.144274,0.002181
2465,0.143545,0.598294,0.568145,0.908929,0.561091,0.495443,0.142960,0.001812


In [ ]:
df_form_trans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1170511 entries, 2464 to 1169030
Data columns (total 8 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   GR_mm    1170511 non-null  float64
 1   DTC_mm   1170511 non-null  float64
 2   CALI_mm  1170511 non-null  float64
 3   DRHO_mm  1170511 non-null  float64
 4   RHOB_mm  1170511 non-null  float64
 5   NPHI_mm  1170511 non-null  float64
 6   SP_mm    1166447 non-null  float64
 7   PEF_mm   1170511 non-null  float64
dtypes: float64(8)
memory usage: 80.4 MB


In [ ]:
df_train = pd.read_csv('./train.csv', sep=';')
df_train['Lithology Keys'] = df_train['FORCE_2020_LITHOFACIES_LITHOLOGY'].replace(lithology_keys)
df_train['Lithology Num'] = df_train['Lithology Keys'].replace(lithology_num)
df_train['Confidence Level'] = df_train['FORCE_2020_LITHOFACIES_CONFIDENCE']
df_train = df_train[['WELL', 'DEPTH_MD', 'FORMATION',
                     'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 
                     'NPHI', 'SP', 'PEF',
                     'RMED', 'RDEP',
                     'FORCE_2020_LITHOFACIES_LITHOLOGY', 'FORCE_2020_LITHOFACIES_CONFIDENCE', 
                     'Lithology Keys', 'Lithology Num']]

In [ ]:
new_df_form_trans = df_train.merge(df_form_trans, left_index=True, right_index=True, how='inner')
new_df_form_trans.head(2)

,WELL,DEPTH_MD,FORMATION,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF,RMED,RDEP,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE,Lithology Keys,Lithology Num,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
0,15/9-13,494.528,NaN,80.200851,161.13118,19.480835,-0.574928,1.884186,NaN,24.612379,20.915468,1.61141,1.798681,65000,1.0,Shale,3,0.119691,0.693391,0.730399,0.468848,0.370705,0.630579,0.438254,0.056954
1,15/9-13,494.680,NaN,79.262886,160.60347,19.468800,-0.570188,1.889794,NaN,23.895531,19.383013,1.61807,1.795641,65000,1.0,Shale,3,0.118229,0.690508,0.729886,0.469620,0.373100,0.637790,0.437445,0.052761


In [ ]:
df_68_sp_last = new_df_form_trans[['FORMATION', 'GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 
                                   'RHOB_mm', 'NPHI_mm', 'SP_mm', 'PEF_mm']]

In [ ]:
imputer_68_sp_last = IterativeImputer(estimator=BayesianRidge(), imputation_order='ascending', max_iter=400, verbose=1, tol=0.0001)

In [ ]:
imputer_68_sp_last.fit(df_68_sp_last.drop(['FORMATION'], axis=1))

[IterativeImputer] Completing matrix with shape (1170511, 8)
[IterativeImputer] Change: 0.26984075213717795, scaled tolerance: 0.008497002318918348 
[IterativeImputer] Change: 0.0, scaled tolerance: 0.008497002318918348 
[IterativeImputer] Early stopping criterion reached.


IterativeImputer(estimator=BayesianRidge(), max_iter=400, tol=0.0001, verbose=1)

In [ ]:
form_comp_68_trans  = imputer_68_sp_last.transform(df_68_sp_last.drop(['FORMATION'], axis=1))

[IterativeImputer] Completing matrix with shape (1170511, 8)


In [ ]:
df_form_comp_68_trans = pd.DataFrame(form_comp_68_trans, columns=['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm', 'NPHI_mm', 'SP_mm', 'PEF_mm'], 
                                     index=df_68_sp_last.index)

In [ ]:
df_form_comp_68_trans.describe()

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
count,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06
mean,2.207431e-01,4.806518e-01,3.959028e-01,5.098526e-01,5.278249e-01,4.261566e-01,4.829458e-01,4.465455e-02
std,1.598461e-01,1.690342e-01,2.054044e-01,2.598591e-01,1.309769e-01,1.668942e-01,2.848388e-01,8.959116e-02
min,0.000000e+00,-4.814331e+01,-9.925236e-01,-4.303741e-01,-8.961315e-01,-8.497002e+01,-1.712155e+00,-8.074584e-01
25%,9.980838e-02,3.521768e-01,2.595733e-01,2.919477e-01,4.434298e-01,3.451218e-01,3.025162e-01,9.340970e-03
50%,1.692883e-01,4.893000e-01,3.851808e-01,5.503278e-01,5.143984e-01,4.354696e-01,4.675996e-01,1.914297e-02
75%,3.073412e-01,6.126025e-01,5.370021e-01,6.638518e-01,6.124128e-01,5.065275e-01,6.285066e-01,3.966071e-02
max,1.000000e+00,1.492584e+00,1.279080e+00,1.003453e+00,1.104305e+00,6.592093e+00,2.660879e+01,1.975035e+00


In [ ]:
df_train = pd.read_csv('./train.csv', sep=';')
df_train['Lithology Keys'] = df_train['FORCE_2020_LITHOFACIES_LITHOLOGY'].replace(lithology_keys)
df_train['Lithology Num'] = df_train['Lithology Keys'].replace(lithology_num)
df_train['Confidence Level'] = df_train['FORCE_2020_LITHOFACIES_CONFIDENCE']
df_train = df_train[['WELL', 'DEPTH_MD', 'FORMATION',
                     'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 
                     'NPHI', 'SP', 'PEF',
                     'RMED', 'RDEP',
                     'FORCE_2020_LITHOFACIES_LITHOLOGY', 'FORCE_2020_LITHOFACIES_CONFIDENCE', 
                     'Lithology Keys', 'Lithology Num']]

In [ ]:
new_df_form_trans_last = df_train.merge(df_form_comp_68_trans, left_index=True, right_index=True, how='inner')
new_df_form_trans_last.head(2)

,WELL,DEPTH_MD,FORMATION,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF,RMED,RDEP,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE,Lithology Keys,Lithology Num,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
0,15/9-13,494.528,NaN,80.200851,161.13118,19.480835,-0.574928,1.884186,NaN,24.612379,20.915468,1.61141,1.798681,65000,1.0,Shale,3,0.119691,0.693391,0.730399,0.468848,0.370705,0.630579,0.438254,0.056954
1,15/9-13,494.680,NaN,79.262886,160.60347,19.468800,-0.570188,1.889794,NaN,23.895531,19.383013,1.61807,1.795641,65000,1.0,Shale,3,0.118229,0.690508,0.729886,0.469620,0.373100,0.637790,0.437445,0.052761


In [ ]:
new_df_form_trans_last.describe()

,DEPTH_MD,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF,RMED,RDEP,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE,Lithology Num,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
count,1.170511e+06,1.170511e+06,1.089648e+06,1.082634e+06,987857.000000,1.009242e+06,765409.000000,864247.000000,671692.000000,1.131518e+06,1.159496e+06,1.170511e+06,1.170332e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06,1.170511e+06
mean,2.184087e+03,7.091370e+01,1.133581e+02,1.318568e+01,0.012196,2.284987e+00,0.331966,60.033980,6.319719,4.986978e+00,1.069103e+01,6.138598e+04,1.164258e+00,2.953153e+00,2.207431e-01,4.806518e-01,3.959028e-01,5.098526e-01,5.278249e-01,4.261566e-01,4.829458e-01,4.465455e-02
std,9.971821e+02,3.423149e+01,2.999050e+01,3.798907e+00,7.477798,2.532835e-01,0.130483,76.574359,10.955360,5.467269e+01,1.139480e+02,1.389170e+04,4.451180e-01,1.541950e+00,1.598461e-01,1.690342e-01,2.054044e-01,2.598591e-01,1.309769e-01,1.668942e-01,2.848388e-01,8.959116e-02
min,1.360860e+02,1.092843e-01,7.415132e+00,2.344000e+00,-7429.338867,7.209712e-01,-0.035822,-999.000000,0.099718,-8.418695e-03,3.170056e-02,3.000000e+04,1.000000e+00,1.000000e+00,0.000000e+00,-4.814331e+01,-9.925236e-01,-4.303741e-01,-8.961315e-01,-8.497002e+01,-1.712155e+00,-8.074584e-01
25%,1.418597e+03,4.762722e+01,8.782584e+01,9.429712e+00,-0.009253,2.092203e+00,0.241365,32.402981,3.414440,9.140862e-01,9.102396e-01,6.500000e+04,1.000000e+00,2.000000e+00,9.980838e-02,3.521768e-01,2.595733e-01,2.919477e-01,4.434298e-01,3.451218e-01,3.025162e-01,9.340970e-03
50%,2.076605e+03,6.836763e+01,1.095852e+02,1.255575e+01,0.001752,2.321228e+00,0.326800,55.390942,4.313530,1.443584e+00,1.439000e+00,6.500000e+04,1.000000e+00,3.000000e+00,1.692883e-01,4.893000e-01,3.851808e-01,5.503278e-01,5.143984e-01,4.354696e-01,4.675996e-01,1.914297e-02
75%,2.864393e+03,8.903551e+01,1.407745e+02,1.671075e+01,0.021702,2.488580e+00,0.422951,83.393299,5.968794,2.680930e+00,2.557220e+00,6.500000e+04,1.000000e+00,3.000000e+00,3.073412e-01,6.126025e-01,5.370021e-01,6.638518e-01,6.124128e-01,5.065275e-01,6.285066e-01,3.966071e-02
max,5.436632e+03,1.076964e+03,3.204789e+02,2.827900e+01,2.836938,3.457820e+00,0.999570,526.547302,383.129974,1.988616e+03,1.999887e+03,9.900000e+04,3.000000e+00,1.200000e+01,1.000000e+00,1.492584e+00,1.279080e+00,1.003453e+00,1.104305e+00,6.592093e+00,2.660879e+01,1.975035e+00


In [ ]:
df_train_mm_all_imp  = imputer_all.transform(df_train_mm_all)

[IterativeImputer] Completing matrix with shape (1170511, 8)


In [ ]:
df_train_mm_all_imp_trans = pd.DataFrame(df_train_mm_all_imp, columns=df_train_mm_all.columns, index=df_train_mm_all.index)

In [ ]:
df_train_mm_all_imp_trans.head(2)

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
0,0.271305,1.592942,1.657098,-0.078516,-1.582421,2.081750,-0.462578,1.332294
1,0.243904,1.575346,1.653930,-0.077882,-1.560281,2.049298,-0.471940,1.192412


In [ ]:
new_df_train_mm_all_imp_trans = df_train.merge(df_train_mm_all_imp_trans, left_index=True, right_index=True, how='outer')
new_df_train_mm_all_imp_trans.head(2)

,WELL,DEPTH_MD,FORMATION,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF,RMED,RDEP,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE,Lithology Keys,Lithology Num,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
0,15/9-13,494.528,NaN,80.200851,161.13118,19.480835,-0.574928,1.884186,NaN,24.612379,20.915468,1.61141,1.798681,65000,1.0,Shale,3,0.271305,1.592942,1.657098,-0.078516,-1.582421,2.081750,-0.462578,1.332294
1,15/9-13,494.680,NaN,79.262886,160.60347,19.468800,-0.570188,1.889794,NaN,23.895531,19.383013,1.61807,1.795641,65000,1.0,Shale,3,0.243904,1.575346,1.653930,-0.077882,-1.560281,2.049298,-0.471940,1.192412


In [ ]:
limestone = new_df_form_trans_last[new_df_form_trans_last['Lithology Keys'] == 'Limestone']
marl = new_df_form_trans_last[new_df_form_trans_last['Lithology Keys'] == 'Marl']
tuff = new_df_form_trans_last[new_df_form_trans_last['Lithology Keys'] == 'Tuff']
chalk = new_df_form_trans_last[new_df_form_trans_last['Lithology Keys'] == 'Chalk']
halite = new_df_form_trans_last[new_df_form_trans_last['Lithology Keys'] == 'Halite']
coal = new_df_form_trans_last[new_df_form_trans_last['Lithology Keys'] == 'Coal']
dolomite = new_df_form_trans_last[new_df_form_trans_last['Lithology Keys'] == 'Dolomite']
anhydrite = new_df_form_trans_last[new_df_form_trans_last['Lithology Keys'] == 'Anhydrite']
basement = new_df_form_trans_last[new_df_form_trans_last['Lithology Keys'] == 'Basement']

In [ ]:
df_tuff      = new_df_form_trans_last.append([tuff]*2, ignore_index=True)

In [ ]:
df_chalk     = df_tuff.append([chalk]*3, ignore_index=True)

In [ ]:
df_halite    = df_chalk.append([halite]*4, ignore_index=True)

In [ ]:
df_coal      = df_halite.append([coal]*8, ignore_index=True)

In [ ]:
df_dolomite  = df_coal.append([dolomite]*20, ignore_index=True)

In [ ]:
df_anhydrite = df_dolomite.append([anhydrite]*30, ignore_index=True)

In [ ]:
df_basement  = df_anhydrite.append([basement]*300, ignore_index=True)

In [ ]:
df_train_form_trans_merge_mm_copylith = df_basement.copy()

In [ ]:
df_form_trans = df_train_form_trans_merge_mm_copylith.copy()

In [ ]:
df_form_trans['Lithology Keys'].unique()

array(['Shale', 'Sandstone', 'Sandstone/Shale', 'Limestone', 'Tuff',
       'Marl', 'Anhydrite', 'Dolomite', 'Chalk', 'Coal', 'Halite',
       'Basement'], dtype=object)

In [ ]:
lithology_numbers = {30000: 0,           65030: 1,                 65000: 2,           80000: 3,      74000: 4,          70000: 5, 
                     70032: 6,           88000: 7,                 86000: 8,           99000: 9,      90000: 10,         93000: 11}
lithology_keys    = {30000: 'Sandstone', 65030: 'Sandstone/Shale', 65000: 'Shale',     80000: 'Marl', 74000: 'Dolomite', 70000: 'Limestone',
                     70032: 'Chalk',     88000: 'Halite',          86000: 'Anhydrite', 99000: 'Tuff', 90000: 'Coal',     93000: 'Basement'}
lithology_int     = {0: 'Sandstone',     1: 'Sandstone/Shale',     2: 'Shale',         3: 'Marl',     4: 'Dolomite',     5: 'Limestone',
                     6: 'Chalk',         7: 'Halite',              8: 'Anhydrite',     9: 'Tuff',     10: 'Coal',        11: 'Basement'}

In [ ]:
df_X = df_form_trans[[#'GR_new', 'DTC_new', 'CALI_new', 'DRHO_new', 'RHOB_new', 
                       #              'NPHI_new', 'SP_new', 'PEF_new',
                        #             'RMED_new', 'RDEP_new',
                                     'GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm',
                                     'NPHI_mm', 'SP_mm', 'PEF_mm',
                         #            'RMED_mm', 'RDEP_mm'
                                     ]]
df_X.head(2)

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
0,0.119691,0.693391,0.730399,0.468848,0.370705,0.630579,0.438254,0.056954
1,0.118229,0.690508,0.729886,0.469620,0.373100,0.637790,0.437445,0.052761


In [ ]:
df_X.describe()

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
count,1.393162e+06,1.393162e+06,1.393162e+06,1.393162e+06,1.393162e+06,1.393162e+06,1.393162e+06,1.393162e+06
mean,2.052839e-01,4.577039e-01,4.000623e-01,5.344857e-01,5.394588e-01,4.051697e-01,4.678682e-01,5.311074e-02
std,1.605621e-01,1.828449e-01,2.028295e-01,2.695871e-01,1.472869e-01,1.806785e-01,2.800806e-01,1.120443e-01
min,0.000000e+00,-4.814331e+01,-9.925236e-01,-4.303741e-01,-8.961315e-01,-8.497002e+01,-1.712155e+00,-8.074584e-01
25%,8.633314e-02,3.220459e-01,2.660114e-01,3.111788e-01,4.450267e-01,3.117599e-01,2.562387e-01,8.883711e-03
50%,1.531129e-01,4.641536e-01,3.919315e-01,5.595020e-01,5.223968e-01,4.247426e-01,4.658998e-01,1.949705e-02
75%,2.927985e-01,6.039577e-01,5.325667e-01,8.289450e-01,6.327417e-01,5.034139e-01,6.205244e-01,4.615802e-02
max,1.000000e+00,1.492584e+00,1.279080e+00,1.003453e+00,1.104305e+00,6.592093e+00,2.660879e+01,1.975035e+00


In [ ]:
#df_y = pd.get_dummies(df_train_scaled['Lithology Keys'])
#df_y = df_train_scaled['FORCE_2020_LITHOFACIES_LITHOLOGY']

df_y = df_form_trans['FORCE_2020_LITHOFACIES_LITHOLOGY']
y_int = df_y.map(lithology_numbers)
y_str = df_y.map(lithology_keys)
y_str.value_counts()

Shale              720803
Sandstone          168937
Sandstone/Shale    150455
Limestone           56320
Tuff                45735
Chalk               42052
Halite              41065
Dolomite            35448
Coal                34380
Anhydrite           33635
Marl                33329
Basement            31003
Name: FORCE_2020_LITHOFACIES_LITHOLOGY, dtype: int64

In [ ]:
A = np.load('penalty_matrix.npy')

def score(y_true, y_pred):
    S = 0.0
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    for i in range(0, y_true.shape[0]):
        S -= A[y_true[i], y_pred[i]]
    return S/y_true.shape[0]

In [ ]:
X = df_X.copy()
#y = df_y.copy()
y = y_int.copy()

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393162 entries, 0 to 1393161
Data columns (total 8 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   GR_mm    1393162 non-null  float64
 1   DTC_mm   1393162 non-null  float64
 2   CALI_mm  1393162 non-null  float64
 3   DRHO_mm  1393162 non-null  float64
 4   RHOB_mm  1393162 non-null  float64
 5   NPHI_mm  1393162 non-null  float64
 6   SP_mm    1393162 non-null  float64
 7   PEF_mm   1393162 non-null  float64
dtypes: float64(8)
memory usage: 85.0 MB


In [ ]:
scaler_mm = MinMaxScaler().fit(X)

In [ ]:
X_scaled_mm = pd.DataFrame(scaler_mm.transform(X), index=X.index,
                           columns=['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm', 
                                    'NPHI_mm', 'SP_mm', 'PEF_mm',
                                    #'RMED_mm', 'RDEP_mm'
                                    ])
X_scaled_mm.head(2)

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
0,0.074375,0.999706,0.766603,0.999541,0.621923,0.999340,0.993593,0.274315
1,0.073504,0.999705,0.766284,0.999542,0.623270,0.999337,0.993591,0.271245


In [ ]:
X_merge_mm = X.merge(X_scaled_mm, left_index=True, right_index=True, how='inner')
X_merge_mm.head(2)

,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,RMED_new,RDEP_new,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,RMED_mm,RDEP_mm
2464,44.851158,138.449585,18.544184,0.000899,2.098715,1.210955,1.332334,0.041549,0.998699,0.768752,0.999618,0.385849,0.384348,0.135836
2465,37.774338,140.845428,18.908665,-0.008508,2.076638,1.082164,1.205886,0.034977,0.998716,0.776400,0.999617,0.381906,0.384308,0.135781


In [ ]:
scaler_std = StandardScaler().fit(X)

In [ ]:
X_scaled_std = pd.DataFrame(scaler_std.transform(X), index=X.index,
                            columns=['GR_std', 'DTC_std', 'CALI_std', 'DRHO_std', 'RHOB_std', 
                                     'NPHI_std', 'SP_std', 'PEF_std',
                                     #'RMED_std', 'RDEP_std'
                                     ])
X_scaled_std.head(2)

,GR_std,DTC_std,CALI_std,DRHO_std,RHOB_std,NPHI_std,SP_std,PEF_std
0,0.409518,0.113243,1.713833,-0.092753,-1.400901,0.038043,-0.155714,1.372699
1,0.383208,0.112097,1.710534,-0.092001,-1.381104,0.035157,-0.158568,1.225324


In [ ]:
X_merge_std = X.merge(X_scaled_std, left_index=True, right_index=True, how='inner')
X_merge_std.head(2)

,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,RMED_new,RDEP_new,GR_std,DTC_std,CALI_std,DRHO_std,RHOB_std,RMED_std,RDEP_std
2464,44.851158,138.449585,18.544184,0.000899,2.098715,1.210955,1.332334,-0.754426,0.192296,1.571818,-0.001287,-0.794172,-0.076839,-0.107035
2465,37.774338,140.845428,18.908665,-0.008508,2.076638,1.082164,1.205886,-0.954089,0.207734,1.673365,-0.002522,-0.882631,-0.078988,-0.107918


In [ ]:
# Split into training, validation, and test sets (80/20)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.0001, random_state=90, stratify=y)
#X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=2)

#print("Train set size: {0}, Val set size: {1}, Test set size: {2}".format(len(X_train), len(X_val), len(X_test)))
print("Train set size: {0}, Test set size: {1}".format(len(X_train), len(X_test)))

Train set size: 1393022, Test set size: 140


In [ ]:
X_train.head(2)

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
270569,0.044694,0.565328,0.678007,0.559208,0.461499,0.423530,0.455065,0.006207
227330,0.232827,0.580648,0.495725,0.911716,0.569637,0.501769,0.107628,0.005719


In [ ]:
# mm_per_form, then impute, all row; no RMED RDEP, with NPHI SP PEF; copy litho; ascending
lgbmclass = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                           importance_type='split', learning_rate=0.1, max_depth=-1,
                           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                           n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                           subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model_merge_mm = make_pipeline(lgbmclass)
model_merge_mm.fit(X_train, y_train)
y_pred_test = model_merge_mm.predict(X_test)

print('Model Score                   : ', model_merge_mm.score(X_train, y_train))
print('Root Mean Squared error       : ', sqrt(mean_squared_error(y_test.values, y_pred_test)))
print('Custom Score (penalty_matrix) : ', score(y_test.values, y_pred_test))
# Xeek score: 0.

Model Score                   :  0.8383277507462193
Root Mean Squared error       :  1.2983506020002016
Custom Score (penalty_matrix) :  -0.44910714285714287


In [ ]:
# mm_per_form, then impute, all row; no RMED RDEP, with NPHI SP PEF; copy litho; ascending
lgbmclass = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                           importance_type='split', learning_rate=0.1, max_depth=-1,
                           min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                           n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                           random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
                           subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

model_merge_mm = make_pipeline(lgbmclass)
model_merge_mm.fit(X_train, y_train)
y_pred_test = model_merge_mm.predict(X_test)

print('Model Score                   : ', model_merge_mm.score(X_train, y_train))
print('Root Mean Squared error       : ', sqrt(mean_squared_error(y_test.values, y_pred_test)))
print('Custom Score (penalty_matrix) : ', score(y_test.values, y_pred_test))
# Xeek score: 0.

Model Score                   :  0.8380750627053988
Root Mean Squared error       :  1.118033988749895
Custom Score (penalty_matrix) :  -0.43392857142857144


## df_test Prediction: Iterative Impute

In [ ]:
pickle.dump(model_merge_mm, open('../Submission/new_model_mm_1.pkl', 'wb'))
#pickle.dump(scaler, open('../Submission/scaler_xgb_3.pkl', 'wb'))
#pickle.dump(imputer_all, open('../Submission/imputer_all.pkl', 'wb'))

In [ ]:
model = pickle.load(open('../Submission/model_xgb_2_trans.pkl', 'rb'))
#scaler = pickle.load(open('../Submission/scaler_lgbm_5.pkl', 'rb'))
#imputer_all = pickle.load(open('../Submission/imputer_all.pkl', 'rb'))

In [ ]:
df_test = pd.read_csv('./test.csv', sep=';')
df_test.head(2)

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,RDEP,RHOB,GR,SGR,NPHI,PEF,DTC,SP,BS,ROP,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO
0,15/9-14,480.628001,423244.5,6461862.5,-455.624420,NORDLAND GP.,NaN,19.2031,NaN,1.613886,1.513937,2.171173,72.078377,NaN,0.551575,55.815659,174.347092,35.525719,NaN,96.461990,NaN,NaN,-0.538873,0.130611,NaN,NaN,NaN
1,15/9-14,480.780001,423244.5,6461862.5,-455.776428,NORDLAND GP.,NaN,19.2031,NaN,1.574376,1.480188,2.171840,72.147697,NaN,0.547566,41.862537,176.112915,36.158520,NaN,96.454399,NaN,NaN,-0.539232,0.130611,NaN,NaN,NaN


In [ ]:
df_test_log = df_test[['GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 
                       'NPHI', 'SP', 'PEF',
                       #'RMED', 'RDEP'
                       ]]
df_test_log.head(2)

,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF
0,72.078377,174.347092,19.2031,-0.538873,2.171173,0.551575,35.525719,55.815659
1,72.147697,176.112915,19.2031,-0.539232,2.171840,0.547566,36.158520,41.862537


In [ ]:
#test_mm_all  = pd.DataFrame(sc_all.transform(df_test_log), df_test_log.index, mm_cols)

In [ ]:
df_test = df_test[['FORMATION', 'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 
                   'NPHI', 'SP', 'PEF',
                   #'RMED', 'RDEP'
                   ]]

In [ ]:
df_test['FORMATION'].fillna('no_form', inplace=True)

In [ ]:
test_form = df_test['FORMATION'].unique()

In [ ]:
test_form_1  = df_test[df_test['FORMATION'] == test_form[1]]
test_form_2  = df_test[df_test['FORMATION'] == test_form[2]]
test_form_3  = df_test[df_test['FORMATION'] == test_form[3]]
test_form_4  = df_test[df_test['FORMATION'] == test_form[4]]
test_form_5  = df_test[df_test['FORMATION'] == test_form[5]]
test_form_6  = df_test[df_test['FORMATION'] == test_form[6]]
test_form_7  = df_test[df_test['FORMATION'] == test_form[7]]
test_form_8  = df_test[df_test['FORMATION'] == test_form[8]]
test_form_9  = df_test[df_test['FORMATION'] == test_form[9]]
test_form_10  = df_test[df_test['FORMATION'] == test_form[10]]

test_form_11  = df_test[df_test['FORMATION'] == test_form[11]]
test_form_12  = df_test[df_test['FORMATION'] == test_form[12]]
test_form_13  = df_test[df_test['FORMATION'] == test_form[13]]
test_form_14  = df_test[df_test['FORMATION'] == test_form[14]]
test_form_15  = df_test[df_test['FORMATION'] == test_form[15]]
test_form_16  = df_test[df_test['FORMATION'] == test_form[16]]
test_form_17  = df_test[df_test['FORMATION'] == test_form[17]]
test_form_18  = df_test[df_test['FORMATION'] == test_form[18]]
test_form_19  = df_test[df_test['FORMATION'] == test_form[19]]
test_form_20  = df_test[df_test['FORMATION'] == test_form[20]]

test_form_21  = df_test[df_test['FORMATION'] == test_form[21]]
test_form_22  = df_test[df_test['FORMATION'] == test_form[22]]
test_form_23  = df_test[df_test['FORMATION'] == test_form[23]]
test_form_24  = df_test[df_test['FORMATION'] == test_form[24]]
test_form_25  = df_test[df_test['FORMATION'] == test_form[25]]
test_form_26  = df_test[df_test['FORMATION'] == test_form[26]]
test_form_27  = df_test[df_test['FORMATION'] == test_form[27]]
test_form_28  = df_test[df_test['FORMATION'] == test_form[28]]
test_form_29  = df_test[df_test['FORMATION'] == test_form[29]]
test_form_30  = df_test[df_test['FORMATION'] == test_form[30]]

test_form_31  = df_test[df_test['FORMATION'] == test_form[31]]
test_form_32  = df_test[df_test['FORMATION'] == test_form[32]]
test_form_33  = df_test[df_test['FORMATION'] == test_form[33]]
test_form_34  = df_test[df_test['FORMATION'] == test_form[34]]
test_form_35  = df_test[df_test['FORMATION'] == test_form[35]]
test_form_36  = df_test[df_test['FORMATION'] == test_form[36]]
test_form_37  = df_test[df_test['FORMATION'] == test_form[37]]
test_form_38  = df_test[df_test['FORMATION'] == test_form[38]]
test_form_39  = df_test[df_test['FORMATION'] == test_form[39]]
test_form_40  = df_test[df_test['FORMATION'] == test_form[40]]

test_form_41  = df_test[df_test['FORMATION'] == test_form[41]]

test_form_sec  = df_test[df_test['FORMATION'] == test_form[0]]

In [ ]:
sc_2

MinMaxScaler(feature_range=(0.0, 1.0))

In [ ]:
test_form_1_merge_mm  = pd.DataFrame(sc_1.transform(test_form_1.drop(['FORMATION'], axis=1)), test_form_1.index, mm_cols)
test_form_2_merge_mm  = pd.DataFrame(sc_2.transform(test_form_2.drop(['FORMATION'], axis=1)), test_form_2.index, mm_cols)
test_form_3_merge_mm  = pd.DataFrame(sc_3.transform(test_form_3.drop(['FORMATION'], axis=1)), test_form_3.index, mm_cols)
test_form_4_merge_mm  = pd.DataFrame(sc_4.transform(test_form_4.drop(['FORMATION'], axis=1)), test_form_4.index, mm_cols)
test_form_5_merge_mm  = pd.DataFrame(sc_5.transform(test_form_5.drop(['FORMATION'], axis=1)), test_form_5.index, mm_cols)
test_form_6_merge_mm  = pd.DataFrame(sc_6.transform(test_form_6.drop(['FORMATION'], axis=1)), test_form_6.index, mm_cols)
test_form_7_merge_mm  = pd.DataFrame(sc_7.transform(test_form_7.drop(['FORMATION'], axis=1)), test_form_7.index, mm_cols)
test_form_8_merge_mm  = pd.DataFrame(sc_8.transform(test_form_8.drop(['FORMATION'], axis=1)), test_form_8.index, mm_cols)
test_form_9_merge_mm  = pd.DataFrame(sc_9.transform(test_form_9.drop(['FORMATION'], axis=1)), test_form_9.index, mm_cols)
test_form_10_merge_mm  = pd.DataFrame(sc_10.transform(test_form_10.drop(['FORMATION'], axis=1)), test_form_10.index, mm_cols)

test_form_11_merge_mm  = pd.DataFrame(sc_11.transform(test_form_11.drop(['FORMATION'], axis=1)), test_form_11.index, mm_cols)
test_form_12_merge_mm  = pd.DataFrame(sc_12.transform(test_form_12.drop(['FORMATION'], axis=1)), test_form_12.index, mm_cols)
test_form_13_merge_mm  = pd.DataFrame(sc_13.transform(test_form_13.drop(['FORMATION'], axis=1)), test_form_13.index, mm_cols)
test_form_14_merge_mm  = pd.DataFrame(sc_14.transform(test_form_14.drop(['FORMATION'], axis=1)), test_form_14.index, mm_cols)
test_form_15_merge_mm  = pd.DataFrame(sc_15.transform(test_form_15.drop(['FORMATION'], axis=1)), test_form_15.index, mm_cols)
test_form_16_merge_mm  = pd.DataFrame(sc_16.transform(test_form_16.drop(['FORMATION'], axis=1)), test_form_16.index, mm_cols)
test_form_17_merge_mm  = pd.DataFrame(sc_17.transform(test_form_17.drop(['FORMATION'], axis=1)), test_form_17.index, mm_cols)
test_form_18_merge_mm  = pd.DataFrame(sc_18.transform(test_form_18.drop(['FORMATION'], axis=1)), test_form_18.index, mm_cols)
test_form_19_merge_mm  = pd.DataFrame(sc_19.transform(test_form_19.drop(['FORMATION'], axis=1)), test_form_19.index, mm_cols)
test_form_20_merge_mm  = pd.DataFrame(sc_20.transform(test_form_20.drop(['FORMATION'], axis=1)), test_form_20.index, mm_cols)

test_form_21_merge_mm  = pd.DataFrame(sc_21.transform(test_form_21.drop(['FORMATION'], axis=1)), test_form_21.index, mm_cols)
test_form_22_merge_mm  = pd.DataFrame(sc_22.transform(test_form_22.drop(['FORMATION'], axis=1)), test_form_22.index, mm_cols)
test_form_23_merge_mm  = pd.DataFrame(sc_23.transform(test_form_23.drop(['FORMATION'], axis=1)), test_form_23.index, mm_cols)
test_form_24_merge_mm  = pd.DataFrame(sc_24.transform(test_form_24.drop(['FORMATION'], axis=1)), test_form_24.index, mm_cols)
test_form_25_merge_mm  = pd.DataFrame(sc_25.transform(test_form_25.drop(['FORMATION'], axis=1)), test_form_25.index, mm_cols)
test_form_26_merge_mm  = pd.DataFrame(sc_26.transform(test_form_26.drop(['FORMATION'], axis=1)), test_form_26.index, mm_cols)
test_form_27_merge_mm  = pd.DataFrame(sc_27.transform(test_form_27.drop(['FORMATION'], axis=1)), test_form_27.index, mm_cols)
test_form_28_merge_mm  = pd.DataFrame(sc_28.transform(test_form_28.drop(['FORMATION'], axis=1)), test_form_28.index, mm_cols)
test_form_29_merge_mm  = pd.DataFrame(sc_29.transform(test_form_29.drop(['FORMATION'], axis=1)), test_form_29.index, mm_cols)
test_form_30_merge_mm  = pd.DataFrame(sc_30.transform(test_form_30.drop(['FORMATION'], axis=1)), test_form_30.index, mm_cols)

test_form_31_merge_mm  = pd.DataFrame(sc_31.transform(test_form_31.drop(['FORMATION'], axis=1)), test_form_31.index, mm_cols)
test_form_32_merge_mm  = pd.DataFrame(sc_32.transform(test_form_32.drop(['FORMATION'], axis=1)), test_form_32.index, mm_cols)
test_form_33_merge_mm  = pd.DataFrame(sc_33.transform(test_form_33.drop(['FORMATION'], axis=1)), test_form_33.index, mm_cols)
test_form_34_merge_mm  = pd.DataFrame(sc_34.transform(test_form_34.drop(['FORMATION'], axis=1)), test_form_34.index, mm_cols)
test_form_35_merge_mm  = pd.DataFrame(sc_35.transform(test_form_35.drop(['FORMATION'], axis=1)), test_form_35.index, mm_cols)
test_form_36_merge_mm  = pd.DataFrame(sc_36.transform(test_form_36.drop(['FORMATION'], axis=1)), test_form_36.index, mm_cols)
test_form_37_merge_mm  = pd.DataFrame(sc_37.transform(test_form_37.drop(['FORMATION'], axis=1)), test_form_37.index, mm_cols)
test_form_38_merge_mm  = pd.DataFrame(sc_38.transform(test_form_38.drop(['FORMATION'], axis=1)), test_form_38.index, mm_cols)
test_form_39_merge_mm  = pd.DataFrame(sc_39.transform(test_form_39.drop(['FORMATION'], axis=1)), test_form_39.index, mm_cols)
test_form_40_merge_mm  = pd.DataFrame(sc_40.transform(test_form_40.drop(['FORMATION'], axis=1)), test_form_40.index, mm_cols)

test_form_41_merge_mm  = pd.DataFrame(sc_41.transform(test_form_41.drop(['FORMATION'], axis=1)), test_form_41.index, mm_cols)

test_form_sec_merge_mm  = pd.DataFrame(sc_sec.transform(test_form_sec.drop(['FORMATION'], axis=1)), test_form_sec.index, mm_cols)

In [ ]:
# fit_transform
test_form_1_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_1.drop(['FORMATION'], axis=1)), test_form_1.index, mm_cols)
test_form_2_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_2.drop(['FORMATION'], axis=1)), test_form_2.index, mm_cols)
test_form_3_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_3.drop(['FORMATION'], axis=1)), test_form_3.index, mm_cols)
test_form_4_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_4.drop(['FORMATION'], axis=1)), test_form_4.index, mm_cols)
test_form_5_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_5.drop(['FORMATION'], axis=1)), test_form_5.index, mm_cols)
test_form_6_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_6.drop(['FORMATION'], axis=1)), test_form_6.index, mm_cols)
test_form_7_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_7.drop(['FORMATION'], axis=1)), test_form_7.index, mm_cols)
test_form_8_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_8.drop(['FORMATION'], axis=1)), test_form_8.index, mm_cols)
test_form_9_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_9.drop(['FORMATION'], axis=1)), test_form_9.index, mm_cols)
test_form_10_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_10.drop(['FORMATION'], axis=1)), test_form_10.index, mm_cols)

test_form_11_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_11.drop(['FORMATION'], axis=1)), test_form_11.index, mm_cols)
test_form_12_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_12.drop(['FORMATION'], axis=1)), test_form_12.index, mm_cols)
test_form_13_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_13.drop(['FORMATION'], axis=1)), test_form_13.index, mm_cols)
test_form_14_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_14.drop(['FORMATION'], axis=1)), test_form_14.index, mm_cols)
test_form_15_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_15.drop(['FORMATION'], axis=1)), test_form_15.index, mm_cols)
test_form_16_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_16.drop(['FORMATION'], axis=1)), test_form_16.index, mm_cols)
test_form_17_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_17.drop(['FORMATION'], axis=1)), test_form_17.index, mm_cols)
test_form_18_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_18.drop(['FORMATION'], axis=1)), test_form_18.index, mm_cols)
test_form_19_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_19.drop(['FORMATION'], axis=1)), test_form_19.index, mm_cols)
test_form_20_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_20.drop(['FORMATION'], axis=1)), test_form_20.index, mm_cols)

test_form_21_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_21.drop(['FORMATION'], axis=1)), test_form_21.index, mm_cols)
test_form_22_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_22.drop(['FORMATION'], axis=1)), test_form_22.index, mm_cols)
test_form_23_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_23.drop(['FORMATION'], axis=1)), test_form_23.index, mm_cols)
test_form_24_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_24.drop(['FORMATION'], axis=1)), test_form_24.index, mm_cols)
test_form_25_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_25.drop(['FORMATION'], axis=1)), test_form_25.index, mm_cols)
test_form_26_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_26.drop(['FORMATION'], axis=1)), test_form_26.index, mm_cols)
test_form_27_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_27.drop(['FORMATION'], axis=1)), test_form_27.index, mm_cols)
test_form_28_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_28.drop(['FORMATION'], axis=1)), test_form_28.index, mm_cols)
test_form_29_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_29.drop(['FORMATION'], axis=1)), test_form_29.index, mm_cols)
test_form_30_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_30.drop(['FORMATION'], axis=1)), test_form_30.index, mm_cols)

test_form_31_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_31.drop(['FORMATION'], axis=1)), test_form_31.index, mm_cols)
test_form_32_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_32.drop(['FORMATION'], axis=1)), test_form_32.index, mm_cols)
test_form_33_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_33.drop(['FORMATION'], axis=1)), test_form_33.index, mm_cols)
test_form_34_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_34.drop(['FORMATION'], axis=1)), test_form_34.index, mm_cols)
test_form_35_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_35.drop(['FORMATION'], axis=1)), test_form_35.index, mm_cols)
test_form_36_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_36.drop(['FORMATION'], axis=1)), test_form_36.index, mm_cols)
test_form_37_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_37.drop(['FORMATION'], axis=1)), test_form_37.index, mm_cols)
test_form_38_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_38.drop(['FORMATION'], axis=1)), test_form_38.index, mm_cols)
test_form_39_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_39.drop(['FORMATION'], axis=1)), test_form_39.index, mm_cols)
test_form_40_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_40.drop(['FORMATION'], axis=1)), test_form_40.index, mm_cols)

test_form_41_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_41.drop(['FORMATION'], axis=1)), test_form_41.index, mm_cols)

test_form_sec_merge_mm  = pd.DataFrame(MinMaxScaler().fit_transform(test_form_sec.drop(['FORMATION'], axis=1)), test_form_sec.index, mm_cols)

In [ ]:
test_form_1_trans  = imputer_1.transform(test_form_1_merge_mm)
test_form_2_trans  = imputer_2.transform(test_form_2_merge_mm)
test_form_3_trans  = imputer_3.transform(test_form_3_merge_mm)
test_form_4_trans  = imputer_4.transform(test_form_4_merge_mm)
test_form_5_trans  = imputer_5.transform(test_form_5_merge_mm)
test_form_6_trans  = imputer_6.transform(test_form_6_merge_mm)
test_form_7_trans  = imputer_7.transform(test_form_7_merge_mm)
test_form_8_trans  = imputer_8.transform(test_form_8_merge_mm)
test_form_9_trans  = imputer_9.transform(test_form_9_merge_mm)
test_form_10_trans = imputer_10.transform(test_form_10_merge_mm)

test_form_11_trans  = imputer_11.transform(test_form_11_merge_mm)
test_form_12_trans  = imputer_12.transform(test_form_12_merge_mm)
test_form_13_trans  = imputer_13.transform(test_form_13_merge_mm)
test_form_14_trans  = imputer_14.transform(test_form_14_merge_mm)
test_form_15_trans  = imputer_15.transform(test_form_15_merge_mm)
test_form_16_trans  = imputer_16.transform(test_form_16_merge_mm)
test_form_17_trans  = imputer_17.transform(test_form_17_merge_mm)
test_form_18_trans  = imputer_18.transform(test_form_18_merge_mm)
test_form_19_trans  = imputer_19.transform(test_form_19_merge_mm)
test_form_20_trans  = imputer_20.transform(test_form_20_merge_mm)

test_form_21_trans  = imputer_21.transform(test_form_21_merge_mm)
test_form_22_trans  = imputer_22.transform(test_form_22_merge_mm)
test_form_23_trans  = imputer_23.transform(test_form_23_merge_mm)
test_form_24_trans  = imputer_24.transform(test_form_24_merge_mm)
test_form_25_trans  = imputer_25.transform(test_form_25_merge_mm)
test_form_26_trans  = imputer_26.transform(test_form_26_merge_mm)
test_form_27_trans  = imputer_27.transform(test_form_27_merge_mm)
test_form_28_trans  = imputer_28.transform(test_form_28_merge_mm)
test_form_29_trans  = imputer_29.transform(test_form_29_merge_mm)
test_form_30_trans = imputer_30.transform(test_form_30_merge_mm)

test_form_31_trans  = imputer_31.transform(test_form_31_merge_mm)
test_form_32_trans  = imputer_32.transform(test_form_32_merge_mm)
test_form_33_trans  = imputer_33.transform(test_form_33_merge_mm)
test_form_34_trans  = imputer_34.transform(test_form_34_merge_mm)
test_form_35_trans  = imputer_35.transform(test_form_35_merge_mm)
test_form_36_trans  = imputer_36.transform(test_form_36_merge_mm)
test_form_37_trans  = imputer_37.transform(test_form_37_merge_mm)
test_form_38_trans  = imputer_38.transform(test_form_38_merge_mm)
test_form_39_trans  = imputer_39.transform(test_form_39_merge_mm)
test_form_40_trans  = imputer_40.transform(test_form_40_merge_mm.drop(['SP_mm'], axis=1))

test_form_41_trans  = imputer_41.transform(test_form_41_merge_mm)

test_form_sec_trans  = imputer_sec.transform(test_form_sec_merge_mm)

[IterativeImputer] Completing matrix with shape (21989, 8)
[IterativeImputer] Completing matrix with shape (1721, 8)
[IterativeImputer] Completing matrix with shape (1132, 8)
[IterativeImputer] Completing matrix with shape (4716, 8)
[IterativeImputer] Completing matrix with shape (2118, 8)
[IterativeImputer] Completing matrix with shape (1250, 8)
[IterativeImputer] Completing matrix with shape (59, 8)
[IterativeImputer] Completing matrix with shape (1079, 8)
[IterativeImputer] Completing matrix with shape (993, 8)
[IterativeImputer] Completing matrix with shape (547, 8)
[IterativeImputer] Completing matrix with shape (2564, 8)
[IterativeImputer] Completing matrix with shape (942, 8)
[IterativeImputer] Completing matrix with shape (191, 8)
[IterativeImputer] Completing matrix with shape (2458, 8)
[IterativeImputer] Completing matrix with shape (513, 8)
[IterativeImputer] Completing matrix with shape (3586, 8)
[IterativeImputer] Completing matrix with shape (493, 8)
[IterativeImputer] Co

In [ ]:
mm_cols = ['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm', 
           'NPHI_mm', 'SP_mm', 'PEF_mm', 
           #'RMED_mm', 'RDEP_mm'
           ]
#std_cols = ['GR_std', 'DTC_std', 'CALI_std', 'DRHO_std', 'RHOB_std', 'RMED_std', 'RDEP_std']

In [ ]:
df_test_form_1_trans = pd.DataFrame(test_form_1_trans, columns=mm_cols, index=test_form_1.index)
df_test_form_2_trans = pd.DataFrame(test_form_2_trans, columns=mm_cols, index=test_form_2.index)
df_test_form_3_trans = pd.DataFrame(test_form_3_trans, columns=mm_cols, index=test_form_3.index)
df_test_form_4_trans = pd.DataFrame(test_form_4_trans, columns=mm_cols, index=test_form_4.index)
df_test_form_5_trans = pd.DataFrame(test_form_5_trans, columns=mm_cols, index=test_form_5.index)
df_test_form_6_trans = pd.DataFrame(test_form_6_trans, columns=mm_cols, index=test_form_6.index)
df_test_form_7_trans = pd.DataFrame(test_form_7_trans, columns=mm_cols, index=test_form_7.index)
df_test_form_8_trans = pd.DataFrame(test_form_8_trans, columns=mm_cols, index=test_form_8.index)
df_test_form_9_trans = pd.DataFrame(test_form_9_trans, columns=mm_cols, index=test_form_9.index)
df_test_form_10_trans = pd.DataFrame(test_form_10_trans, columns=mm_cols, index=test_form_10.index)

df_test_form_11_trans = pd.DataFrame(test_form_11_trans, columns=mm_cols, index=test_form_11.index)
df_test_form_12_trans = pd.DataFrame(test_form_12_trans, columns=mm_cols, index=test_form_12.index)
df_test_form_13_trans = pd.DataFrame(test_form_13_trans, columns=mm_cols, index=test_form_13.index)
df_test_form_14_trans = pd.DataFrame(test_form_14_trans, columns=mm_cols, index=test_form_14.index)
df_test_form_15_trans = pd.DataFrame(test_form_15_trans, columns=mm_cols, index=test_form_15.index)
df_test_form_16_trans = pd.DataFrame(test_form_16_trans, columns=mm_cols, index=test_form_16.index)
df_test_form_17_trans = pd.DataFrame(test_form_17_trans, columns=mm_cols, index=test_form_17.index)
df_test_form_18_trans = pd.DataFrame(test_form_18_trans, columns=mm_cols, index=test_form_18.index)
df_test_form_19_trans = pd.DataFrame(test_form_19_trans, columns=mm_cols, index=test_form_19.index)
df_test_form_20_trans = pd.DataFrame(test_form_20_trans, columns=mm_cols, index=test_form_20.index)

df_test_form_21_trans = pd.DataFrame(test_form_21_trans, columns=mm_cols, index=test_form_21.index)
df_test_form_22_trans = pd.DataFrame(test_form_22_trans, columns=mm_cols, index=test_form_22.index)
df_test_form_23_trans = pd.DataFrame(test_form_23_trans, columns=mm_cols, index=test_form_23.index)
df_test_form_24_trans = pd.DataFrame(test_form_24_trans, columns=mm_cols, index=test_form_24.index)
df_test_form_25_trans = pd.DataFrame(test_form_25_trans, columns=mm_cols, index=test_form_25.index)
df_test_form_26_trans = pd.DataFrame(test_form_26_trans, columns=mm_cols, index=test_form_26.index)
df_test_form_27_trans = pd.DataFrame(test_form_27_trans, columns=mm_cols, index=test_form_27.index)
df_test_form_28_trans = pd.DataFrame(test_form_28_trans, columns=mm_cols, index=test_form_28.index)
df_test_form_29_trans = pd.DataFrame(test_form_29_trans, columns=mm_cols, index=test_form_29.index)
df_test_form_30_trans = pd.DataFrame(test_form_30_trans, columns=mm_cols, index=test_form_30.index)

df_test_form_31_trans = pd.DataFrame(test_form_31_trans, columns=mm_cols, index=test_form_31.index)
df_test_form_32_trans = pd.DataFrame(test_form_32_trans, columns=mm_cols, index=test_form_32.index)
df_test_form_33_trans = pd.DataFrame(test_form_33_trans, columns=mm_cols, index=test_form_33.index)
df_test_form_34_trans = pd.DataFrame(test_form_34_trans, columns=mm_cols, index=test_form_34.index)
df_test_form_35_trans = pd.DataFrame(test_form_35_trans, columns=mm_cols, index=test_form_35.index)
df_test_form_36_trans = pd.DataFrame(test_form_36_trans, columns=mm_cols, index=test_form_36.index)
df_test_form_37_trans = pd.DataFrame(test_form_37_trans, columns=mm_cols, index=test_form_37.index)
df_test_form_38_trans = pd.DataFrame(test_form_38_trans, columns=mm_cols, index=test_form_38.index)
df_test_form_39_trans = pd.DataFrame(test_form_39_trans, columns=mm_cols, index=test_form_39.index)
df_test_form_40_trans = pd.DataFrame(test_form_40_trans, columns=['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm', 'NPHI_mm', 'PEF_mm'], 
                                     index=test_form_40.index)

df_test_form_41_trans = pd.DataFrame(test_form_41_trans, columns=mm_cols, index=test_form_41.index)

df_test_form_sec_trans = pd.DataFrame(test_form_sec_trans, columns=mm_cols, index=test_form_sec.index)

In [ ]:
df_test_form_40_trans['SP_mm'] = np.nan
df_test_form_40_trans = df_test_form_40_trans[['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 
                                         'RHOB_mm', 'NPHI_mm', 'SP_mm', 'PEF_mm']]
df_test_form_40_trans.head(2)                                         

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
128060,0.033869,0.190964,0.44136,0.023791,0.489335,0.115835,NaN,0.295272
128061,0.003732,0.164248,0.44811,0.023454,0.475190,0.087419,NaN,0.285566


In [ ]:
df_test_form_trans = pd.concat([df_test_form_1_trans, df_test_form_2_trans, df_test_form_3_trans, df_test_form_4_trans, df_test_form_5_trans,
                                df_test_form_6_trans, df_test_form_7_trans, df_test_form_8_trans, df_test_form_9_trans, df_test_form_10_trans,
                                df_test_form_11_trans, df_test_form_12_trans, df_test_form_13_trans, df_test_form_14_trans, df_test_form_15_trans,
                                df_test_form_16_trans, df_test_form_17_trans, df_test_form_18_trans, df_test_form_19_trans, df_test_form_20_trans,
                                df_test_form_21_trans, df_test_form_22_trans, df_test_form_23_trans, df_test_form_24_trans, df_test_form_25_trans,
                                df_test_form_26_trans, df_test_form_27_trans, df_test_form_28_trans, df_test_form_29_trans, df_test_form_30_trans,
                                df_test_form_31_trans, df_test_form_32_trans, df_test_form_33_trans, df_test_form_34_trans, df_test_form_35_trans,
                                df_test_form_36_trans, df_test_form_37_trans, df_test_form_38_trans, df_test_form_39_trans, df_test_form_40_trans,
                                df_test_form_41_trans, df_test_form_sec_trans
                                ])

In [ ]:
df_test_form_trans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136786 entries, 2924 to 73482
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   GR_mm    136786 non-null  float64
 1   DTC_mm   136786 non-null  float64
 2   CALI_mm  136786 non-null  float64
 3   DRHO_mm  136786 non-null  float64
 4   RHOB_mm  136786 non-null  float64
 5   NPHI_mm  136786 non-null  float64
 6   SP_mm    134338 non-null  float64
 7   PEF_mm   136786 non-null  float64
dtypes: float64(8)
memory usage: 9.4 MB


In [ ]:
df_test_form_trans_merge = df_test.merge(df_test_form_trans, left_index=True, right_index=True, how='outer')
df_test_form_trans_merge.head(2)

,FORMATION,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
0,no_form,72.078377,174.347092,19.2031,-0.538873,2.171173,0.551575,35.525719,55.815659,0.107031,0.765583,0.718561,0.474727,0.493260,0.553314,0.450560,0.152447
1,no_form,72.147697,176.112915,19.2031,-0.539232,2.171840,0.547566,36.158520,41.862537,0.107140,0.775229,0.718561,0.474668,0.493545,0.549316,0.451273,0.114269


In [ ]:
df_test_form_trans_merge.describe()

In [ ]:
df_40_sp_last = df_test_form_trans_merge[['FORMATION', 'GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 
                                          'RHOB_mm', 'NPHI_mm', 'SP_mm', 'PEF_mm']]

In [ ]:
imputer_40_sp_last = IterativeImputer(estimator=BayesianRidge(), imputation_order='ascending', max_iter=400, verbose=1, tol=0.0001, min_value=0, max_value=1)

In [ ]:
imputer_40_sp_last.fit(df_40_sp_last.drop(['FORMATION'], axis=1))

[IterativeImputer] Completing matrix with shape (136786, 8)
[IterativeImputer] Change: 0.19482262301296854, scaled tolerance: 0.0005691294612756477 
[IterativeImputer] Change: 0.0, scaled tolerance: 0.0005691294612756477 
[IterativeImputer] Early stopping criterion reached.


IterativeImputer(estimator=BayesianRidge(), max_iter=400, max_value=1,
                 min_value=0, tol=0.0001, verbose=1)

In [ ]:
form_comp_40_trans  = imputer_40_sp_last.transform(df_40_sp_last.drop(['FORMATION'], axis=1))

[IterativeImputer] Completing matrix with shape (136786, 8)


In [ ]:
df_form_comp_40_trans = pd.DataFrame(form_comp_40_trans, columns=['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm', 'NPHI_mm', 'SP_mm', 'PEF_mm'], 
                                     index=df_40_sp_last.index)

In [ ]:
df_form_comp_40_trans.describe()

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
count,136786.000000,136786.000000,136786.000000,136786.000000,136786.000000,136786.000000,136786.000000,136786.000000
mean,0.239519,0.457264,0.355496,0.439599,0.522927,0.418918,0.425244,0.053140
std,0.174837,0.159582,0.165154,0.284981,0.142634,0.144968,0.255557,0.104960
min,-0.116442,-0.180652,0.025199,-1.689715,-0.640906,-0.343404,-3.441696,-0.091178
25%,0.115742,0.338916,0.255951,0.185809,0.434162,0.339877,0.217342,0.008517
50%,0.187697,0.451819,0.323845,0.337437,0.493808,0.419918,0.446795,0.022315
75%,0.310413,0.590956,0.471672,0.622959,0.603270,0.483833,0.577917,0.044756
max,1.786133,1.621629,1.214885,1.917928,1.202513,1.964674,3.063829,5.691295


In [ ]:
df_form_comp_40_trans

In [ ]:
test_mm_all_imp  = imputer_all.transform(test_mm_all)

[IterativeImputer] Completing matrix with shape (136786, 8)


In [ ]:
df_test_mm_all_imp = pd.DataFrame(test_mm_all_imp, columns=test_mm_all.columns, index=test_mm_all.index)

In [ ]:
df_test_mm_all_imp.head(2)

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
0,0.034024,2.033612,1.583989,-0.073694,-0.449355,1.683058,-0.320059,4.517969
1,0.036049,2.092492,1.583989,-0.073742,-0.446720,1.652334,-0.311795,3.244334


In [ ]:
df_test_mm_all_imp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136786 entries, 0 to 136785
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   GR_mm    136786 non-null  float64
 1   DTC_mm   136786 non-null  float64
 2   CALI_mm  136786 non-null  float64
 3   DRHO_mm  136786 non-null  float64
 4   RHOB_mm  136786 non-null  float64
 5   NPHI_mm  136786 non-null  float64
 6   SP_mm    136786 non-null  float64
 7   PEF_mm   136786 non-null  float64
dtypes: float64(8)
memory usage: 8.3 MB


In [ ]:
test_prediction = model_merge_mm.predict(df_form_comp_40_trans)
test_prediction

array([2, 2, 2, ..., 0, 0, 0])

In [ ]:
pd.Series(test_prediction).value_counts()

2     104748
0      14599
10      5081
5       4584
1       3145
3       2411
9       1244
6        584
4        366
8         19
7          5
dtype: int64

In [ ]:
pd.Series(test_prediction).value_counts()

2     99876
0     20845
1      5983
5      5191
9      1604
3      1004
10      999
6       857
4       322
8       105
dtype: int64

In [ ]:
category_to_lithology = {y:x for x,y in lithology_numbers.items()}
test_prediction_for_submission = np.vectorize(category_to_lithology.get)(test_prediction)
test_prediction_for_submission

array([65000, 65000, 65000, ..., 30000, 30000, 30000])

In [ ]:
np.savetxt('../Submission/test_predictions.csv', test_prediction_for_submission, header='lithology', fmt='%i')

In [ ]:
test_predictions = pd.read_csv('../Submission/test_predictions.csv')
test_predictions.columns = ['lithology']
test_predictions.head(2)

,lithology
0,65000
1,65000


In [ ]:
test_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136786 entries, 0 to 136785
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   lithology  136786 non-null  int64
dtypes: int64(1)
memory usage: 1.0 MB


In [ ]:
# Imputer from test dataset

In [ ]:
df_test = df_test[['FORMATION', 'GR', 'DTC', 'CALI', 'DRHO', 'RHOB', 
                   'NPHI', 'SP', 'PEF',
                   #'RMED', 'RDEP'
                   ]]

In [ ]:
df_test['FORMATION'].fillna('no_form', inplace=True)

In [ ]:
test_form = df_test['FORMATION'].unique()

In [ ]:
test_form_1  = df_test[df_test['FORMATION'] == test_form[1]]
test_form_2  = df_test[df_test['FORMATION'] == test_form[2]]
test_form_3  = df_test[df_test['FORMATION'] == test_form[3]]
test_form_4  = df_test[df_test['FORMATION'] == test_form[4]]
test_form_5  = df_test[df_test['FORMATION'] == test_form[5]]
test_form_6  = df_test[df_test['FORMATION'] == test_form[6]]
test_form_7  = df_test[df_test['FORMATION'] == test_form[7]]
test_form_8  = df_test[df_test['FORMATION'] == test_form[8]]
test_form_9  = df_test[df_test['FORMATION'] == test_form[9]]
test_form_10  = df_test[df_test['FORMATION'] == test_form[10]]

test_form_11  = df_test[df_test['FORMATION'] == test_form[11]]
test_form_12  = df_test[df_test['FORMATION'] == test_form[12]]
test_form_13  = df_test[df_test['FORMATION'] == test_form[13]]
test_form_14  = df_test[df_test['FORMATION'] == test_form[14]]
test_form_15  = df_test[df_test['FORMATION'] == test_form[15]]
test_form_16  = df_test[df_test['FORMATION'] == test_form[16]]
test_form_17  = df_test[df_test['FORMATION'] == test_form[17]]
test_form_18  = df_test[df_test['FORMATION'] == test_form[18]]
test_form_19  = df_test[df_test['FORMATION'] == test_form[19]]
test_form_20  = df_test[df_test['FORMATION'] == test_form[20]]

test_form_21  = df_test[df_test['FORMATION'] == test_form[21]]
test_form_22  = df_test[df_test['FORMATION'] == test_form[22]]
test_form_23  = df_test[df_test['FORMATION'] == test_form[23]]
test_form_24  = df_test[df_test['FORMATION'] == test_form[24]]
test_form_25  = df_test[df_test['FORMATION'] == test_form[25]]
test_form_26  = df_test[df_test['FORMATION'] == test_form[26]]
test_form_27  = df_test[df_test['FORMATION'] == test_form[27]]
test_form_28  = df_test[df_test['FORMATION'] == test_form[28]]
test_form_29  = df_test[df_test['FORMATION'] == test_form[29]]
test_form_30  = df_test[df_test['FORMATION'] == test_form[30]]

test_form_31  = df_test[df_test['FORMATION'] == test_form[31]]
test_form_32  = df_test[df_test['FORMATION'] == test_form[32]]
test_form_33  = df_test[df_test['FORMATION'] == test_form[33]]
test_form_34  = df_test[df_test['FORMATION'] == test_form[34]]
test_form_35  = df_test[df_test['FORMATION'] == test_form[35]]
test_form_36  = df_test[df_test['FORMATION'] == test_form[36]]
test_form_37  = df_test[df_test['FORMATION'] == test_form[37]]
test_form_38  = df_test[df_test['FORMATION'] == test_form[38]]
test_form_39  = df_test[df_test['FORMATION'] == test_form[39]]
test_form_40  = df_test[df_test['FORMATION'] == test_form[40]]

test_form_41  = df_test[df_test['FORMATION'] == test_form[41]]

test_form_sec  = df_test[df_test['FORMATION'] == test_form[0]]

In [ ]:
test_form_merge = pd.concat([test_form_1,  test_form_2,  test_form_3,  test_form_4,  test_form_5,  
                             test_form_6,  test_form_7,  test_form_8,  test_form_9,  test_form_10,
                             test_form_11, test_form_12, test_form_13, test_form_14, test_form_15, 
                             test_form_16, test_form_17, test_form_18, test_form_19, test_form_20,
                             test_form_21, test_form_22, test_form_23, test_form_24, test_form_25, 
                             test_form_26, test_form_27, test_form_28, test_form_29, test_form_30,
                             test_form_31, test_form_32, test_form_33, test_form_34, test_form_35, 
                             test_form_36, test_form_37, test_form_38, test_form_39, test_form_40, 
                             test_form_41,
                             test_form_sec])

In [ ]:
test_form_1_trans  = imputer_1.transform(test_form_1.drop(['FORMATION'], axis=1))
test_form_2_trans  = imputer_2.transform(test_form_2.drop(['FORMATION'], axis=1))
test_form_3_trans  = imputer_3.transform(test_form_3.drop(['FORMATION'], axis=1))
test_form_4_trans  = imputer_4.transform(test_form_4.drop(['FORMATION'], axis=1))
test_form_5_trans  = imputer_5.transform(test_form_5.drop(['FORMATION'], axis=1))
test_form_6_trans  = imputer_6.transform(test_form_6.drop(['FORMATION'], axis=1))
test_form_7_trans  = imputer_7.transform(test_form_7.drop(['FORMATION'], axis=1))
test_form_8_trans  = imputer_8.transform(test_form_8.drop(['FORMATION'], axis=1))
test_form_9_trans  = imputer_9.transform(test_form_9.drop(['FORMATION'], axis=1))
test_form_10_trans = imputer_10.transform(test_form_10.drop(['FORMATION'], axis=1))

test_form_11_trans  = imputer_11.transform(test_form_11.drop(['FORMATION'], axis=1))
test_form_12_trans  = imputer_12.transform(test_form_12.drop(['FORMATION'], axis=1))
test_form_13_trans  = imputer_13.transform(test_form_13.drop(['FORMATION'], axis=1))
test_form_14_trans  = imputer_14.transform(test_form_14.drop(['FORMATION'], axis=1))
test_form_15_trans  = imputer_15.transform(test_form_15.drop(['FORMATION'], axis=1))
test_form_16_trans  = imputer_16.transform(test_form_16.drop(['FORMATION'], axis=1))
test_form_17_trans  = imputer_17.transform(test_form_17.drop(['FORMATION'], axis=1))
test_form_18_trans  = imputer_18.transform(test_form_18.drop(['FORMATION'], axis=1))
test_form_19_trans  = imputer_19.transform(test_form_19.drop(['FORMATION'], axis=1))
test_form_20_trans  = imputer_20.transform(test_form_20.drop(['FORMATION'], axis=1))

test_form_21_trans  = imputer_21.transform(test_form_21.drop(['FORMATION'], axis=1))
test_form_22_trans  = imputer_22.transform(test_form_22.drop(['FORMATION'], axis=1))
test_form_23_trans  = imputer_23.transform(test_form_23.drop(['FORMATION'], axis=1))
test_form_24_trans  = imputer_24.transform(test_form_24.drop(['FORMATION'], axis=1))
test_form_25_trans  = imputer_25.transform(test_form_25.drop(['FORMATION'], axis=1))
test_form_26_trans  = imputer_26.transform(test_form_26.drop(['FORMATION'], axis=1))
test_form_27_trans  = imputer_27.transform(test_form_27.drop(['FORMATION'], axis=1))
test_form_28_trans  = imputer_28.transform(test_form_28.drop(['FORMATION'], axis=1))
test_form_29_trans  = imputer_29.transform(test_form_29.drop(['FORMATION'], axis=1))
test_form_30_trans = imputer_30.transform(test_form_30.drop(['FORMATION'], axis=1))

test_form_31_trans  = imputer_31.transform(test_form_31.drop(['FORMATION'], axis=1))
test_form_32_trans  = imputer_32.transform(test_form_32.drop(['FORMATION'], axis=1))
test_form_33_trans  = imputer_33.transform(test_form_33.drop(['FORMATION'], axis=1))
test_form_34_trans  = imputer_34.transform(test_form_34.drop(['FORMATION'], axis=1))
test_form_35_trans  = imputer_35.transform(test_form_35.drop(['FORMATION'], axis=1))
test_form_36_trans  = imputer_36.transform(test_form_36.drop(['FORMATION'], axis=1))
test_form_37_trans  = imputer_37.transform(test_form_37.drop(['FORMATION'], axis=1))
test_form_38_trans  = imputer_38.transform(test_form_38.drop(['FORMATION'], axis=1))
test_form_39_trans  = imputer_39.transform(test_form_39.drop(['FORMATION'], axis=1))
test_form_40_trans  = imputer_40.transform(test_form_40.drop(['FORMATION'], axis=1))

test_form_41_trans  = imputer_41.transform(test_form_41.drop(['FORMATION'], axis=1))

test_form_sec_trans  = imputer_sec.transform(test_form_sec.drop(['FORMATION'], axis=1))

In [ ]:
df_test_form_1_trans = pd.DataFrame(test_form_1_trans, columns=test_form_1.columns[1:], index=test_form_1.index)
df_test_form_2_trans = pd.DataFrame(test_form_2_trans, columns=test_form_2.columns[1:], index=test_form_2.index)
df_test_form_3_trans = pd.DataFrame(test_form_3_trans, columns=test_form_3.columns[1:], index=test_form_3.index)
df_test_form_4_trans = pd.DataFrame(test_form_4_trans, columns=test_form_4.columns[1:], index=test_form_4.index)
df_test_form_5_trans = pd.DataFrame(test_form_5_trans, columns=test_form_5.columns[1:], index=test_form_5.index)
df_test_form_6_trans = pd.DataFrame(test_form_6_trans, columns=test_form_6.columns[1:], index=test_form_6.index)
df_test_form_7_trans = pd.DataFrame(test_form_7_trans, columns=test_form_7.columns[1:], index=test_form_7.index)
df_test_form_8_trans = pd.DataFrame(test_form_8_trans, columns=test_form_8.columns[1:], index=test_form_8.index)
df_test_form_9_trans = pd.DataFrame(test_form_9_trans, columns=test_form_9.columns[1:], index=test_form_9.index)
df_test_form_10_trans = pd.DataFrame(test_form_10_trans, columns=test_form_10.columns[1:], index=test_form_10.index)

df_test_form_11_trans = pd.DataFrame(test_form_11_trans, columns=test_form_11.columns[1:], index=test_form_11.index)
df_test_form_12_trans = pd.DataFrame(test_form_12_trans, columns=test_form_12.columns[1:], index=test_form_12.index)
df_test_form_13_trans = pd.DataFrame(test_form_13_trans, columns=test_form_13.columns[1:], index=test_form_13.index)
df_test_form_14_trans = pd.DataFrame(test_form_14_trans, columns=test_form_14.columns[1:], index=test_form_14.index)
df_test_form_15_trans = pd.DataFrame(test_form_15_trans, columns=test_form_15.columns[1:], index=test_form_15.index)
df_test_form_16_trans = pd.DataFrame(test_form_16_trans, columns=test_form_16.columns[1:], index=test_form_16.index)
df_test_form_17_trans = pd.DataFrame(test_form_17_trans, columns=test_form_17.columns[1:], index=test_form_17.index)
df_test_form_18_trans = pd.DataFrame(test_form_18_trans, columns=test_form_18.columns[1:], index=test_form_18.index)
df_test_form_19_trans = pd.DataFrame(test_form_19_trans, columns=test_form_19.columns[1:], index=test_form_19.index)
df_test_form_20_trans = pd.DataFrame(test_form_20_trans, columns=test_form_20.columns[1:], index=test_form_20.index)

df_test_form_21_trans = pd.DataFrame(test_form_21_trans, columns=test_form_21.columns[1:], index=test_form_21.index)
df_test_form_22_trans = pd.DataFrame(test_form_22_trans, columns=test_form_22.columns[1:], index=test_form_22.index)
df_test_form_23_trans = pd.DataFrame(test_form_23_trans, columns=test_form_23.columns[1:], index=test_form_23.index)
df_test_form_24_trans = pd.DataFrame(test_form_24_trans, columns=test_form_24.columns[1:], index=test_form_24.index)
df_test_form_25_trans = pd.DataFrame(test_form_25_trans, columns=test_form_25.columns[1:], index=test_form_25.index)
df_test_form_26_trans = pd.DataFrame(test_form_26_trans, columns=test_form_26.columns[1:], index=test_form_26.index)
df_test_form_27_trans = pd.DataFrame(test_form_27_trans, columns=test_form_27.columns[1:], index=test_form_27.index)
df_test_form_28_trans = pd.DataFrame(test_form_28_trans, columns=test_form_28.columns[1:], index=test_form_28.index)
df_test_form_29_trans = pd.DataFrame(test_form_29_trans, columns=test_form_29.columns[1:], index=test_form_29.index)
df_test_form_30_trans = pd.DataFrame(test_form_30_trans, columns=test_form_30.columns[1:], index=test_form_30.index)

df_test_form_31_trans = pd.DataFrame(test_form_31_trans, columns=test_form_31.columns[1:], index=test_form_31.index)
df_test_form_32_trans = pd.DataFrame(test_form_32_trans, columns=test_form_32.columns[1:], index=test_form_32.index)
df_test_form_33_trans = pd.DataFrame(test_form_33_trans, columns=test_form_33.columns[1:], index=test_form_33.index)
df_test_form_34_trans = pd.DataFrame(test_form_34_trans, columns=test_form_34.columns[1:], index=test_form_34.index)
df_test_form_35_trans = pd.DataFrame(test_form_35_trans, columns=test_form_35.columns[1:], index=test_form_35.index)
df_test_form_36_trans = pd.DataFrame(test_form_36_trans, columns=test_form_36.columns[1:], index=test_form_36.index)
df_test_form_37_trans = pd.DataFrame(test_form_37_trans, columns=test_form_37.columns[1:], index=test_form_37.index)
df_test_form_38_trans = pd.DataFrame(test_form_38_trans, columns=test_form_38.columns[1:], index=test_form_38.index)
df_test_form_39_trans = pd.DataFrame(test_form_39_trans, columns=test_form_39.columns[1:], index=test_form_39.index)
df_test_form_40_trans = pd.DataFrame(test_form_40_trans, columns=test_form_40.columns[1:], index=test_form_40.index)

df_test_form_41_trans = pd.DataFrame(test_form_41_trans, columns=test_form_41.columns[1:], index=test_form_41.index)

df_test_form_sec_trans = pd.DataFrame(test_form_sec_trans, columns=test_form_sec.columns[1:], index=test_form_sec.index)

In [ ]:
df_test_form_trans = pd.concat([df_test_form_1_trans,  df_test_form_2_trans,  df_test_form_3_trans,  df_test_form_4_trans,  df_test_form_5_trans,
                                df_test_form_6_trans,  df_test_form_7_trans,  df_test_form_8_trans,  df_test_form_9_trans,  df_test_form_10_trans,
                                df_test_form_11_trans, df_test_form_12_trans, df_test_form_13_trans, df_test_form_14_trans, df_test_form_15_trans,
                                df_test_form_16_trans, df_test_form_17_trans, df_test_form_18_trans, df_test_form_19_trans, df_test_form_20_trans,
                                df_test_form_21_trans, df_test_form_22_trans, df_test_form_23_trans, df_test_form_24_trans, df_test_form_25_trans,
                                df_test_form_26_trans, df_test_form_27_trans, df_test_form_28_trans, df_test_form_29_trans, df_test_form_30_trans,
                                df_test_form_31_trans, df_test_form_32_trans, df_test_form_33_trans, df_test_form_34_trans, df_test_form_35_trans,
                                df_test_form_36_trans, df_test_form_37_trans, df_test_form_38_trans, df_test_form_39_trans, df_test_form_40_trans,
                                df_test_form_41_trans,
                                df_test_form_sec_trans])

df_test_form_trans.columns = ['GR_new', 'DTC_new', 'CALI_new', 'DRHO_new', 'RHOB_new', 
                              'NPHI_new', 'SP_new', 'PEF_new',
                              #'RMED_new', 'RDEP_new'
                              ]
df_test_form_trans.head(2)

,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new
2924,68.656174,142.063416,17.359400,0.012434,2.106125,0.488468,34.949097,2.085944
2925,64.126785,144.903183,17.348019,0.009665,2.110649,0.482300,35.064674,2.041033


In [ ]:
df_test_form_trans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136786 entries, 2924 to 73482
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   GR_new    136786 non-null  float64
 1   DTC_new   136786 non-null  float64
 2   CALI_new  136786 non-null  float64
 3   DRHO_new  136786 non-null  float64
 4   RHOB_new  136786 non-null  float64
 5   NPHI_new  136786 non-null  float64
 6   SP_new    136786 non-null  float64
 7   PEF_new   136786 non-null  float64
dtypes: float64(8)
memory usage: 9.4 MB


In [ ]:
df_test_form_trans_merge = df_test.merge(df_test_form_trans, left_index=True, right_index=True, how='outer')
df_test_form_trans_merge.head()

,FORMATION,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new
0,no_form,72.078377,174.347092,19.2031,-0.538873,2.171173,0.551575,35.525719,55.815659,72.078377,174.347092,19.2031,-0.538873,2.171173,0.551575,35.525719,55.815659
1,no_form,72.147697,176.112915,19.2031,-0.539232,2.171840,0.547566,36.158520,41.862537,72.147697,176.112915,19.2031,-0.539232,2.171840,0.547566,36.158520,41.862537
2,no_form,75.074219,177.662476,19.2031,-0.540830,2.161303,0.551840,36.873703,38.587872,75.074219,177.662476,19.2031,-0.540830,2.161303,0.551840,36.873703,38.587872
3,no_form,80.735229,177.437714,19.2031,-0.543943,2.151947,0.552511,37.304054,41.417999,80.735229,177.437714,19.2031,-0.543943,2.151947,0.552511,37.304054,41.417999
4,no_form,81.665024,176.797653,19.2031,-0.542104,2.143010,0.566789,37.864922,49.471237,81.665024,176.797653,19.2031,-0.542104,2.143010,0.566789,37.864922,49.471237


In [ ]:
df_test_form_trans_merge['GR_new'] = df_test_form_trans_merge.apply(
    lambda row: row['GR'] if np.isnan(row['GR_new']) else row['GR_new'],
    axis=1
)
df_test_form_trans_merge['DTC_new'] = df_test_form_trans_merge.apply(
    lambda row: row['DTC'] if np.isnan(row['DTC_new']) else row['DTC_new'],
    axis=1
)
df_test_form_trans_merge['CALI_new'] = df_test_form_trans_merge.apply(
    lambda row: row['CALI'] if np.isnan(row['CALI_new']) else row['CALI_new'],
    axis=1
)
df_test_form_trans_merge['DRHO_new'] = df_test_form_trans_merge.apply(
    lambda row: row['DRHO'] if np.isnan(row['DRHO_new']) else row['DRHO_new'],
    axis=1
)
df_test_form_trans_merge['RHOB_new'] = df_test_form_trans_merge.apply(
    lambda row: row['RHOB'] if np.isnan(row['RHOB_new']) else row['RHOB_new'],
    axis=1
)
df_test_form_trans_merge['NPHI_new'] = df_test_form_trans_merge.apply(
    lambda row: row['NPHI'] if np.isnan(row['NPHI_new']) else row['NPHI_new'],
    axis=1
)
df_test_form_trans_merge['SP_new'] = df_test_form_trans_merge.apply(
    lambda row: row['SP'] if np.isnan(row['SP_new']) else row['SP_new'],
    axis=1
)
df_test_form_trans_merge['PEF_new'] = df_test_form_trans_merge.apply(
    lambda row: row['PEF'] if np.isnan(row['PEF_new']) else row['PEF_new'],
    axis=1
)
df_test_form_trans_merge['RMED_new'] = df_test_form_trans_merge.apply(
    lambda row: row['RMED'] if np.isnan(row['RMED_new']) else row['RMED_new'],
    axis=1
)
df_test_form_trans_merge['RDEP_new'] = df_test_form_trans_merge.apply(
    lambda row: row['RDEP'] if np.isnan(row['RDEP_new']) else row['RDEP_new'],
    axis=1
)

In [ ]:
df_test_form_trans_merge.head()

,FORMATION,GR,DTC,CALI,DRHO,RHOB,NPHI,SP,PEF,RMED,RDEP,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,RMED_new,RDEP_new
0,no_form,72.078377,174.347092,19.2031,-0.538873,2.171173,0.551575,35.525719,55.815659,1.613886,1.513937,72.078377,174.347092,19.2031,-0.538873,2.171173,0.551575,35.525719,55.815659,1.613886,1.513937
1,no_form,72.147697,176.112915,19.2031,-0.539232,2.171840,0.547566,36.158520,41.862537,1.574376,1.480188,72.147697,176.112915,19.2031,-0.539232,2.171840,0.547566,36.158520,41.862537,1.574376,1.480188
2,no_form,75.074219,177.662476,19.2031,-0.540830,2.161303,0.551840,36.873703,38.587872,1.436627,1.469700,75.074219,177.662476,19.2031,-0.540830,2.161303,0.551840,36.873703,38.587872,1.436627,1.469700
3,no_form,80.735229,177.437714,19.2031,-0.543943,2.151947,0.552511,37.304054,41.417999,1.276094,1.469341,80.735229,177.437714,19.2031,-0.543943,2.151947,0.552511,37.304054,41.417999,1.276094,1.469341
4,no_form,81.665024,176.797653,19.2031,-0.542104,2.143010,0.566789,37.864922,49.471237,1.204704,1.466462,81.665024,176.797653,19.2031,-0.542104,2.143010,0.566789,37.864922,49.471237,1.204704,1.466462


In [ ]:
df_test_form_trans_merge = df_test_form_trans_merge[['GR_new', 'DTC_new', 'CALI_new', 'DRHO_new', 'RHOB_new',
                                                     'NPHI_new', 'SP_new', 'PEF_new',
                                                     #'RMED_new', 'RDEP_new'
                                                     ]]

In [ ]:
mm_cols = ['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm', 
           'NPHI_mm', 'SP_mm', 'PEF_mm',
           #'RMED_mm', 'RDEP_mm'
           ]
#std_cols = ['GR_std', 'DTC_std', 'CALI_std', 'DRHO_std', 'RHOB_std', 'RMED_std', 'RDEP_std']

In [ ]:
test_form_1_merge_mm  = pd.DataFrame(sc_1.transform(df_test_form_1_trans), df_test_form_1_trans.index, mm_cols)
test_form_2_merge_mm  = pd.DataFrame(sc_2.transform(df_test_form_2_trans), df_test_form_2_trans.index, mm_cols)
test_form_3_merge_mm  = pd.DataFrame(sc_3.transform(df_test_form_3_trans), df_test_form_3_trans.index, mm_cols)
test_form_4_merge_mm  = pd.DataFrame(sc_4.transform(df_test_form_4_trans), df_test_form_4_trans.index, mm_cols)
test_form_5_merge_mm  = pd.DataFrame(sc_5.transform(df_test_form_5_trans), df_test_form_5_trans.index, mm_cols)
test_form_6_merge_mm  = pd.DataFrame(sc_6.transform(df_test_form_6_trans), df_test_form_6_trans.index, mm_cols)
test_form_7_merge_mm  = pd.DataFrame(sc_7.transform(df_test_form_7_trans), df_test_form_7_trans.index, mm_cols)
test_form_8_merge_mm  = pd.DataFrame(sc_8.transform(df_test_form_8_trans), df_test_form_8_trans.index, mm_cols)
test_form_9_merge_mm  = pd.DataFrame(sc_9.transform(df_test_form_9_trans), df_test_form_9_trans.index, mm_cols)
test_form_10_merge_mm  = pd.DataFrame(sc_10.transform(df_test_form_10_trans), df_test_form_10_trans.index, mm_cols)

test_form_11_merge_mm  = pd.DataFrame(sc_11.transform(df_test_form_11_trans), df_test_form_11_trans.index, mm_cols)
test_form_12_merge_mm  = pd.DataFrame(sc_12.transform(df_test_form_12_trans), df_test_form_12_trans.index, mm_cols)
test_form_13_merge_mm  = pd.DataFrame(sc_13.transform(df_test_form_13_trans), df_test_form_13_trans.index, mm_cols)
test_form_14_merge_mm  = pd.DataFrame(sc_14.transform(df_test_form_14_trans), df_test_form_14_trans.index, mm_cols)
test_form_15_merge_mm  = pd.DataFrame(sc_15.transform(df_test_form_15_trans), df_test_form_15_trans.index, mm_cols)
test_form_16_merge_mm  = pd.DataFrame(sc_16.transform(df_test_form_16_trans), df_test_form_16_trans.index, mm_cols)
test_form_17_merge_mm  = pd.DataFrame(sc_17.transform(df_test_form_17_trans), df_test_form_17_trans.index, mm_cols)
test_form_18_merge_mm  = pd.DataFrame(sc_18.transform(df_test_form_18_trans), df_test_form_18_trans.index, mm_cols)
test_form_19_merge_mm  = pd.DataFrame(sc_19.transform(df_test_form_19_trans), df_test_form_19_trans.index, mm_cols)
test_form_20_merge_mm  = pd.DataFrame(sc_20.transform(df_test_form_20_trans), df_test_form_20_trans.index, mm_cols)

test_form_21_merge_mm  = pd.DataFrame(sc_21.transform(df_test_form_21_trans), df_test_form_21_trans.index, mm_cols)
test_form_22_merge_mm  = pd.DataFrame(sc_22.transform(df_test_form_22_trans), df_test_form_22_trans.index, mm_cols)
test_form_23_merge_mm  = pd.DataFrame(sc_23.transform(df_test_form_23_trans), df_test_form_23_trans.index, mm_cols)
test_form_24_merge_mm  = pd.DataFrame(sc_24.transform(df_test_form_24_trans), df_test_form_24_trans.index, mm_cols)
test_form_25_merge_mm  = pd.DataFrame(sc_25.transform(df_test_form_25_trans), df_test_form_25_trans.index, mm_cols)
test_form_26_merge_mm  = pd.DataFrame(sc_26.transform(df_test_form_26_trans), df_test_form_26_trans.index, mm_cols)
test_form_27_merge_mm  = pd.DataFrame(sc_27.transform(df_test_form_27_trans), df_test_form_27_trans.index, mm_cols)
test_form_28_merge_mm  = pd.DataFrame(sc_28.transform(df_test_form_28_trans), df_test_form_28_trans.index, mm_cols)
test_form_29_merge_mm  = pd.DataFrame(sc_29.transform(df_test_form_29_trans), df_test_form_29_trans.index, mm_cols)
test_form_30_merge_mm  = pd.DataFrame(sc_30.transform(df_test_form_30_trans), df_test_form_30_trans.index, mm_cols)

test_form_31_merge_mm  = pd.DataFrame(sc_31.transform(df_test_form_31_trans), df_test_form_31_trans.index, mm_cols)
test_form_32_merge_mm  = pd.DataFrame(sc_32.transform(df_test_form_32_trans), df_test_form_32_trans.index, mm_cols)
test_form_33_merge_mm  = pd.DataFrame(sc_33.transform(df_test_form_33_trans), df_test_form_33_trans.index, mm_cols)
test_form_34_merge_mm  = pd.DataFrame(sc_34.transform(df_test_form_34_trans), df_test_form_34_trans.index, mm_cols)
test_form_35_merge_mm  = pd.DataFrame(sc_35.transform(df_test_form_35_trans), df_test_form_35_trans.index, mm_cols)
test_form_36_merge_mm  = pd.DataFrame(sc_36.transform(df_test_form_36_trans), df_test_form_36_trans.index, mm_cols)
test_form_37_merge_mm  = pd.DataFrame(sc_37.transform(df_test_form_37_trans), df_test_form_37_trans.index, mm_cols)
test_form_38_merge_mm  = pd.DataFrame(sc_38.transform(df_test_form_38_trans), df_test_form_38_trans.index, mm_cols)
test_form_39_merge_mm  = pd.DataFrame(sc_39.transform(df_test_form_39_trans), df_test_form_39_trans.index, mm_cols)
test_form_40_merge_mm  = pd.DataFrame(sc_40.transform(df_test_form_40_trans), df_test_form_40_trans.index, mm_cols)

test_form_41_merge_mm  = pd.DataFrame(sc_41.transform(df_test_form_41_trans), df_test_form_41_trans.index, mm_cols)

test_form_sec_merge_mm  = pd.DataFrame(sc_sec.transform(df_test_form_sec_trans), df_test_form_sec_trans.index, mm_cols)

In [ ]:
test_form_mm = pd.concat([test_form_1_merge_mm,  test_form_2_merge_mm,  test_form_3_merge_mm,  test_form_4_merge_mm,  test_form_5_merge_mm,
                          test_form_6_merge_mm,  test_form_7_merge_mm,  test_form_8_merge_mm,  test_form_9_merge_mm,  test_form_10_merge_mm,
                          test_form_11_merge_mm, test_form_12_merge_mm, test_form_13_merge_mm, test_form_14_merge_mm, test_form_15_merge_mm,
                          test_form_16_merge_mm, test_form_17_merge_mm, test_form_18_merge_mm, test_form_19_merge_mm, test_form_20_merge_mm,
                          test_form_21_merge_mm, test_form_22_merge_mm, test_form_23_merge_mm, test_form_24_merge_mm, test_form_25_merge_mm,
                          test_form_26_merge_mm, test_form_27_merge_mm, test_form_28_merge_mm, test_form_29_merge_mm, test_form_30_merge_mm,
                          test_form_31_merge_mm, test_form_32_merge_mm, test_form_33_merge_mm, test_form_34_merge_mm, test_form_35_merge_mm,
                          test_form_36_merge_mm, test_form_37_merge_mm, test_form_38_merge_mm, test_form_39_merge_mm, test_form_40_merge_mm,
                          test_form_41_merge_mm,
                          test_form_sec_merge_mm])

test_form_mm.columns = ['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm',
                        'NPHI_mm', 'SP_mm', 'PEF_mm',
                        #'RMED_mm', 'RDEP_mm'
                        ]
test_form_mm.head(2)

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
2924,0.101698,0.412072,0.760191,0.678592,0.465482,0.186450,0.269145,0.245215
2925,0.094638,0.422920,0.759868,0.678258,0.467414,0.184111,0.269213,0.245122


In [ ]:
test_form_trans_merge_mm = df_test_form_trans_merge.merge(test_form_mm, left_index=True, right_index=True, how='outer')
test_form_trans_merge_mm.head(2)

,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
0,72.078377,174.347092,19.2031,-0.538873,2.171173,0.551575,35.525719,55.815659,0.107031,0.535400,0.812534,0.612224,0.493260,0.210373,0.269483,0.356785
1,72.147697,176.112915,19.2031,-0.539232,2.171840,0.547566,36.158520,41.862537,0.107140,0.542145,0.812534,0.612180,0.493545,0.208853,0.269854,0.327811


In [ ]:
test_form_trans_merge_mm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136786 entries, 0 to 136785
Data columns (total 16 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   GR_new    136786 non-null  float64
 1   DTC_new   136786 non-null  float64
 2   CALI_new  136786 non-null  float64
 3   DRHO_new  136786 non-null  float64
 4   RHOB_new  136786 non-null  float64
 5   NPHI_new  136786 non-null  float64
 6   SP_new    136786 non-null  float64
 7   PEF_new   136786 non-null  float64
 8   GR_mm     136786 non-null  float64
 9   DTC_mm    136786 non-null  float64
 10  CALI_mm   136786 non-null  float64
 11  DRHO_mm   136786 non-null  float64
 12  RHOB_mm   136786 non-null  float64
 13  NPHI_mm   136786 non-null  float64
 14  SP_mm     136786 non-null  float64
 15  PEF_mm    136786 non-null  float64
dtypes: float64(16)
memory usage: 17.7 MB


In [ ]:
test_form_trans_merge_mm = test_form_trans_merge_mm[['GR_new', 'DTC_new', 'CALI_new', 'DRHO_new', 'RHOB_new', 
                                                     'NPHI_new', 'SP_new', 'PEF_new',
                                                     #'RMED_new', 'RDEP_new', 
                                                     #'GR_mm',  'DTC_mm',  'CALI_mm',  'DRHO_mm',  'RHOB_mm', 
                                                     #'NPHI_mm', 'SP_mm', 'PEF_mm',
                                                     #'RMED_mm',  'RDEP_mm'
                                                     ]]

In [ ]:
test_form_trans_merge_mm.head(2)

,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,NPHI_new,SP_new,PEF_new
0,72.078377,174.347092,19.2031,-0.538873,2.171173,0.551575,35.525719,55.815659
1,72.147697,176.112915,19.2031,-0.539232,2.171840,0.547566,36.158520,41.862537


In [ ]:
test_form_trans_mm_all_data = pd.DataFrame(scaler_mm.transform(test_form_trans_merge_mm), index=test_form_trans_merge_mm.index,
                                           columns=['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm', 
                                                    'NPHI_mm', 'SP_mm', 'PEF_mm',
                                                    #'RMED_mm', 'RDEP_mm'
                                           ])
test_form_trans_mm_all_data.head(2)

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
0,0.066833,0.999731,0.759236,0.999546,0.690874,0.999346,0.993632,0.344237
1,0.066897,0.999734,0.759236,0.999546,0.691034,0.999345,0.993634,0.316282


In [ ]:
test_form_trans_std_all_data = pd.DataFrame(scaler_std.transform(test_form_trans_merge_mm), index=test_form_trans_merge_mm.index,
                                           columns=['GR_std', 'DTC_std', 'CALI_std', 'DRHO_std', 'RHOB_std', 
                                                    'NPHI_std', 'SP_std', 'PEF_std',
                                                    #'RMED_mm', 'RDEP_mm'
                                           ])
test_form_trans_mm_all_data.head(2)

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,NPHI_mm,SP_mm,PEF_mm
0,0.132925,0.549128,0.605961,0.417001,0.494092,0.260684,0.712444,0.153765
1,0.133065,0.556665,0.605961,0.416779,0.494466,0.258819,0.712777,0.117579


In [ ]:
imputer_test_rest = IterativeImputer(estimator=BayesianRidge(), imputation_order='ascending', max_iter=100, verbose=1, tol=0.001)

In [ ]:
test_form_trans_merge_mm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136786 entries, 0 to 136785
Data columns (total 14 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   GR_new    136786 non-null  float64
 1   DTC_new   136734 non-null  float64
 2   CALI_new  136786 non-null  float64
 3   DRHO_new  134399 non-null  float64
 4   RHOB_new  136786 non-null  float64
 5   RMED_new  136786 non-null  float64
 6   RDEP_new  136786 non-null  float64
 7   GR_mm     129712 non-null  float64
 8   DTC_mm    129712 non-null  float64
 9   CALI_mm   129712 non-null  float64
 10  DRHO_mm   129712 non-null  float64
 11  RHOB_mm   129712 non-null  float64
 12  RMED_mm   129712 non-null  float64
 13  RDEP_mm   129712 non-null  float64
dtypes: float64(14)
memory usage: 15.7 MB


In [ ]:
imputer_test_rest.fit(test_form_trans_merge_mm)

In [ ]:
test_form_trans_merge_mm_rest = imputer_test_rest.transform(test_form_trans_merge_mm)

[IterativeImputer] Completing matrix with shape (136786, 14)


In [ ]:
df_test_form_trans_merge_rest_final = pd.DataFrame(test_form_trans_merge_mm_rest, 
                                      columns=['GR_new', 'DTC_new', 'CALI_new', 'DRHO_new', 'RHOB_new', 'RMED_new', 'RDEP_new', 
                                               'GR_mm',  'DTC_mm',  'CALI_mm',  'DRHO_mm',  'RHOB_mm',  'RMED_mm',  'RDEP_mm'
                                               ], 
                                      index=df_test_form_trans_merge.index)
df_test_form_trans_merge_rest_final.head(2)

,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,RMED_new,RDEP_new,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,RMED_mm,RDEP_mm
0,72.078377,174.347092,19.2031,-0.538873,2.171173,1.613886,1.513937,0.230700,5.181914,2.242323,-2.283085,-0.364079,0.152654,-0.005164
1,72.147697,176.112915,19.2031,-0.539232,2.171840,1.574376,1.480188,0.231023,5.246846,2.242323,-2.264135,-0.363049,0.152572,-0.005209


In [ ]:
df_test_form_trans_merge_rest_final_mm = df_test_form_trans_merge_rest_final[['GR_mm',  'DTC_mm',  'CALI_mm',  'DRHO_mm',  
                                                                              'RHOB_mm',  'RMED_mm',  'RDEP_mm']]
df_test_form_trans_merge_rest_final_mm.head(2)                                                             

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,RMED_mm,RDEP_mm
0,0.230700,5.181914,2.242323,-2.283085,-0.364079,0.152654,-0.005164
1,0.231023,5.246846,2.242323,-2.264135,-0.363049,0.152572,-0.005209


In [ ]:
df_test_form_trans_merge_rest_final_scaled = scaler_mm.transform(df_test_form_trans_merge_rest_final)
df_test_form_trans_merge_rest_final_scaled

In [ ]:
df_test_form_trans_merge_rest_final_scaled = pd.DataFrame(df_test_form_trans_merge_rest_final_scaled, 
                                             columns=['GR_mm', 'DTC_mm', 'CALI_mm', 'DRHO_mm', 'RHOB_mm', 'RMED_mm', 'RDEP_mm'], 
                                             index=df_test_form_trans_merge_rest_final.index)
df_test_form_trans_merge_rest_final_scaled.head(2)

,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,RMED_mm,RDEP_mm
0,0.066833,0.998956,0.782579,0.999546,0.398789,0.384472,0.135914
1,0.066897,0.998968,0.782579,0.999546,0.398908,0.384460,0.135899


In [ ]:
df_test_merge_mm = df_test_form_trans_merge_rest_final.merge(df_test_form_trans_merge_rest_final_scaled,
                                                             left_index=True, right_index=True, how='inner')
df_test_merge_mm.head(2)

,GR_new,DTC_new,CALI_new,DRHO_new,RHOB_new,RMED_new,RDEP_new,GR_mm,DTC_mm,CALI_mm,DRHO_mm,RHOB_mm,RMED_mm,RDEP_mm
0,72.078377,174.347092,19.2031,-0.538873,2.171173,1.613886,1.513937,0.066833,0.998956,0.782579,0.999546,0.398789,0.384472,0.135914
1,72.147697,176.112915,19.2031,-0.539232,2.171840,1.574376,1.480188,0.066897,0.998968,0.782579,0.999546,0.398908,0.384460,0.135899


In [ ]:
df_test_form_trans_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136786 entries, 0 to 136785
Data columns (total 10 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   GR_new    136786 non-null  float64
 1   DTC_new   136786 non-null  float64
 2   CALI_new  136786 non-null  float64
 3   DRHO_new  136786 non-null  float64
 4   RHOB_new  136786 non-null  float64
 5   NPHI_new  136786 non-null  float64
 6   SP_new    136786 non-null  float64
 7   PEF_new   136786 non-null  float64
 8   RMED_new  136786 non-null  float64
 9   RDEP_new  136786 non-null  float64
dtypes: float64(10)
memory usage: 11.5 MB


In [ ]:
### Predict ###

In [ ]:
test_prediction = model_merge_mm.predict(test_form_trans_std_all_data)
test_prediction

array([2, 2, 2, ..., 1, 1, 1])

In [ ]:
pd.Series(test_prediction).value_counts()

2     91953
0     23993
1     10755
5      3607
3      2307
9      1719
10     1154
6      1085
4       125
8        82
7         5
11        1
dtype: int64

In [ ]:
pd.Series(test_prediction).value_counts()

2     92375
0     23806
1     11008
5      3620
3      1942
9      1738
10     1108
6      1000
4        95
8        90
11        2
7         2
dtype: int64

In [ ]:
pd.Series(test_prediction).value_counts()

2     92987
0     23805
1     10633
5      3805
3      1861
9      1727
10      900
6       894
8        91
4        81
7         2
dtype: int64

In [ ]:
pd.Series(test_prediction).value_counts()

2     92779
0     23895
1     10669
5      3688
3      1992
9      1733
10     1022
6       841
8        94
4        68
7         5
dtype: int64

In [ ]:
pd.Series(test_prediction).value_counts()

2     92589
0     23446
1     11635
5      4571
3      1690
9      1594
10      969
4       119
6        87
8        83
7         3
dtype: int64

In [ ]:
pd.Series(test_prediction).value_counts()

2     91164
0     23965
1     10912
5      4680
3      2979
9      1760
10     1004
6       150
4        92
8        80
dtype: int64

In [ ]:
pd.Series(test_prediction).value_counts()

2     91830
0     23482
1     10541
5      4691
3      3109
9      1714
10     1056
6       244
4        67
8        52
dtype: int64

In [ ]:
pd.Series(test_prediction).value_counts()

2     90892
0     23511
1     11870
5      4021
3      2988
9      1588
10     1003
6       775
4        73
8        65
dtype: int64

In [ ]:
category_to_lithology = {y:x for x,y in lithology_numbers.items()}
test_prediction_for_submission = np.vectorize(category_to_lithology.get)(test_prediction)
test_prediction_for_submission

array([65000, 65000, 65000, ..., 65030, 65030, 65030])

In [ ]:
np.savetxt('../Submission/test_predictions.csv', test_prediction_for_submission, header='lithology', fmt='%i')

In [ ]:
test_predictions = pd.read_csv('../Submission/test_predictions.csv')
test_predictions.columns = ['lithology']
test_predictions.head(2)

,lithology
0,65000
1,65000


In [ ]:
test_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136786 entries, 0 to 136785
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   lithology  136786 non-null  int64
dtypes: int64(1)
memory usage: 1.0 MB
